# Imports and Installs

In this notebook we will have all the plots relative to the work we did, in a format good for the dissertation and separated by sections for easy navigation and understanding

In [ ]:
%pip install qutip --upgrade
%pip install matplotlib --upgrade
# for progress bar
%pip install tqdm --upgrade
# for latex from function to use in plots
%pip install latexify-py --upgrade
# parallel process
%pip install loky --upgrade
# for running julia scripts inside python
#%pip install julia --upgrade
# see https://qutip-jax.readthedocs.io/en/latest/installation.html for installing qutip-jax 
# I recommend creating its own environment

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
import scipy as scp
from tqdm.auto import tqdm
from functools import partial
from IPython.display import display
from fractions import Fraction
# Strain term 
E_gs=0.0 #7.5 is an usual value
E_es=70.0
run_ks = True
run_opt= True

In [ ]:
mpl.rcParams

## Defining scale of lines in plots

In [ ]:
default_area=mpl.rcParams['figure.figsize'][0]*mpl.rcParams['figure.figsize'][1]
new_figsize=[14,8]
mpl.rcParams['figure.figsize'] = new_figsize
mpl.rcParams['lines.linewidth']=5
mpl.rcParams['xtick.minor.visible']=True
mpl.rcParams['ytick.minor.visible']=True
mpl.rcParams['axes.grid']=True
mpl.rc('xtick', labelsize=18)           # x-tick numbers only
mpl.rc('ytick', labelsize=18)           # y-tick numbers only
mpl.rcParams['figure.dpi']=300

# Normalizations

In [ ]:
def normaliz(arr): 
    """
    Normalize the input array to the range [0, 1].

    Parameters
    ----------
    arr : np.ndarray
        Input array to normalize.

    Returns
    -------
    np.ndarray
        Array with values scaled between 0 and 1.
    """
    return (arr - np.min(arr)) / (np.max(arr) - np.min(arr)).real

def stand(arr):
    """
    Standardize the input array to zero mean and unit variance.

    Parameters
    ----------
    arr : array-like
        Input array to standardize.

    Returns
    -------
    np.ndarray
        Standardized array with mean 0 and standard deviation 1.
    """
    return (arr - np.mean(arr)) / np.std(arr)

def max_reg(arr):
    """
    Normalize the input array by dividing by the maximum absolute value.

    Parameters
    ----------
    arr : np.ndarray
        Input array to normalize.

    Returns
    -------
    np.ndarray
        Normalized array with maximum absolute value scaled to 1.
    """
    return arr / np.max(np.abs(arr))

def l2_norm(arr):
    """
    Normalize the input array using the L2 norm.

    Parameters
    ----------
    arr : np.ndarray
        Input array to normalize.

    Returns
    -------
    np.ndarray
        Array normalized to unit L2 norm.
    """
    return arr / np.linalg.norm(arr)


# Fit functions

In [ ]:
def lorent(x,x0,gam,A,D):
    """Lorentzian function.
    Parameters:
    x (numpy.ndarray): Input array.
    x0 (float): Center of the Lorentzian peak.
    gam (float): Half-width at half-maximum (HWHM) of the Lorentzian peak.
    D (float): Offset value.
    Returns:
    numpy.ndarray: Lorentzian values.
    """
    return -A/(1+((x-x0)/gam)**2)*1/(gam*np.pi) + D
def gen_gauss(x,mu,alp,bet,A,D):
    """Generalized Gaussian function.
    Parameters:
    x (numpy.ndarray): Input array.
    mu (float): Mean of the distribution.
    alp (float): Scale parameter (related to the width).
    bet (float): Shape parameter (controls the "peakedness" of the distribution).
    D (float): Offset value.
    Returns:
    numpy.ndarray: Generalized Gaussian values.
    """
    return -A*np.exp(-np.abs((x-mu)/alp)**bet)*bet/(2*alp*np.sqrt(np.pi)*scp.special.gamma(1/bet)) + D
def damp_sin(x,A,omega,phi,tau, D):
    """
    Damped sine function.

    Parameters:
    x (numpy.ndarray): Input array.
    A (float): Amplitude.
    omega (float): Angular frequency.
    phi (float): Phase shift.
    tau (float): Damping time constant.
    D (float): Offset value.

    Returns:
    numpy.ndarray: Damped sine values.
    """
    return A * np.exp(-x/tau) * np.sin(omega*x + phi) + D
def damp_cos(x,A,omega,phi,tau, D):
    """
    Damped sine function.

    Parameters:
    x (numpy.ndarray): Input array.
    A (float): Amplitude.
    omega (float): Angular frequency.
    phi (float): Phase shift.
    tau (float): Damping time constant.
    D (float): Offset value.

    Returns:
    numpy.ndarray: Damped sine values.
    """
    return A * np.exp(-x/tau) * np.cos(omega*x + phi) + D
def expo(x,A, tau, D):
    """
    Exponential decay function.

    Parameters:
    x (numpy.ndarray): Input array.
    A (float): Amplitude.
    tau (float): Time constant.
    D (float): Offset value.

    Returns:
    numpy.ndarray: Exponential decay values.
    """
    return A * np.exp(-x/tau) + D
def sinn(x, x0, A, omega,phi, D):
    """
    Sin function.

    Parameters:
    x (numpy.ndarray): Input array.
    A (float): Amplitude.
    tau (float): Time constant.
    D (float): Offset value.

    Returns:
    numpy.ndarray: Exponential decay values.
    """
    return A * np.sin(omega*(x-x0) + phi) + D
x=np.linspace(-10,10,1000)
plt.figure(figsize=(14,8))
plt.plot(x,lorent(x,0,1,1,0),lw=5,label='Lorentzian (0,1)')
plt.plot(x,gen_gauss(x,0,1,2,1,0),lw=5,label='Generalized Gaussian (0,1,2)')
plt.xlabel("x",fontsize=25)
plt.ylabel(r"$f(x)$",fontsize=25)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
plt.show()

plt.figure(figsize=(14,8))
x=np.linspace(0,10,500)
plt.plot(x,damp_sin(x,1,4,0,5,0),lw=5,label='Damp Sin (1,4,0,5)')
plt.xlabel("x",fontsize=25)
plt.ylabel(r"$f(x)$",fontsize=25)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
plt.show()

Y = scp.fft.fftshift(scp.fft.fft(damp_sin(x,1,4,0,5,0)))                       
f = scp.fft.fftshift(scp.fft.fftfreq(x.shape[-1],d=x[1]-x[0])) 
floor = 1e-3 * f.max()
peaks, _ = scp.signal.find_peaks(np.abs(Y), height=floor)
first_peak_freq = f[peaks[0]]
last_peak_freq  = f[peaks[-1]]

plt.figure(figsize=(14,8))
plt.plot(f, np.abs(Y),lw=5,label='Damp Sin (1,4,0,5,0) FFT') 
plt.xlabel("Frequency",fontsize=25)
plt.ylabel(r"$|Y(f)|$",fontsize=25)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
margin = 0.5 * (last_peak_freq - first_peak_freq)
plt.xlim(first_peak_freq - margin, last_peak_freq + margin)
plt.show()

## How to fit functions using scipy

In [ ]:
# -------------------------------------------------
# 1.  Load or create your x, y, and (optionally) sigma (y-errors)
xdata = np.linspace(0, 10, 50)
ydata = 3.0 * np.exp(-0.5 * xdata) + 0.4   # synthetic exact model
noise = 0.05 * np.random.randn(xdata.size)
ydata += noise                             # noisy data
sigma = np.full_like(ydata, 0.05)          # same error bar for all pts

# -------------------------------------------------
# 2.  Define the model = f(x, *params)
def my_model(x, A, k, C):
    """Exponential decay plus offset:  y = A * exp(-k x) + C"""
    return A * np.exp(-k * x) + C

# -------------------------------------------------
# 3.  Provide initial guesses for parameters
p0 = [1.0, 1.0, 0.0]       # [A_guess, k_guess, C_guess]
def fit_plot_data(func,x_data,y_data,fit_params,fit_guess,ax=None,errors=None,xaxis_label='x',yaxis_label='y',label='label'):
    # -------------------------------------------------
    #  Curve-fit
    if len(fit_params) != len(fit_guess):
        raise ValueError("Length of fit_params must match length of fit_guess")
    else:
        if errors is None:
            popt, pcov = scp.optimize.curve_fit(func, x_data, y_data, p0=fit_guess,maxfev=20000)
        else:
            popt, pcov = scp.optimize.curve_fit(func, x_data, y_data, p0=fit_guess, sigma=errors, 
                                                absolute_sigma=True,maxfev=20000)

        perr = np.sqrt(np.diag(pcov))   # 1σ uncertainties
        fitted_par={}
        print("Fit parameters:")
        for name, val, err in zip(fit_params, popt, perr):
            fitted_par[name]=(val,err)
            print(f"  {name} = {val:.4g} ± {err:.2g}")
        full_label=f"{label} fit: \n" + ", \n".join([f"{name}={val:.4g}±{err:.2g}" for name, val, err in zip(fit_params, popt, perr)])
        # -------------------------------------------------
        #  Evaluate fitted curve and plot
        xfit = np.linspace(x_data.min(), x_data.max(), 500)
        yfit = func(xfit, *popt)
        if ax is None:
            fig, ax = plt.subplots(figsize=(14,8))
        
        ax.errorbar(x_data, y_data, yerr=errors, fmt='o', label=f'{label} data', lw=3,alpha=0.7)
        ax.plot(xfit, yfit, 'k--', label=f'{full_label}', lw=5)
        ax.set_xlabel(xaxis_label,fontsize=25)
        ax.set_ylabel(yaxis_label,fontsize=25)
        ax.minorticks_on()
        ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
        
        return ax,fitted_par
fig, ax = plt.subplots(figsize=(14,8))
fit_plot_data(my_model, xdata, ydata, ['A', 'k', 'C'],p0, ax=ax,errors=sigma,label="Abubu",yaxis_label='pinpum',xaxis_label='pompom')
plt.tight_layout()
plt.show()

# FFT

In [ ]:
def plot_fft(t, y, ax=None, label=None, unit='MHz'):
            """
            Plot the single-sided amplitude spectrum |Y(f)| of a real-valued signal y(t).

            Parameters
            ----------
            t : 1-D array
                Time axis [µs, ms, s …] - must be equally spaced.
            y : 1-D array
                Signal values at the same sampling points as `t`.
            ax : matplotlib Axes, optional
                If given, plot into this Axes; otherwise create a new figure.
            label : str, optional
                Legend label for this spectrum.
            unit : str
                Text for the x-axis (e.g. "MHz", "kHz", "Hz").
            """
            
            # ---- 2. FFT and frequency axis -----------------------------------------
            Y = scp.fft.fftshift(scp.fft.fft(y))                         # normalised DFT
            f = scp.fft.fftshift(scp.fft.fftfreq(t.shape[-1],d=t[1]-t[0]))                   # matching frequencies

            # Find peaks for closer look to data (need to be reformulated 
            # when ran with multiple data use same axis)
            
            #floor = 1e-3 * f.max().real
            #peaks, abu = scp.signal.find_peaks(np.abs(Y), height=floor)
            ##print(f"peaks:{peaks}")
            ##print(f"The other output form signal:{abu}")
            #first_peak_freq = f[peaks[0]]
            #last_peak_freq  = f[peaks[-1]]
            # ---- 3. plot ------------------------------------------------------------
            if ax is None:
                fig, ax = plt.subplots(figsize=(14, 8))
            ax.plot(f, np.abs(Y.real)+0.5, label=f"Re{{{label}}}",lw=5)          # factor 2 for single-sided
            ax.plot(f, -np.abs(Y.imag)-0.5, label=f"Im{{{label}}}",lw=5)          # factor 2 for single-sided
            ax.set_xlabel(f"Frequency [{unit}]",fontsize=25)
            ax.set_ylabel(r"$|\mathcal{F}(f)|$",fontsize=25)
            ax.set_title("Amplitude spectrum (FFT)",fontsize=25)
            margin = 5
            ax.set_xlim(- margin, margin)
            ax.minorticks_on()
            ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
            return ax

# Compartive tables function

In [ ]:
def compare_regions(data1, data2, regions, labels=["A", "B", "C"]):
    """
    Compare two datasets over given index regions.
    
    Parameters:
        data1, data2 : array-like
            The datasets to compare (must be same length).
        regions : list of tuples
            Each tuple (start, end) defines a region.
        labels : list of str
            Labels for the regions (must match len(regions)).
    
    Returns:
        summary : list of dict
            Each dict contains region label, mean diff, std dev.
    """
    summary = []

    for (start, end), label in zip(regions, labels):
        seg1 = np.array(data1[start:end])
        seg2 = np.array(data2[start:end])
        diff = seg1 - seg2
        mean_diff = np.mean(np.abs(diff))
        std_diff = np.std(diff)
        rmse = np.sqrt(np.mean(diff**2))
        summary.append({
            "region": label,
            "mean_abs_diff": mean_diff,
            "std_dev_diff": std_diff,
            "rmse": rmse,
            "max_abs_diff": np.max(np.abs(diff)),
            "min_abs_diff": np.min(np.abs(diff))
            
        })

    return summary
# To print as a simple table:
def print_summary_table(summary):
    print(f"{'Region':<8} {'Mean Abs Diff':<15} {'Std Dev Diff':<15} {'RMSE':<15} {'Max Diff':<15} {'Min Diff':<15}")
    for entry in summary:
        print(f"{entry['region']:<8} {entry['mean_abs_diff']:<15.6f} {entry['std_dev_diff']:<15.6f} {entry['rmse']:<15.6f} {entry['max_abs_diff']:<15.6f} {entry['min_abs_diff']:<15.6f}")

# Hamiltonians

In [ ]:
def B0(b_0, phi, theta):
    """
    Calculate the magnetic field vector in Cartesian coordinates.
    Parameters:
    - b_0 (float): The magnitude of the magnetic field.
    - theta (float): The azimuthal angle in radians.
    - phi (float): The polar angle in radians.
    Returns:
    - np.array: The magnetic field vector in Cartesian coordinates np.array([Bx, By, Bz]).
    """
    return np.array([b_0 * np.sin(theta) * np.cos(phi), b_0 * np.sin(theta) * np.sin(phi), b_0 * np.cos(theta)])

def construct_spin_matrices(basis):
    """
    Constructs the spin matrices for a given basis.
    Parameters:
    - basis (list): A list of basis elements.
    Returns:
    - s_x (Qobj): The x-component of the spin matrix.
    - s_y (Qobj): The y-component of the spin matrix.
    - s_z (Qobj): The z-component of the spin matrix.
    Raises:
    - ValueError: If the number of basis elements is not 3.
    """
    if len(basis) == 3:
        # 3-level system (spin-1)
        s_x = (basis[0] * basis[1].dag() + basis[1] * basis[0].dag() +
               basis[1] * basis[2].dag() + basis[2] * basis[1].dag()) / np.sqrt(2)
        
        s_y = -1j * (basis[0] * basis[1].dag() - basis[1] * basis[0].dag() +
                     basis[1] * basis[2].dag() - basis[2] * basis[1].dag()) / np.sqrt(2)
        
        s_z = basis[0] * basis[0].dag() - basis[2] * basis[2].dag()
        return s_x, s_y, s_z

    else:
        raise ValueError("This function supports only 3 basis elements.")

## Magaletti

In [ ]:
# Dimention of the Hilbert space
dim = 7
k_ind=2
# dephasing times(microseconds)
t1_gs=1e3
t2_gs=1.5
t1_es=1e3
t2_es=6*1e-3
# Constants (MHz)
gamma_gs = 1/t1_gs,1/t2_gs
gamma_es = 1/t1_es,1/t2_es
W_p = 1.9
# I had to modify this two value to repruduce the results of the paper
Om_r = 15.7
omega = 0.1

# States (I broke them in a few lines to be more readable)
excited = qt.basis(dim, 4), qt.basis(dim, 3),qt.basis(dim, 5) # |+1>_es, |0>_es, |-1>_es
isc = qt.basis(dim, 6)
ground = qt.basis(dim, 1), qt.basis(dim, 0), qt.basis(dim, 2) # |+1>_gs, |0>_gs, |-1>_gs
IdNV=qt.qeye(dim)

n1, n2, n3 = (
    ground[1] * ground[1].dag(), #type: ignore
    ground[2] * ground[2].dag(), #type: ignore
    ground[0] * ground[0].dag(), #type: ignore
)
n4, n5, n6 = (
    excited[1] * excited[1].dag(), #type: ignore
    excited[2] * excited[2].dag(), #type: ignore
    excited[0] * excited[0].dag(), #type: ignore
)
n7, nc = (
    isc * isc.dag(), #type: ignore
    ground[2] * ground[1].dag(), #type: ignore
)

sx_gs, sy_gs, sz_gs = construct_spin_matrices(ground)
sx_es, sy_es, sz_es = construct_spin_matrices(excited)
sm_gs,sp_gs= sx_gs - 1j * sy_gs, sx_gs + 1j * sy_gs
sm_es,sp_es= sx_es - 1j * sy_es, sx_es + 1j * sy_es

S_gs=np.array([sx_gs, sy_gs, sz_gs])
S_es=np.array([sx_es, sy_es, sz_es])

def H_mg(om_r):
    """Returns the Hamiltonian of the system based on whether the MW is on or off
    Parameters:
        om_r (float) - Rabi frequency

    Returns:
        H_0 (list) - list of the Hamiltonian terms and their time dependence
    """
    H_0 = [[0.5 * om_r * (ground[1]*ground[2].dag()), "exp(1j*w*t)"],  #type: ignore
           [0.5 * om_r * (ground[2]*ground[1].dag()), "exp(-1j*w*t)"]] #type: ignore
    return H_0


def L_mg(w_p,k_index=k_ind):
    """Returns the Lindblad operators of the system.
    
    Parameters:
        w_p (float) - Laser pump rate
        
    Returns:
        c_ops (list) - list of the Lindblad operators
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055],
         [64.0,11.8,79.8,5.6,0.0]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append(np.sqrt(w_p) * (excited[1] * ground[1].dag()))  # n1 to n4 #type: ignore 
    c_ops.append(np.sqrt(w_p) * (excited[2] * ground[2].dag()))  # n2 to n5 #type: ignore
    c_ops.append(np.sqrt(w_p) * (excited[0] * ground[0].dag())) # n3 to n6  #type: ignore

    c_ops.append(np.sqrt(k41) * (ground[1] * excited[1].dag()))  # n4 to n1 #type: ignore
    c_ops.append(np.sqrt(k71) * (ground[1] * isc.dag()))  # n7 to n1    #type: ignore

    c_ops.append(np.sqrt(k52) * (ground[2] * excited[2].dag()))  # n5 to n2 #type: ignore
    c_ops.append(np.sqrt(k72) * (ground[2] * isc.dag()))  # n7 to n2 #type: ignore

    c_ops.append(np.sqrt(k63) * (ground[0] * excited[0].dag()))  # n6 to n3 #type: ignore
    c_ops.append(np.sqrt(k73) * (ground[0] * isc.dag()))  # n7 to n3    #type: ignore

    c_ops.append(np.sqrt(k47) * (isc * excited[1].dag()))  # n4 to n7   #type: ignore
    c_ops.append(np.sqrt(k57) * (isc * excited[2].dag()))  # n5 to n7   #type: ignore
    c_ops.append(np.sqrt(k67) * (isc * excited[0].dag()))  # n6 to n7   #type: ignore
    
    # Add collapse operators for decoherence
    c_ops.append(np.sqrt(gamma_gs[1]) * sz_gs)
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sm_gs))
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sp_gs))
    c_ops.append(np.sqrt(gamma_es[1]) * sz_es)
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sm_es))
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sp_es))
    return c_ops

def dynamics_mg(
    dt,
    init_state,
    om=None,
    om_r=None,
    w_p=None,
    k_index=k_ind,
    ti=0.0,    
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Perform dynamics_mg simulation based on the given parameters, including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Time step for the calculations.
    - init_state: Initial state for the simulation.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - om_r (float, optional): Angular frequency for MW-ON evolution. Defaults to Om_r.
    - w_p (float, optional): Frequency for laser-ON evolution. Defaults to W_p.
    - k_index=k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.
    - exp_ops (list, optional): List of expectation operators. Defaults to 
        [n1, n2, n3, n4, n5, n6, n7, nc, 
        sx_gs, sy_gs, sz_gs, 
        sx_es, sy_es, sz_es].
    - ti (float, optional): Initial time for the simulation. Defaults to 0.0.
    - mode (str, optional): Mode of the simulation. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Progress bar option. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Iteration number. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result: Result of the simulation.
    """
    # Default values
    if om is None: om = omega
    if om_r is None: om_r = Om_r
    if w_p is None: w_p = W_p

    # Arguments for the Hamiltonian
    args = {"w": om}
    
    # Define the time resolution
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt

    # Define collapse operators and Hamiltonian based on mode
    match mode:
        case "Free":
            c_ops = L_mg(0.0, k_index=k_index)
            H = H_mg(0.0)
        case "MW":
            c_ops = L_mg(0.0, k_index=k_index)
            H = H_mg(om_r)
        case "Laser":
            c_ops = L_mg(w_p, k_index=k_index)
            H = H_mg(0.0)
        case "Laser-MW":
            c_ops = L_mg(w_p, k_index=k_index)
            H = H_mg(om_r)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')
    
    # Call the master equation solver
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti | tf \n {int(ti)} | {int(tf)}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')
    
    return tf, result

#### HF

In [ ]:
#Dimention of the Hilbert space
dim = 7

t1_n=1e5
t2_n=1e3
# Constants (MHz)
gamma_n=1/t1_n,1/t2_n
a_gs=3.03,3.65
a_es=-57.8,-39.2
mu_n = 431.7*1e-6 # (MHz/G)
mu_e = 2.8 # (Mhz/G)

# Nitrogen Nucleous
IdN15 = qt.qeye(2)
nit = qt.basis(2, 0), qt.basis(2, 1)
sx_n, sy_n, sz_n = qt.sigmax()*0.5, qt.sigmay()*0.5, qt.sigmaz()*0.5
sm_n,sp_n=sx_n-1j*sy_n,sx_n+1j*sy_n
S_n=np.array([sx_n,sy_n,sz_n])

def H_mg_hf(om_r,a_gs=a_gs,a_es=a_es):
    """Returns the Hamiltonian of the system based on whether the MW is on or off
    Parameters:
        om_r (float) - Rabi frequency

    Returns:
        H_0 (list) - list of the Hamiltonian terms and their time dependence
    """
    H_0 = [[(0.5 * om_r * (ground[1]*ground[2].dag()))&IdN15, "exp(1j*w*t)"],  #type: ignore
           [(0.5 * om_r * (ground[2]*ground[1].dag()))&IdN15, "exp(-1j*w*t)"], #type: ignore
           a_gs[0]*(sz_gs&sz_n) + a_gs[1]*((sx_gs&sx_n) + (sy_gs&sy_n)),
           a_es[0]*(sz_es&sz_n) + a_es[1]*((sx_es&sx_n) + (sy_es&sy_n))]
    H_n=[[IdNV&(0.5*om_r*(mu_n/mu_e)*(nit[0]*nit[1].dag())),"exp(1j*w*t)"],   #type: ignore
          [IdNV&(0.5*om_r*(mu_n/mu_e)*(nit[1]*nit[0].dag())),"exp(-1j*w*t)"]] #type: ignore
    return [*H_0 , *H_n]

def L_mg_hf(w_p,k_index=k_ind):
    """Returns the Lindblad operators of the system
    Parameters:
        w_p (float) - Laser pump rate
    Returns:
        c_ops (list) - list of the Lindblad operators
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append((np.sqrt(w_p) * (excited[1] * ground[1].dag()))&IdN15)  # n1 to n4 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[2] * ground[2].dag()))&IdN15)  # n2 to n5 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[0] * ground[0].dag()))&IdN15) # n3 to n6 #type: ignore

    c_ops.append((np.sqrt(k41) * (ground[1] * excited[1].dag()))&IdN15)  # n4 to n1 #type: ignore
    c_ops.append((np.sqrt(k71) * (ground[1] * isc.dag()))&IdN15)  # n7 to n1 #type: ignore

    c_ops.append((np.sqrt(k52) * (ground[2] * excited[2].dag()))&IdN15)  # n5 to n2 #type: ignore
    c_ops.append((np.sqrt(k72) * (ground[2] * isc.dag()))&IdN15)  # n7 to n2 #type: ignore

    c_ops.append((np.sqrt(k63) * (ground[0] * excited[0].dag()))&IdN15)  # n6 to n3 #type: ignore
    c_ops.append((np.sqrt(k73) * (ground[0] * isc.dag()))&IdN15)  # n7 to n3 #type: ignore

    c_ops.append((np.sqrt(k47) * (isc * excited[1].dag()))&IdN15)  # n4 to n7 #type: ignore
    c_ops.append((np.sqrt(k57) * (isc * excited[2].dag()))&IdN15)  # n5 to n7 #type: ignore
    c_ops.append((np.sqrt(k67) * (isc * excited[0].dag()))&IdN15)  # n6 to n7 #type: ignore
    # Add collapse operators for decoherence   
    c_ops.append((np.sqrt(gamma_gs[1]) * sz_gs)&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sm_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sp_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_es[1]) * sz_es)&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sm_es))&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sp_es))&IdN15)
    c_ops.append(IdNV&(np.sqrt(gamma_n[1]) * sz_n))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sm_n)))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sp_n)))

    return c_ops

def dynamics_mg_hf(
    dt,
    init_state,
    om_r=None,
    om=None,
    w_p=None,
    k_index=k_ind,
    ti=0.0,
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Simulate the dynamics of a quantum system under hyperfine interaction using the Hamiltonian and collapse operators.
    including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Total simulation time.
    - init_state (qutip.Qobj): Initial quantum state of the system.
    - om_r (float, optional): Rabi frequency for microwave interactions. Defaults to Om_r.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - w_p (float, optional): Laser frequency. Defaults to W_p.
    - k_index(int, optional): Index for the optical transition rates. Defaults to k_ind.
    - exp_ops (list of qutip.Qobj, optional): List of operators for which to compute expectation values. 
        Defaults to a list of tensor products including different Pauli operators and state projectors.
    - ti (float, optional): Initial time of the simulation. Defaults to 0.0.
    - mode (str, optional): Simulation mode. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Option to display a progress bar. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Counter for the progress bar. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result (qutip.solver.Result): Result object containing the simulation output.
    """
    # Default values
    if om_r is None: om_r = Om_r
    if om is None: om = omega
    if w_p is None: w_p = W_p

    # Time resolution based on dt
    t_bins = 1000 if dt <= 5 else 5000
    
    # Define Hamiltonian and collapse operators based on mode
    match mode:
        case "Free":
            H = H_mg_hf(0.0)
            c_ops = L_mg_hf(0.0, k_index=k_index)
        case "MW":
            H = H_mg_hf(om_r)
            c_ops = L_mg_hf(0.0, k_index=k_index)
        case "Laser":
            H = H_mg_hf(0.0)
            c_ops = L_mg_hf(w_p, k_index=k_index)
        case "Laser-MW":
            H = H_mg_hf(om_r)
            c_ops = L_mg_hf(w_p, k_index=k_index)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')

    # Arguments for the Hamiltonian
    args = {"w": om}
    
    tf = ti + dt

    # Solve the master equation
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti | tf \n {int(ti)} | {int(tf)}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')

    return tf, result

## No - HF

In [ ]:
# Dimention of the Hilbert space
dim = 7
# Optical transition rates index
k_ind=2
# dephasing times(microseconds)
t1_gs=1e3
t2_gs=1.5
t1_es=1e3
t2_es=6*1e-3
    
mu_e = 2.8 # (Mhz/G)
B = B0(100.0,0.0,0.0) # (G)
# Constants (MHz)
D_gs = 2870.0
D_es = 1420.0
W_p = 1.9
gamma_gs = 1/t1_gs,1/t2_gs
gamma_es = 1/t1_es,1/t2_es

# Microwave frequency
omega = D_gs-mu_e*B[2]
# Microwave phase
phi = 0.0
# I had to modify this two value to repruduce the results of the paper
# Microwave magnectic field*mu_e 
Om_r = 15.7

# States (I broke them in a few lines to be more readable)
excited = qt.basis(dim, 4), qt.basis(dim, 3),qt.basis(dim, 5) # |+1>_es, |0>_es, |-1>_es
isc = qt.basis(dim, 6)
ground = qt.basis(dim, 1), qt.basis(dim, 0), qt.basis(dim, 2) # |+1>_gs, |0>_gs, |-1>_gs
IdNV=qt.qeye(dim)

n1, n2, n3 = (
    ground[1] * ground[1].dag(),#type: ignore
    ground[2] * ground[2].dag(),#type: ignore
    ground[0] * ground[0].dag(),#type: ignore
)
n4, n5, n6 = (
    excited[1] * excited[1].dag(),#type: ignore
    excited[2] * excited[2].dag(),#type: ignore
    excited[0] * excited[0].dag(),#type: ignore
)
n7, nc = (
    isc * isc.dag(),#type: ignore
    ground[2] * ground[1].dag(),#type: ignore
)

sx_gs, sy_gs, sz_gs = construct_spin_matrices(ground)
sx_es, sy_es, sz_es = construct_spin_matrices(excited)
sm_gs,sp_gs= sx_gs - 1j * sy_gs, sx_gs + 1j * sy_gs
sm_es,sp_es= sx_es - 1j * sy_es, sx_es + 1j * sy_es

S_gs=np.array([sx_gs, sy_gs, sz_gs])
S_es=np.array([sx_es, sy_es, sz_es])


def H_no(b, om_r):
    """
    Calculates the Hamiltonian for a given magnetic field and Rabi frequency.
    
    Parameters:
    b (np.array): Magnetic field vector B0(B_amp,phi_B,theta_B).
    om_r (float): Rabi frequency.
    
    Returns:
    list: A list containing the Hamiltonian Qobj terms.
    """
    H_0 = [D_gs*sz_gs**2+E_gs*(sx_gs**2-sy_gs**2)+mu_e*np.dot(b,S_gs)+
           D_es*sz_es**2+E_es*(sx_es**2-sy_es**2)+mu_e*np.dot(b,S_es)]
    H_int = [[np.sqrt(2)*om_r*(sx_gs), "cos(w*t)*cos(p)"],
           [np.sqrt(2)*om_r*(sy_gs), "cos(w*t)*sin(p)"],
           [np.sqrt(2)*om_r*(sx_es), "cos(w*t)*cos(p)"],
           [np.sqrt(2)*om_r*(sy_es), "cos(w*t)*sin(p)"]]
    return [*H_0,*H_int]

def L_no(w_p,k_index=k_ind):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.

    Returns:
    - c_ops (list): List of Lindblad operators Qobj.
    """
    #optical transitions
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055],
         [64.0,11.8,79.8,5.6,0.0]]#Hirose
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append(np.sqrt(w_p) * (excited[1] * ground[1].dag()))  # n1 to n4 #type: ignore
    c_ops.append(np.sqrt(w_p) * (excited[2] * ground[2].dag()))  # n2 to n5 #type: ignore
    c_ops.append(np.sqrt(w_p) * (excited[0] * ground[0].dag())) # n3 to n6 #type: ignore

    c_ops.append(np.sqrt(k41) * (ground[1] * excited[1].dag()))  # n4 to n1 #type: ignore
    c_ops.append(np.sqrt(k71) * (ground[1] * isc.dag()))  # n7 to n1 #type: ignore

    c_ops.append(np.sqrt(k52) * (ground[2] * excited[2].dag()))  # n5 to n2 #type: ignore
    c_ops.append(np.sqrt(k72) * (ground[2] * isc.dag()))  # n7 to n2 #type: ignore

    c_ops.append(np.sqrt(k63) * (ground[0] * excited[0].dag()))  # n6 to n3 #type: ignore
    c_ops.append(np.sqrt(k73) * (ground[0] * isc.dag()))  # n7 to n3 #type: ignore

    c_ops.append(np.sqrt(k47) * (isc * excited[1].dag()))  # n4 to n7 #type: ignore
    c_ops.append(np.sqrt(k57) * (isc * excited[2].dag()))  # n5 to n7 #type: ignore
    c_ops.append(np.sqrt(k67) * (isc * excited[0].dag()))  # n6 to n7 #type: ignore
    # Add collapse operators for decoherence
    c_ops.append(np.sqrt(gamma_gs[1]) * sz_gs)
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sm_gs))
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sp_gs))
    c_ops.append(np.sqrt(gamma_es[1]) * sz_es)
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sm_es))
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sp_es))
    return c_ops


def dynamics_no(
    dt,
    init_state,
    b=None,
    om=None,
    p=None,
    om_r=None,
    w_p=None,
    k_index=k_ind,
    ti=0.0,    
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Perform dynamics simulation based on the given parameters, including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Time step for the calculations.
    - init_state: Initial state for the simulation.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - om_r (float, optional): Angular frequency for MW-ON evolution. Defaults to Om_r.
    - w_p (float, optional): Frequency for laser-ON evolution. Defaults to W_p.
    - k_index=k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.
    - ti (float, optional): Initial time for the simulation. Defaults to 0.0.
    - mode (str, optional): Mode of the simulation. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Progress bar option. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Iteration number. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result: Result of the simulation.
    """
    # Default values
    if b is None: b = B
    if om is None: om = omega
    if p is None: p = phi
    if om_r is None: om_r = Om_r
    if w_p is None: w_p = W_p
    

    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    
    # Define the time resolution   
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt

    # Define collapse operators and Hamiltonian based on mode
    match mode:
        case "Free":
            c_ops = L_no(0.0, k_index=k_index)
            H = H_no(b, 0.0)
        case "MW":
            c_ops = L_no(0.0, k_index=k_index)
            H = H_no(b, om_r)
        case "Laser":
            c_ops = L_no(w_p, k_index=k_index)
            H = H_no(b, 0.0)
        case "Laser-MW":
            c_ops = L_no(w_p, k_index=k_index)
            H = H_no(b, om_r)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')

    # Call the master equation solver
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti    |    tf \n {ti:.2f} | {tf:.2f}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')
    
    return tf, result

## Model 3

In [ ]:
# Dimention of the Hilbert space
dim = 7

t1_n=1e5
t2_n=1e3
# Constants (MHz)
gamma_n=1/t1_n,1/t2_n
mu_n = 431.7*1e-6 # (MHz/G)

a_gs=3.03,3.65
a_es=-57.8,-39.2

# Nitrogen Nucleous
IdN15 = qt.qeye(2)
nit = qt.basis(2, 0), qt.basis(2, 1) # |-1/2>_n, |+1/2>_n
sx_n, sy_n, sz_n = qt.sigmax()*0.5, qt.sigmay()*0.5, qt.sigmaz()*0.5
sm_n,sp_n=sx_n-1j*sy_n,sx_n+1j*sy_n
S_n=np.array([sx_n,sy_n,sz_n])

def H_doh_hf(b, om_r,a_gs=a_gs,a_es=a_es):
    """
    Calculate the Hamiltonian for a given magnetic field and resonant frequency.
    
    Parameters:
    b (numpy.ndarray): Magnetic field vector [bx,by,bz].
    om_r: Resonant frequency.
    
    Returns:
    list: List of Hamiltonian terms.
    """
    
    H_0 = [((D_gs*sz_gs**2+E_gs*(sx_gs**2-sy_gs**2)+mu_e*np.dot(b,S_gs))&IdN15)+
           ((D_es*sz_es**2+E_es*(sx_es**2-sy_es**2)+mu_e*np.dot(b,S_es))&IdN15)+
           a_gs[0]*(sz_gs&sz_n) + a_gs[1]*((sx_gs&sx_n) + (sy_gs&sy_n))+
           a_es[0]*(sz_es&sz_n) + a_es[1]*((sx_es&sx_n) + (sy_es&sy_n))]
    H_n = [IdNV&(mu_n*np.dot(b,S_n))]
    H_int=[[(np.sqrt(2)*om_r*sx_gs)&IdN15, "cos(w*t)*cos(p)"],
           [(np.sqrt(2)*om_r*sy_gs)&IdN15, "cos(w*t)*sin(p)"],
           [(np.sqrt(2)*om_r*sx_es)&IdN15, "cos(w*t)*cos(p)"],
           [(np.sqrt(2)*om_r*sy_es)&IdN15, "cos(w*t)*sin(p)"],
           [IdNV&(2*om_r/mu_e*mu_n*sx_n), "cos(w*t)*cos(p)"],
           [IdNV&(2*om_r/mu_e*mu_n*sy_n), "cos(w*t)*sin(p)"]]
    return [*H_0,*H_n,*H_int] 

def L_doh_hf(w_p,k_index=k_ind):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.

    Returns:
    - c_ops (list): List of Lindblad operators.
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055],
         [64.0,11.8,79.8,5.6,0.0]]#Hirose
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append((np.sqrt(w_p) * (excited[1] * ground[1].dag()))&IdN15)  # n1 to n4 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[2] * ground[2].dag()))&IdN15)  # n2 to n5 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[0] * ground[0].dag()))&IdN15) # n3 to n6 #type: ignore

    c_ops.append((np.sqrt(k41) * (ground[1] * excited[1].dag()))&IdN15)  # n4 to n1 #type: ignore
    c_ops.append((np.sqrt(k71) * (ground[1] * isc.dag()))&IdN15)  # n7 to n1 #type: ignore

    c_ops.append((np.sqrt(k52) * (ground[2] * excited[2].dag()))&IdN15)  # n5 to n2 #type: ignore
    c_ops.append((np.sqrt(k72) * (ground[2] * isc.dag()))&IdN15)  # n7 to n2 #type: ignore

    c_ops.append((np.sqrt(k63) * (ground[0] * excited[0].dag()))&IdN15)  # n6 to n3 #type: ignore
    c_ops.append((np.sqrt(k73) * (ground[0] * isc.dag()))&IdN15)  # n7 to n3 #type: ignore

    c_ops.append((np.sqrt(k47) * (isc * excited[1].dag()))&IdN15)  # n4 to n7 #type: ignore
    c_ops.append((np.sqrt(k57) * (isc * excited[2].dag()))&IdN15)  # n5 to n7 #type: ignore
    c_ops.append((np.sqrt(k67) * (isc * excited[0].dag()))&IdN15)  # n6 to n7 #type: ignore
    # Collapse operators for decoherence   
    c_ops.append((np.sqrt(gamma_gs[1]) * sz_gs)&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sm_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sp_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_es[1]) * sz_es)&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sm_es))&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sp_es))&IdN15)
    c_ops.append(IdNV&(np.sqrt(gamma_n[1]) * sz_n))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sm_n)))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sp_n)))
    return c_ops

def dynamics_doh_hf(
    dt,
    init_state,
    b=None,
    om_r=None,
    om=None,
    p=None,
    w_p=None,
    k_index=k_ind,
    ti=0.0,
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Simulate the dynamics of a quantum system under hyperfine interaction using the Hamiltonian and collapse operators.
    including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Total simulation time.
    - init_state (qt.Qobj): Initial quantum state of the system.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om_r (float, optional): Rabi frequency for microwave interactions. Defaults to Om_r.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - p (float, optional): Microwave phase. Defaults to 0.0.
    - w_p (float, optional): Laser frequency. Defaults to W_p.
    - k_index(int, optional): Index for the optical transition rates. Defaults to k_ind.
    - exp_ops (list of qt.Qobj, optional): List of operators for which to compute expectation values. 
        Defaults to a list of tensor products including different Pauli operators and state projectors.
    - ti (float, optional): Initial time of the simulation. Defaults to 0.0.
    - mode (str, optional): Simulation mode. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Option to display a progress bar. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Counter for the progress bar. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result (qutip.solver.Result): Result object containing the simulation output.
    """
    # Default values
    if b is None: b = B
    if om_r is None: om_r = Om_r
    if om is None: om = omega
    if p is None: p = phi
    if w_p is None: w_p = W_p
    
    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    # Time resolution based on dt
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt
    
    # Define Hamiltonian and collapse operators based on mode
    match mode:
        case "Free":
            H = H_doh_hf(b, 0.0)
            c_ops = L_doh_hf(0.0, k_index=k_index)
        case "MW":
            H = H_doh_hf(b, om_r)
            c_ops = L_doh_hf(0.0, k_index=k_index)
        case "Laser":
            H = H_doh_hf(b, 0.0)
            c_ops = L_doh_hf(w_p, k_index=k_index)
        case "Laser-MW":
            H = H_doh_hf(b, om_r)
            c_ops = L_doh_hf(w_p, k_index=k_index)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')

    # Solve the master equation
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti    |    tf \n {ti:.2f} | {tf:.2f}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')

    return tf, result

## Model 2

In [ ]:
# Dimention of the Hilbert space of the NV center
dim = 7

t1_n=1e5
t2_n=1e3
# Constants (MHz)
gamma_n=1/t1_n,1/t2_n
mu_n = 431.7*1e-6 # (MHz/G)

A_par = 3.4,-58.1
A_perp = 7.8,-77.0
A_ani = 0.0, 0.0
A_perp_prime = 0.0,0.0
phi_H = 0.0

def H_dua_hf(b, om_r, phi_h):
    """
    Calculate the Hamiltonian for a given magnetic field and resonant frequency, including hyperfine interactions.
    
    Parameters:
    b (numpy.ndarray): Magnetic field vector [bx, by, bz].
    om_r: Resonant frequency.
    
    Returns:
    list: List of Hamiltonian terms.
    """
    H_0 = [((D_gs*sz_gs**2 + E_gs*(sx_gs**2-sy_gs**2) + mu_e*np.dot(b, S_gs))&IdN15)+
           ((D_es*sz_es**2 + E_es*(sx_es**2-sy_es**2) + mu_e*np.dot(b, S_es))&IdN15)+
           A_par[0]*(sz_gs&sz_n) + A_perp[0]/4*((sp_gs&sm_n)+(sm_gs&sp_n))+
           A_perp_prime[0]/4*((sp_gs&sp_n)*np.exp(-2j*phi_h) + (sm_gs&sm_n)*np.exp(2j*phi_h))+
           A_ani[0]/2*((sp_gs&sz_n) + (sz_gs&sp_n)*np.exp(-1j*phi_h) + (sm_gs&sz_n) + (sz_gs&sm_n)*np.exp(1j*phi_h))+
           A_par[1]*(sz_es&sz_n) + A_perp[1]/4*((sp_es&sm_n)+(sm_es&sp_n))+
           A_perp_prime[1]/4*((sp_es&sp_n)*np.exp(-2j*phi_h) + (sm_es&sm_n)*np.exp(2j*phi_h))+
           A_ani[1]/2*((sp_es&sz_n) + (sz_es&sp_n)*np.exp(-1j*phi_h) + (sm_es&sz_n) + (sz_es&sm_n)*np.exp(1j*phi_h))]
    H_n = [IdNV&mu_n*np.dot(b, S_n)]
    H_int = [[(np.sqrt(2)*om_r*sx_gs)&IdN15, "cos(w*t)*cos(p)"],
             [(np.sqrt(2)*om_r*sy_gs)&IdN15, "cos(w*t)*sin(p)"],
             [(np.sqrt(2)*om_r*sx_es)&IdN15, "cos(w*t)*cos(p)"],
             [(np.sqrt(2)*om_r*sy_es)&IdN15, "cos(w*t)*sin(p)"],
             [IdNV&(2*om_r/mu_e*mu_n*sx_n), "cos(w*t)*cos(p)"],
             [IdNV&(2*om_r/mu_e*mu_n*sy_n), "cos(w*t)*sin(p)"]]
    return [*H_0, *H_n, *H_int]

def L_dua_hf(w_p,k_index=k_ind):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.

    Returns:
    - c_ops (list): List of Lindblad operators.
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055],
         [64.0,11.8,79.8,5.6,0.0]]#Hirose
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append((np.sqrt(w_p) * (excited[1] * ground[1].dag()))&IdN15)  # n1 to n4 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[2] * ground[2].dag()))&IdN15)  # n2 to n5 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[0] * ground[0].dag()))&IdN15) # n3 to n6 #type: ignore

    c_ops.append((np.sqrt(k41) * (ground[1] * excited[1].dag()))&IdN15)  # n4 to n1 #type: ignore
    c_ops.append((np.sqrt(k71) * (ground[1] * isc.dag()))&IdN15)  # n7 to n1 #type: ignore

    c_ops.append((np.sqrt(k52) * (ground[2] * excited[2].dag()))&IdN15)  # n5 to n2 #type: ignore
    c_ops.append((np.sqrt(k72) * (ground[2] * isc.dag()))&IdN15)  # n7 to n2 #type: ignore

    c_ops.append((np.sqrt(k63) * (ground[0] * excited[0].dag()))&IdN15)  # n6 to n3 #type: ignore
    c_ops.append((np.sqrt(k73) * (ground[0] * isc.dag()))&IdN15)  # n7 to n3 #type: ignore

    c_ops.append((np.sqrt(k47) * (isc * excited[1].dag()))&IdN15)  # n4 to n7 #type: ignore
    c_ops.append((np.sqrt(k57) * (isc * excited[2].dag()))&IdN15)  # n5 to n7 #type: ignore
    c_ops.append((np.sqrt(k67) * (isc * excited[0].dag()))&IdN15)  # n6 to n7 #type: ignore
    # Add collapse operators for decoherence   
    c_ops.append((np.sqrt(gamma_gs[1]) * sz_gs)&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sm_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sp_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_es[1]) * sz_es)&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sm_es))&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sp_es))&IdN15)
    c_ops.append(IdNV&(np.sqrt(gamma_n[1]) * sz_n))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sm_n)))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sp_n)))
    return c_ops

def dynamics_dua_hf(
    dt,
    init_state,
    b=None,
    om_r=None,
    om=None,
    p=None,
    w_p=None,
    k_index=k_ind,
    ti=0.0,
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Simulate the dynamics of a quantum system under hyperfine interaction using the Hamiltonian and collapse operators.
    including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Total simulation time.
    - init_state (qt.Qobj): Initial quantum state of the system.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om_r (float, optional): Rabi frequency for microwave interactions. Defaults to Om_r.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - p (float, optional): Microwave phase. Defaults to 0.0.
    - w_p (float, optional): Laser frequency. Defaults to W_p.
    - k_index(int, optional): Index for the optical transition rates. Defaults to k_ind.
    - ti (float, optional): Initial time of the simulation. Defaults to 0.0.
    - mode (str, optional): Simulation mode. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Option to display a progress bar. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Counter for the progress bar. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result (qutip.solver.Result): Result object containing the simulation output.
    """
    # Default values
    if b is None: b = B
    if om_r is None: om_r = Om_r
    if om is None: om = omega
    if p is None: p = phi
    if w_p is None: w_p = W_p
    
    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    # Time resolution based on dt
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt
    
    # Define Hamiltonian and collapse operators based on mode
    match mode:
        case "Free":
            H = H_dua_hf(b, 0.0, phi_H)
            c_ops = L_dua_hf(0.0, k_index=k_index)
        case "MW":
            H = H_dua_hf(b, om_r, phi_H)
            c_ops = L_dua_hf(0.0, k_index=k_index)
        case "Laser":
            H = H_dua_hf(b, 0.0, phi_H)
            c_ops = L_dua_hf(w_p, k_index=k_index)
        case "Laser-MW":
            H = H_dua_hf(b, om_r, phi_H)
            c_ops = L_dua_hf(w_p, k_index=k_index)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')
    # Solve the master equation
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti    |    tf \n {ti:.2f} | {tf:.2f}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')

    return tf, result

# Contrast

In [ ]:
def contrast(dynamics,init_state,exp,b_v,w_p_v,w_p_r,omR,readtime=3.0):
    """
    Function find S0 S1 contrast the dynamics of the system.
    """
    _,res=dynamics(10.0,init_state,b=B0(b_v,0.0,0.0),w_p=w_p_v,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=0.0,mode="Laser",progress_bar="OFF")
    a=res.states[-1]
    t=res.times[-1]
    _,res=dynamics(1.0,a,b=B0(b_v,0.0,0.0),w_p=w_p_v,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=t,mode="Free",progress_bar="OFF")
    a=res.states[-1]
    t=res.times[-1]
    _,res=dynamics(10.0,a,b=B0(b_v,0.0,0.0),w_p=w_p_r,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=t,mode="Laser",progress_bar="OFF")
    wind=int(5000*readtime/10)
    s0=np.sum(qt.expect(exp,res.states[:wind]))
    s0_int=scp.integrate.simpson(qt.expect(exp,res.states[:wind]),res.times[:wind])
    s0_full=np.sum(qt.expect(exp,res.states))
    s0_int_full=scp.integrate.simpson(qt.expect(exp,res.states),res.times)
    _,res=dynamics(np.pi/omR,a,b=B0(b_v,0.0,0.0),w_p=w_p_v,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=t,mode="MW",progress_bar="OFF")
    a=res.states[-1]
    t=res.times[-1]
    _,res=dynamics(10.0,a,b=B0(b_v,0.0,0.0),w_p=w_p_r,om_r=omR,
                      om=D_gs-mu_e*b_v,ti=t,mode="Laser",progress_bar="OFF")
    s1=np.sum(qt.expect(exp,res.states[:wind]))
    s1_int=scp.integrate.simpson(qt.expect(exp,res.states[:wind]),res.times[:wind])
    s1_full=np.sum(qt.expect(exp,res.states))
    s1_int_full=scp.integrate.simpson(qt.expect(exp,res.states),res.times)
    return s0,s1,s0_full,s1_full,s0_int,s1_int,s0_int_full,s1_int_full

In [ ]:
#assert(1==2)

# Energy Levels

## No HF

In [ ]:
import os
import re

# ------------------------ appearance defaults ------------------------
# ---- labels / colors ----
label = ["$E_1$","$E_2$","$E_3$","$E_4$","$E_5$","$E_6$"]
e_colors = [
    "dodgerblue", "chocolate", "darkgoldenrod",
    "mediumpurple", "mediumseagreen", "lightskyblue"
]

# RGB definitions for LaTeX xcolor (matching the names above)
_MPL_COLOR_RGB = {
    "dodgerblue":     (30, 144, 255),
    "chocolate":      (210, 105,  30),
    "darkgoldenrod":  (184, 134,  11),
    "mediumpurple":   (147, 112, 219),
    "mediumseagreen": (60, 179, 113),
    "lightskyblue":   (135, 206, 250),
}

# ---- B values to tabulate (dedup + sort) ----
annotate_at_B = sorted(set([0, 100, 350, 475, 508, 525, 750, 1000, 1025, 1050, 1200]))

# ---- basis naming (remap to internal order) ----
user_labels_gs = [r"|1\rangle", r"|2\rangle", r"|3\rangle"]
internal_to_user_gs = [0, 2, 1]
basis_gs = [user_labels_gs[i] for i in internal_to_user_gs]

user_labels_es = [r"|4\rangle", r"|5\rangle", r"|6\rangle"]
internal_to_user_es = [0, 2, 1]
basis_es = [user_labels_es[i] for i in internal_to_user_es]

# ---- focus controls for zooms (Gauss) ----
# Top-right (GS zoom): center and ±range in B
B_focus_gs = 2870/mu_e
r_gs       = 15
# Bottom-right (ES zoom): center and ±range in B
B_focus_es = 1420/mu_e  # convert MHz to G
r_es       = 15

# ---- NEW: vertical (energy) zoom controls (MHz) ----
# If Y_focus_* is None, no y-zoom is applied. Otherwise use [Y_focus_* - Ry_*, Y_focus_* + Ry_*].
Y_focus_gs = 0
Ry_gs      = 10
Y_focus_es = 0
Ry_es      = 10

# ------------------------ helpers ------------------------
def nearest_idx(xarr, x):
    return int(np.abs(xarr - x).argmin())

def superposition_label(vec_qobj, basis_labels, kmax=3, prob_thresh=0.005, gap=0.15,
                        decimals=3, terms_per_line=2):
    """
    Build a compact LaTeX label for a superposition in the provided basis.
    If a term would print as '1.00|i⟩' (to 'decimals' places), omit '1.00' and print only '|i⟩'.
    """
    v = np.array(vec_qobj.full()).ravel()
    if v.size == 0:
        core_lines = [basis_labels[0]]
    else:
        top = np.argmax(np.abs(v))
        ang0 = np.angle(v[top]) if np.abs(v[top]) > 0 else 0.0
        v = v * np.exp(-1j * ang0)

        probs = np.abs(v) ** 2
        order = np.argsort(probs)[::-1]
        pmax = probs[order[0]]

        sel = [i for i in order if (probs[i] >= prob_thresh) or (probs[i] >= pmax - gap)]
        sel = sel[:kmax] if len(sel) > kmax else sel
        if not sel:
            sel = [order[0]]

        def fmt_term(i):
            amp = np.abs(v[i])
            s = f"{amp:.{decimals}f}"
            if s == f"{1.0:.{decimals}f}":
                return f"{basis_labels[i]}"
            return f"{s}{basis_labels[i]}"

        terms = [fmt_term(i) for i in sel]
        core_lines = []
        for s in range(0, len(terms), terms_per_line):
            chunk = terms[s:s+terms_per_line]
            if s > 0 and not chunk[0].startswith("-"):
                chunk[0] = "+" + chunk[0]
            core_lines.append("+".join(chunk).replace("+-", "-"))

    fmt = (r"${line}$")
    lines = [fmt.format(line=L) for L in core_lines]
    return "\n".join(lines), len(lines)

# ---------- write LaTeX table to file (ABNTEX-style float) ----------
def save_table_as_tex(filepath, col_labels, rows, caption, label_tex):
    """
    Writes an ABNTEX-style table float (no preamble). Intended for \\input{...}.
    NOTE: Your main LaTeX preamble should include \\usepackage{physics} so \\ket{} is defined.
    """
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    num_levels = len(col_labels) - 1

    def latex_levels(s: str) -> str:
        return re.sub(r"\|(\d+)\\rangle", r"\\ket{\1}", s).replace("\n", r"")

    header_cells = [r"\textbf{B}"] + [rf"\textbf{{{col_labels[j]}}}" for j in range(1, num_levels + 1)]
    header_line  = " & ".join(header_cells) + r" \\"

    body_lines = []
    for r in rows:
        bval = r[0]
        cells = [bval] + [latex_levels(c) for c in r[1:]]
        body_lines.append("    " + " & ".join(cells) + r" \\")
    body_block = "\n\\hline\n".join(body_lines)  # fixed invalid escape

    col_spec = "l|" + "c"*num_levels

    tex = rf"""% Requires in preamble: \usepackage{{physics}}
\begin{{table}}[hbt]
  \ABNTEXfontereduzida
  \caption{{{caption}}}
  \label{{{label_tex}}}
  \centering
  \begin{{tabular}}{{{col_spec}}}
    \hline
    {header_line}
    \hline
{body_block}
    \hline
  \end{{tabular}}
  \begin{{flushleft}}
    Source: By the author
  \end{{flushleft}}
\end{{table}}
"""
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(tex)

# ------------------------ MAIN ------------------------
init_b = 0
os.makedirs("Figs", exist_ok=True)
os.makedirs("Tables", exist_ok=True)
B_END = 1500

thetas = [0.0, np.pi/2, 2*np.pi/3, np.pi]
phis   = [0.0, np.pi/2, 2*np.pi/3, np.pi]

for thetab in thetas:
    for phib in phis:

        # ---- sweep field ----
        B_values = np.arange(init_b, B_END, 0.25)

        # pass 1: eigenenergies only (fast)
        energies_gs = np.empty((len(B_values), 3))
        energies_es = np.empty((len(B_values), 3))
        for k, i in enumerate(B_values):
            bs = B0(i, phib, thetab)
            H  = H_no(bs, 0.0)[0]
            H_full = H.full()
            h_gs = qt.Qobj(H_full[:3,  :3])
            h_es = qt.Qobj(H_full[3:6, 3:6])
            energies_gs[k] = h_gs.eigenenergies()
            energies_es[k] = h_es.eigenenergies()

        # ---- decide which B's we actually have and map to indices ----
        k_list = [nearest_idx(B_values, x) for x in annotate_at_B
                  if (B_values.min() <= x <= B_values.max())]
        k_list = sorted(set(k_list))

        # pass 2: eigenstates only at required rows (avoid 1200× eigenvectors)
        rows = []
        max_lines = 1
        num_levels = 6
        col_labels = ["B"] + label

        for k in k_list:
            i = B_values[k]
            bs = B0(i, phib, thetab)
            H  = H_no(bs, 0.0)[0]
            H_full = H.full()
            h_gs = qt.Qobj(H_full[:3,  :3])
            h_es = qt.Qobj(H_full[3:6, 3:6])
            _, evecs_g = h_gs.eigenstates()
            _, evecs_e = h_es.eigenstates()

            row = [str(i)]
            for j in range(num_levels):
                vec = evecs_g[j] if j < 3 else evecs_e[j-3]
                txt, nL = superposition_label(
                    vec, basis_gs if j < 3 else basis_es,
                    kmax=3, prob_thresh=0.005, gap=0.15, decimals=3,
                    terms_per_line=2
                )
                row.append(txt)
                max_lines = max(max_lines, nL)
            rows.append(row)

        # ---------- PLOTTING ----------
        theta_frac = Fraction(thetab/np.pi).limit_denominator()
        phi_frac   = Fraction(phib/np.pi).limit_denominator()

        special_layout = (np.isclose(thetab, 0.0) or np.isclose(thetab, np.pi)) and np.isclose(phib, 0.0)

        if special_layout:
            # 2×2 grid: GS full/zoom (top), ES full/zoom (bottom)
            fig, axes = plt.subplots(2, 2, figsize=(16, 10))  # no sharey so we can y-zoom independently

            ax_gs_full  = axes[0, 0]
            ax_gs_zoom  = axes[0, 1]
            ax_es_full  = axes[1, 0]
            ax_es_zoom  = axes[1, 1]

            # --- top row: first 3 energies (ground-state block)
            for j in range(3):
                y = energies_gs[:, j]
                ax_gs_full.plot(B_values, y, label=label[j], color=e_colors[j])
                #if j not in excl_gs:
                ax_gs_zoom.plot(B_values, y, label=label[j], color=e_colors[j])

            # --- bottom row: last 3 energies (excited-state block)
            for j in range(3, 6):
                y = energies_es[:, j-3]
                ax_es_full.plot(B_values, y, label=label[j], color=e_colors[j])
                #if (j-3) not in excl_es:  # indices 0..2 within ES block
                ax_es_zoom.plot(B_values, y, label=label[j], color=e_colors[j])

            fig.suptitle(f"$\\theta_B={theta_frac}\\pi$, $\\phi_B={phi_frac}\\pi$", fontsize=24, y=0.99)

            # Titles
            ax_gs_full.set_title("Ground states — full range", fontsize=18)
            ax_es_full.set_title("Excited states — full range", fontsize=18)
            ax_gs_zoom.set_title(
                rf"Ground states — zoom",#@ ${B_focus_gs:.2f}\,\mathrm{{G}}\pm{r_gs}\,\mathrm{{G}}$",
                fontsize=18
            )
            ax_es_zoom.set_title(
                rf"Excited states — zoom", #@ ${B_focus_es:.2f}\,\mathrm{{G}}\pm{r_es}\,\mathrm{{G}}$",
                fontsize=18
            )

            # Labels
            ax_gs_full.set_ylabel('Eigenenergy (MHz)', fontsize=16)
            ax_es_full.set_ylabel('Eigenenergy (MHz)', fontsize=16)
            ax_es_full.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=16, labelpad=6)
            ax_es_zoom.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=16, labelpad=6)

            # Horizontal zoom windows (symmetric in B)
            ax_gs_zoom.set_xlim(B_focus_gs - r_gs, B_focus_gs + r_gs)
            ax_es_zoom.set_xlim(B_focus_es - r_es, B_focus_es + r_es)

            # ---- NEW: Vertical zoom windows (if specified)
            if Y_focus_gs is not None:
                ax_gs_zoom.set_ylim(Y_focus_gs - Ry_gs, Y_focus_gs + Ry_gs)
            if Y_focus_es is not None:
                ax_es_zoom.set_ylim(Y_focus_es - Ry_es, Y_focus_es + Ry_es)

            # Ticks/legends
            for ax in (ax_gs_full, ax_gs_zoom, ax_es_full, ax_es_zoom):
                ax.tick_params(labelsize=12)
            ax_gs_full.legend(loc="best", fontsize=12, frameon=False, title="GS (E1–E3)")
            ax_es_full.legend(loc="best", fontsize=12, frameon=False, title="ES (E4–E6)")

            fig.subplots_adjust(top=0.90, wspace=0.25, hspace=0.28, left=0.07, right=0.98)

            # ---- save plot ----
            plt.savefig(
                f'Figs/energy_levels_GRID_t-{int(100*thetab)}_p-{int(100*phib)}.png',
                bbox_inches='tight', dpi=300
            )
            plt.show()

        else:
            # Original single-axes plot for all other angles
            fig = plt.figure(figsize=(16, 8.0))
            ax  = fig.add_subplot(111)

            for j in range(num_levels):
                y = energies_gs[:, j] if j < 3 else energies_es[:, j-3]
                ax.plot(B_values, y, label=label[j], color=e_colors[j])

            ax.set_title(
                f"$\\theta_B={theta_frac}\\pi$, $\\phi_B={phi_frac}\\pi$",
                fontsize=24
            )
            ax.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=22, labelpad=8)
            ax.set_ylabel('Eigenenergy (MHz)', fontsize=22)
            ax.tick_params(labelsize=16)
            ax.legend(bbox_to_anchor=(1.03, 1), loc="upper left", fontsize=20, frameon=False)

            fig.subplots_adjust(top=0.96, bottom=0.10, left=0.07, right=0.86)

            # ---- save plot ----
            plt.savefig(
                f'Figs/energy_levels_SINGLE_t-{int(100*thetab)}_p-{int(100*phib)}.png',
                bbox_inches='tight', dpi=300
            )
            plt.show()

        # ---------- save LaTeX table (ABNTEX-style) ----------
        tex_path   = f'Tables/energy_levels_table_t-{int(100*thetab)}_p-{int(100*phib)}.tex'
        caption    = rf"Eigenstate compositions at $\theta_B={theta_frac}\pi$, $\phi_B={phi_frac}\pi$."
        label_tex  = rf"tab:energy_levels_t-{int(100*thetab)}_p-{int(100*phib)}"

        save_table_as_tex(tex_path, col_labels, rows, caption, label_tex)

        # keep only the first phi when theta is 0 or pi (i.e., run only phi=0 for theta=0,π)
        if np.isclose(thetab, 0) or np.isclose(thetab, np.pi):
            break

plt.close('all')


## Model 3

In [ ]:
import os
import re

# ------------------------ labels / colors ------------------------
labels = [
    r"$E_{1,\downarrow}$", r"$E_{1,\uparrow}$",
    r"$E_{2,\downarrow}$", r"$E_{2,\uparrow}$",
    r"$E_{3,\downarrow}$", r"$E_{3,\uparrow}$",
    r"$E_{4,\downarrow}$", r"$E_{4,\uparrow}$",
    r"$E_{5,\downarrow}$", r"$E_{5,\uparrow}$",
    r"$E_{6,\downarrow}$", r"$E_{6,\uparrow}$",
]
# Pairs share a color; ↑ is dashed. (We only use the first 6 entries.)
e_colors = [
    "dodgerblue", "chocolate", "darkgoldenrod",
    "mediumpurple", "mediumseagreen", "lightskyblue", "magenta"
]

# ---------- table rows at the SAME B values ----------
annotate_at_B = sorted(set([0, 100, 350, 475, 508, 525, 750, 1000, 1025, 1050, 1200]))

# ---------- BASIS LABELS (match to your *internal* basis order) ----------
user_labels_gs6 = [
    r"|1,\downarrow\rangle", r"|1,\uparrow\rangle",
    r"|2,\downarrow\rangle", r"|2,\uparrow\rangle",
    r"|3,\downarrow\rangle", r"|3,\uparrow\rangle",
]
internal_to_user_gs6 = [0, 1, 4, 5, 2, 3]
basis_gs6 = [user_labels_gs6[i] for i in internal_to_user_gs6]

user_labels_es6 = [
    r"|4,\downarrow\rangle", r"|4,\uparrow\rangle",
    r"|5,\downarrow\rangle", r"|5,\uparrow\rangle",
    r"|6,\downarrow\rangle", r"|6,\uparrow\rangle",
]
internal_to_user_es6 = [0, 1, 4, 5, 2, 3]
basis_es6 = [user_labels_es6[i] for i in internal_to_user_es6]

# ---------- horizontal zoom controls (Gauss) ----------
B_focus_gs = 2870/mu_e
r_gs       = 15
B_focus_es = 496  # convert MHz to G
r_es       = 60

# ---------- NEW: vertical zoom controls (MHz) ----------
# If Y_focus_* is None, no y-zoom is applied.
Y_focus_gs = 0.0   # e.g. 2870.0
Ry_gs      = 10.0
Y_focus_es = 0.0   # e.g. 1500.0
Ry_es      = 50.0

# ---------- helpers ----------
def nearest_idx(xarr, x):
    return int(np.abs(xarr - x).argmin())

def superposition_label(vec_qobj, basis_labels, kmax=6, prob_thresh=0.005, gap=0.15,
                        decimals=3, terms_per_line=2):
    """
    Return (cell_text, n_lines); each line is its own $...$.
    If a coefficient would print as '1.00' (with given 'decimals'), omit it and show only the state.
    """
    v = np.array(vec_qobj.full()).ravel()
    if v.size == 0:
        core_lines = [basis_labels[0]]
    else:
        # fix global phase (largest component real-positive)
        top = np.argmax(np.abs(v))
        ang0 = np.angle(v[top]) if np.abs(v[top]) > 0 else 0.0
        v = v * np.exp(-1j * ang0)

        probs = np.abs(v) ** 2
        order = np.argsort(probs)[::-1]
        pmax = probs[order[0]]

        sel = [i for i in order if (probs[i] >= prob_thresh) or (probs[i] >= pmax - gap)]
        sel = sel[:kmax] if len(sel) > kmax else sel
        if not sel:
            sel = [order[0]]

        def fmt_term(i):
            amp = np.abs(v[i])
            s = f"{amp:.{decimals}f}"
            if s == f"{1.0:.{decimals}f}":
                return f"{basis_labels[i]}"
            return f"{s}{basis_labels[i]}"

        terms = [fmt_term(i) for i in sel]
        core_lines = []
        for s in range(0, len(terms), terms_per_line):
            chunk = terms[s:s+terms_per_line]
            if s > 0 and not chunk[0].startswith("-"):
                chunk[0] = "+" + chunk[0]
            core_lines.append("+".join(chunk).replace("+-", "-"))

    fmt = r"${line}$"
    lines = [fmt.format(line=L) for L in core_lines]
    return "\n".join(lines), len(lines)

# ---------- LaTeX table writer (ABNTEX-style float; no preamble) ----------
def save_table_as_tex(filepath, col_labels, rows, caption, label):
    """
    Writes an ABNTEX-style table float. Use \\input{...} in your main doc.
    Cells are assumed to be TeX-ready (may contain \\shortstack and \\).
    Your preamble must include: \\usepackage{physics} (for \\ket{}).
    """
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    num_levels = len(col_labels) - 1

    header_cells = [r"\textbf{B}"] + [rf"{c}" for c in col_labels[1:]]
    header_line  = " & ".join(header_cells) + r" \\"

    body_lines = []
    for r in rows:
        bval  = r[0]
        cells = [bval] + r[1:]  # already formatted
        body_lines.append("    " + " & ".join(cells) + r" \\")
    body_block = "\n\\hline\n".join(body_lines)  # fixed invalid escape

    col_spec = "l|" + "c"*num_levels

    tex = rf"""% Requires in preamble: \usepackage{{physics}}
\begin{{table}}[hbt]
  \ABNTEXfontereduzida
  \caption{{{caption}}}
  \label{{{label}}}
  \centering
  \begin{{tabular}}{{{col_spec}}}
    \hline
    {header_line}
    \hline
{body_block}
    \hline
  \end{{tabular}}
  \begin{{flushleft}}
    Source: By the author
  \end{{flushleft}}
\end{{table}}
"""
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(tex)

# ---------------------------------------------------------------
# MAIN: sweep, compute spectra, plot + write LaTeX table
# ---------------------------------------------------------------
init_b = 0
B_END = 1500
os.makedirs("Figs", exist_ok=True)
os.makedirs("Tables", exist_ok=True)

thetas = [0.0, np.pi/2, 2*np.pi/3, np.pi]
phis   = [0.0, np.pi/2, 2*np.pi/3, np.pi]

for thetab in thetas:
    for phib in phis:

        # ---- sweep field (energies only: fast pass) ----
        B_values = np.arange(init_b, B_END, 0.25)
        nB = len(B_values)

        energies_gs = np.empty((nB, 6))
        energies_es = np.empty((nB, 6))

        for k, i in enumerate(B_values):
            bs = B0(i, phib, thetab)
            H  = H_doh_hf(bs, 0.0)[0] + H_doh_hf(bs, 0.0)[1]   # full 12×12
            H_full = H.full()

            h_gs = qt.Qobj(H_full[:6,  :6])        # first 6
            h_es = qt.Qobj(H_full[6:12, 6:12])     # last 6

            energies_gs[k] = h_gs.eigenenergies()
            energies_es[k] = h_es.eigenenergies()

        # ---- indices for table rows ----
        k_list = [nearest_idx(B_values, x) for x in annotate_at_B
                  if (B_values.min() <= x <= B_values.max())]
        k_list = sorted(set(k_list))

        # ---- compute eigenstates only at table rows ----
        rows = []
        num_levels = 12

        # Two-line headers: E_{i,↓} over E_{i,↑}
        pair_headers = []
        for i_pair in range(6):
            hd = labels[2*i_pair]     # down
            hu = labels[2*i_pair+1]   # up
            pair_headers.append(rf"\shortstack{{\textbf{{{hd}}} \\ \textbf{{{hu}}}}}")
        col_labels = ["B"] + pair_headers

        # helper to make \ket{} and preserve newlines as \\ inside stacks
        def to_ket_and_preserve(s: str) -> str:
            s = re.sub(r"\|(.*?)\\rangle", r"\\ket{\1}", s)
            return s.replace("\n", r"\\")

        for k in k_list:
            iB = B_values[k]
            bs = B0(iB, phib, thetab)
            H  = H_doh_hf(bs, 0.0)[0] + H_doh_hf(bs, 0.0)[1]   # full 12×12
            H_full = H.full()

            h_gs = qt.Qobj(H_full[:6,  :6])
            h_es = qt.Qobj(H_full[6:12, 6:12])
            _, evecs_g = h_gs.eigenstates()
            _, evecs_e = h_es.eigenstates()

            # 12 state strings (may be multi-line)
            state_strs = []
            for j in range(num_levels):
                vec = evecs_g[j] if j < 6 else evecs_e[j-6]
                txt, _ = superposition_label(
                    vec,
                    basis_gs6 if j < 6 else basis_es6,
                    kmax=6, prob_thresh=0.005, gap=0.15, decimals=3,
                    terms_per_line=2
                )
                state_strs.append(to_ket_and_preserve(txt))

            # collapse to 6 columns: \shortstack of (down \\ up)
            pair_cells = []
            for i_pair in range(6):
                down_tex = state_strs[2*i_pair]
                up_tex   = state_strs[2*i_pair+1]
                pair_cells.append(rf"\shortstack[c]{{{down_tex} \\ {up_tex}}}")

            rows.append([str(iB)] + pair_cells)

        # ---------- PLOTTING ----------
        theta_frac = Fraction(thetab/np.pi).limit_denominator()
        phi_frac   = Fraction(phib/np.pi).limit_denominator()
        special_layout = (np.isclose(thetab, 0.0) or np.isclose(thetab, np.pi)) and np.isclose(phib, 0.0)

        if special_layout:
            # 2×2 grid: first 6 (top), last 6 (bottom); left=full, right=zoom
            # NOTE: no sharey so vertical zoom applies only to the zoom panels
            fig, axes = plt.subplots(2, 2, figsize=(16, 10))

            ax_gs_full  = axes[0, 0]
            ax_gs_zoom  = axes[0, 1]
            ax_es_full  = axes[1, 0]
            ax_es_zoom  = axes[1, 1]

            # --- top row: first 6 levels (pairs share color; ↑ dashed)
            for j in range(6):
                y = energies_gs[:, j]
                ax_gs_full.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )
                
                ax_gs_zoom.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )

            # --- bottom row: last 6 levels
            for j in range(6, 12):
                y = energies_es[:, j-6]
                ax_es_full.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],  
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )
                ax_es_zoom.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )

            fig.suptitle(f"$\\theta_B={theta_frac}\\pi$, $\\phi_B={phi_frac}\\pi$", fontsize=24, y=0.99)

            # Titles
            ax_gs_full.set_title("First 6 levels — full range", fontsize=18)
            ax_es_full.set_title("Last 6 levels — full range", fontsize=18)
            ax_gs_zoom.set_title(
                rf"First 6 — zoom",# @ ${B_focus_gs:.2f}\,\mathrm{{G}}\pm{r_gs}\,\mathrm{{G}}$",
                fontsize=18
            )
            ax_es_zoom.set_title(
                rf"Last 6 — zoom",# @ ${B_focus_es:.2f}\,\mathrm{{G}}\pm{r_es}\,\mathrm{{G}}$",
                fontsize=18
            )

            # Labels
            ax_gs_full.set_ylabel('Eigenenergy (MHz)', fontsize=16)
            ax_es_full.set_ylabel('Eigenenergy (MHz)', fontsize=16)
            ax_es_full.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=16, labelpad=6)
            ax_es_zoom.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=16, labelpad=6)

            # Horizontal zoom windows (symmetric)
            ax_gs_zoom.set_xlim(B_focus_gs - r_gs, B_focus_gs + r_gs)
            ax_es_zoom.set_xlim(B_focus_es - r_es, B_focus_es + r_es)

            # Vertical zoom windows (if specified)
            if Y_focus_gs is not None:
                ax_gs_zoom.set_ylim(Y_focus_gs - Ry_gs, Y_focus_gs + Ry_gs)
            if Y_focus_es is not None:
                ax_es_zoom.set_ylim(Y_focus_es - Ry_es, Y_focus_es + Ry_es)

            # Ticks/legends
            for ax in (ax_gs_full, ax_gs_zoom, ax_es_full, ax_es_zoom):
                ax.tick_params(labelsize=12)
            ax_gs_full.legend(loc="best", fontsize=11, frameon=False, title="First 6")
            ax_es_full.legend(loc="best", fontsize=11, frameon=False, title="Last 6")

            fig.subplots_adjust(top=0.90, wspace=0.25, hspace=0.28, left=0.07, right=0.98)

            # ---- save plot ----
            plt.savefig(
                f'Figs/energy_levels_M3_GRID_t-{int(100*thetab)}_p-{int(100*phib)}.png',
                bbox_inches='tight', dpi=300
            )
            plt.show()

        else:
            # Original single-axes plot for all other angles
            fig = plt.figure(figsize=(16, 8.0))
            ax  = fig.add_subplot(111)

            for j in range(num_levels):
                y = energies_gs[:, j] if j < 6 else energies_es[:, j-6]
                ax.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[(j // 2) if j < 12 else -1],
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )

            ax.set_title(
                f"$\\theta_B={theta_frac}\\pi$, $\\phi_B={phi_frac}\\pi$",
                fontsize=28
            )
            ax.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=26, labelpad=8)
            ax.set_ylabel('Eigenenergy (MHz)', fontsize=26)
            ax.tick_params(labelsize=20)
            fig.subplots_adjust(left=0.07, right=0.86, top=0.96, bottom=0.10)
            ax.legend(bbox_to_anchor=(1.00, 1.00), loc="upper left", fontsize=22, frameon=False)

            # ---- save plot ----
            plt.savefig(
                f'Figs/energy_levels_M3_no_table_t-{int(100*thetab)}_p-{int(100*phib)}.png',
                bbox_inches='tight', dpi=300
            )
            plt.show()

        # ---- save LaTeX table (ABNTEX-style, 6 columns with stacked ↑/↓) ----
        tex_path   = f'Tables/energy_levels_M3_table_t-{int(100*thetab)}_p-{int(100*phib)}.tex'
        caption    = rf"Eigenstate compositions at $\theta_B={theta_frac}\pi$, $\phi_B={phi_frac}\pi$."
        label_tex  = rf"tab:energy_levels_M3_t-{int(100*thetab)}_p-{int(100*phib)}"

        save_table_as_tex(tex_path, col_labels, rows, caption, label_tex)

        # keep only the first phi when theta is 0 or pi (=> only (0,0) and (π,0))
        if np.isclose(thetab, 0) or np.isclose(thetab, np.pi):
            break

plt.close('all')


## Model 2

In [ ]:
import os
import re

# ------------------------ labels / colors ------------------------
labels = [
    r"$E_{1,\downarrow}$", r"$E_{1,\uparrow}$",
    r"$E_{2,\downarrow}$", r"$E_{2,\uparrow}$",
    r"$E_{3,\downarrow}$", r"$E_{3,\uparrow}$",
    r"$E_{4,\downarrow}$", r"$E_{4,\uparrow}$",
    r"$E_{5,\downarrow}$", r"$E_{5,\uparrow}$",
    r"$E_{6,\downarrow}$", r"$E_{6,\uparrow}$",
]
# Pairs share a color; ↑ is dashed.
e_colors = [
    "dodgerblue", "chocolate", "darkgoldenrod",
    "mediumpurple", "mediumseagreen", "lightskyblue", "magenta"
]

# ---------- table rows at the SAME B values ----------
annotate_at_B = sorted(set([0, 100, 350, 475, 508, 525, 750, 1000, 1025, 1050, 1200]))

# ---------- BASIS LABELS (match to your *internal* basis order) ----------
user_labels_gs6 = [
    r"|1,\downarrow\rangle", r"|1,\uparrow\rangle",
    r"|2,\downarrow\rangle", r"|2,\uparrow\rangle",
    r"|3,\downarrow\rangle", r"|3,\uparrow\rangle",
]
internal_to_user_gs6 = [0, 1, 4, 5, 2, 3]
basis_gs6 = [user_labels_gs6[i] for i in internal_to_user_gs6]

user_labels_es6 = [
    r"|4,\downarrow\rangle", r"|4,\uparrow\rangle",
    r"|5,\downarrow\rangle", r"|5,\uparrow\rangle",
    r"|6,\downarrow\rangle", r"|6,\uparrow\rangle",
]
internal_to_user_es6 = [0, 1, 4, 5, 2, 3]
basis_es6 = [user_labels_es6[i] for i in internal_to_user_es6]

# ---------- horizontal zoom controls (Gauss) ----------
B_focus_gs = 2870/mu_e
r_gs       = 15
B_focus_es = 496 # convert MHz to G
r_es       = 60

# ---------- NEW: vertical zoom controls (MHz) ----------
# If Y_focus_* is None, no y-zoom is applied on the zoom panels.
Y_focus_gs = 0.0   # e.g. 2870.0
Ry_gs      = 10.0
Y_focus_es = 0.0   # e.g. 1500.0
Ry_es      = 50.0

# ---------- helpers ----------
def nearest_idx(xarr, x):
    return int(np.abs(xarr - x).argmin())

def superposition_label(vec_qobj, basis_labels, kmax=6, prob_thresh=0.005, gap=0.15,
                        decimals=3, terms_per_line=2, pad_math_lines=0):
    """Return (cell_text, n_lines); multi-line, each line wrapped in its own $...$.
       If a coefficient prints as '1.00' (to 'decimals'), omit it and show only the state."""
    v = np.array(vec_qobj.full()).ravel()
    if v.size == 0:
        core_lines = [basis_labels[0]]
    else:
        # fix global phase (largest component real-positive)
        top = np.argmax(np.abs(v))
        ang0 = np.angle(v[top]) if np.abs(v[top]) > 0 else 0.0
        v = v * np.exp(-1j * ang0)

        probs = np.abs(v) ** 2
        order = np.argsort(probs)[::-1]
        pmax = probs[order[0]]

        sel = [i for i in order if (probs[i] >= prob_thresh) or (probs[i] >= pmax - gap)]
        sel = sel[:kmax] if len(sel) > kmax else sel
        if not sel:
            sel = [order[0]]

        def fmt_term(i):
            amp = np.abs(v[i])
            s = f"{amp:.{decimals}f}"
            if s == f"{1.0:.{decimals}f}":
                return f"{basis_labels[i]}"
            return f"{s}{basis_labels[i]}"

        terms = [fmt_term(i) for i in sel]
        core_lines = []
        for s in range(0, len(terms), terms_per_line):
            chunk = terms[s:s+terms_per_line]
            if s > 0 and not chunk[0].startswith("-"):
                chunk[0] = "+" + chunk[0]
            core_lines.append("+".join(chunk).replace("+-", "-"))

    fmt = (r"${line}$")
    lines = [fmt.format(line=L) for L in core_lines]
    if pad_math_lines > 0:
        pad = "$\\,$"
        lines = [pad]*pad_math_lines + lines + [pad]*pad_math_lines
    return "\n".join(lines), len(lines)

# ---------- LaTeX table writer (ABNTEX-style float; no preamble) ----------
def save_table_as_tex(filepath, col_labels, rows, caption, label):
    """
    Writes an ABNTEX-style table float. Use \\input{...} in your main doc.
    Cells may contain \\shortstack and internal line breaks (\\).
    Your preamble must include: \\usepackage{physics} (for \\ket{}).
    """
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    num_levels = len(col_labels) - 1

    header_cells = [r"\textbf{B}"] + [c for c in col_labels[1:]]
    header_line  = " & ".join(header_cells) + r" \\"

    body_lines = []
    for r in rows:
        bval  = r[0]
        cells = [bval] + r[1:]  # already formatted
        body_lines.append("    " + " & ".join(cells) + r" \\")
    body_block = "\n\\hline\n".join(body_lines)  # fixed invalid escape

    col_spec = "l|" + "c"*num_levels

    tex = rf"""% Requires in preamble: \usepackage{{physics}}
\begin{{table}}[hbt]
  \ABNTEXfontereduzida
  \caption{{{caption}}}
  \label{{{label}}}
  \centering
  \begin{{tabular}}{{{col_spec}}}
    \hline
    {header_line}
    \hline
{body_block}
    \hline
  \end{{tabular}}
  \begin{{flushleft}}
    Source: By the author
  \end{{flushleft}}
\end{{table}}
"""
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(tex)

# ---------------------------------------------------------------
# MAIN: sweep, compute spectra, plot + write LaTeX table (stack ↑ under ↓)
# ---------------------------------------------------------------
init_b = 0
B_END = 1500
os.makedirs("Figs", exist_ok=True)
os.makedirs("Tables", exist_ok=True)

thetas = [0.0, np.pi/2, 2*np.pi/3, np.pi]
phis   = [0.0, np.pi/2, 2*np.pi/3, np.pi]

for thetab in thetas:
    for phib in phis:

        # ---- sweep field (energies only: fast pass) ----
        B_values = np.arange(init_b, B_END, 0.25)
        nB = len(B_values)

        energies_gs = np.empty((nB, 6))
        energies_es = np.empty((nB, 6))

        for k, i in enumerate(B_values):
            bs = B0(i, phib, thetab)
            # full 12×12 via TWO calls (pattern preserved)
            H  = H_dua_hf(bs, 0.0, 0.0)[0] + H_dua_hf(bs, 0.0, 0.0)[1]
            H_full = H.full()
            h_gs = qt.Qobj(H_full[:6,  :6])        # GS 6×6
            h_es = qt.Qobj(H_full[6:12, 6:12])     # ES 6×6
            energies_gs[k] = h_gs.eigenenergies()
            energies_es[k] = h_es.eigenenergies()

        # ---- indices for table rows ----
        k_list = [nearest_idx(B_values, x) for x in annotate_at_B
                  if (B_values.min() <= x <= B_values.max())]
        k_list = sorted(set(k_list))

        # ---- compute eigenstates only at table rows ----
        rows = []
        num_levels = 12

        # 6 headers: stack E_{i,↓} over E_{i,↑}
        pair_headers = []
        for i_pair in range(6):
            hd = labels[2*i_pair]     # already with $...$
            hu = labels[2*i_pair+1]
            pair_headers.append(rf"\shortstack{{\textbf{{{hd}}} \\ \textbf{{{hu}}}}}")
        col_labels = ["B"] + pair_headers

        def to_ket_and_preserve(s: str) -> str:
            s = re.sub(r"\|(.*?)\\rangle", r"\\ket{\1}", s)
            return s.replace("\n", r"\\")  # preserve multi-line layout

        for k in k_list:
            iB = B_values[k]
            bs = B0(iB, phib, thetab)
            H  = H_dua_hf(bs, 0.0, 0.0)[0] + H_dua_hf(bs, 0.0, 0.0)[1]
            H_full = H.full()

            h_gs = qt.Qobj(H_full[:6,  :6])
            h_es = qt.Qobj(H_full[6:12, 6:12])
            _, evecs_g = h_gs.eigenstates()
            _, evecs_e = h_es.eigenstates()

            state_strs = []
            for j in range(num_levels):
                vec = evecs_g[j] if j < 6 else evecs_e[j-6]
                txt, _ = superposition_label(
                    vec,
                    basis_gs6 if j < 6 else basis_es6,
                    kmax=6, prob_thresh=0.005, gap=0.15, decimals=3,
                    terms_per_line=2, pad_math_lines=0,
                )
                state_strs.append(to_ket_and_preserve(txt))

            # collapse to 6 columns: \shortstack of (down \\ up)
            pair_cells = []
            for i_pair in range(6):
                down_tex = state_strs[2*i_pair]
                up_tex   = state_strs[2*i_pair+1]
                pair_cells.append(rf"\shortstack[c]{{{down_tex} \\ {up_tex}}}")

            rows.append([str(iB)] + pair_cells)

        # ---------- PLOTTING ----------
        theta_frac = Fraction(thetab/np.pi).limit_denominator()
        phi_frac   = Fraction(phib/np.pi).limit_denominator()
        special_layout = (np.isclose(thetab, 0.0) or np.isclose(thetab, np.pi)) and np.isclose(phib, 0.0)

        if special_layout:
            # 2×2 grid: first 6 (top), last 6 (bottom); left=full, right=zoom
            # No sharey to allow vertical zoom on the right panels
            fig, axes = plt.subplots(2, 2, figsize=(16, 10))

            ax_gs_full  = axes[0, 0]
            ax_gs_zoom  = axes[0, 1]
            ax_es_full  = axes[1, 0]
            ax_es_zoom  = axes[1, 1]

            # --- top row: first 6 levels (pairs share color; ↑ dashed)
            for j in range(6):
                y = energies_gs[:, j]
                pair_idx = j // 2  # 0→E1, 1→E2, 2→E3
                ax_gs_full.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )
                #if pair_idx not in excl_gs_pairs:
                ax_gs_zoom.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )

            # --- bottom row: last 6 levels
            for j in range(6, 12):
                y = energies_es[:, j-6]
                ax_es_full.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],  # pairs 3..5 map to indices 3..5
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )
                ax_es_zoom.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )

            fig.suptitle(f"$\\theta_B={theta_frac}\\pi$, $\\phi_B={phi_frac}\\pi$", fontsize=24, y=0.99)

            # Titles
            ax_gs_full.set_title("First 6 levels — full range", fontsize=18)
            ax_es_full.set_title("Last 6 levels — full range", fontsize=18)
            ax_gs_zoom.set_title(
                rf"First 6 — zoom",# @ ${B_focus_gs:.2f}\,\mathrm{{G}}\pm{r_gs}\,\mathrm{{G}}$",
                fontsize=18
            )
            ax_es_zoom.set_title(
                rf"Last 6 — zoom",# @ ${B_focus_es:.2f}\,\mathrm{{G}}\pm{r_es}\,\mathrm{{G}}$",
                fontsize=18
            )

            # Labels
            ax_gs_full.set_ylabel('Eigenenergy (MHz)', fontsize=16)
            ax_es_full.set_ylabel('Eigenenergy (MHz)', fontsize=16)
            ax_es_full.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=16, labelpad=6)
            ax_es_zoom.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=16, labelpad=6)

            # Horizontal zoom windows (symmetric)
            ax_gs_zoom.set_xlim(B_focus_gs - r_gs, B_focus_gs + r_gs)
            ax_es_zoom.set_xlim(B_focus_es - r_es, B_focus_es + r_es)

            # Vertical zoom windows (if specified)
            if Y_focus_gs is not None:
                ax_gs_zoom.set_ylim(Y_focus_gs - Ry_gs, Y_focus_gs + Ry_gs)
            if Y_focus_es is not None:
                ax_es_zoom.set_ylim(Y_focus_es - Ry_es, Y_focus_es + Ry_es)

            # Ticks/legends
            for ax in (ax_gs_full, ax_gs_zoom, ax_es_full, ax_es_zoom):
                ax.tick_params(labelsize=12)
            ax_gs_full.legend(loc="best", fontsize=11, frameon=False, title="First 6")
            ax_es_full.legend(loc="best", fontsize=11, frameon=False, title="Last 6")

            fig.subplots_adjust(top=0.90, wspace=0.25, hspace=0.28, left=0.07, right=0.98)

            # ---- save plot ----
            plt.savefig(
                f'Figs/energy_levels_M2_GRID_t-{int(100*thetab)}_p-{int(100*phib)}.png',
                bbox_inches='tight', dpi=300
            )
            plt.show()

        else:
            # Original single-axes plot for all other angles
            fig = plt.figure(figsize=(16, 8.0))
            ax  = fig.add_subplot(111)

            for j in range(num_levels):
                y = energies_gs[:, j] if j < 6 else energies_es[:, j-6]
                ax.plot(
                    B_values, y,
                    ls='--' if (j % 2 == 1) else '-',
                    label=labels[j],
                    color=e_colors[j // 2],
                    alpha=1.0 if (j % 2 == 1) else 0.55
                )

            ax.set_title(
                f"$\\theta_B={theta_frac}\\pi$, $\\phi_B={phi_frac}\\pi$",
                fontsize=28
            )
            ax.set_xlabel('Magnetic Field $B_0$ (G)', fontsize=26, labelpad=8)
            ax.set_ylabel('Eigenenergy (MHz)', fontsize=26)
            ax.tick_params(labelsize=20)
            fig.subplots_adjust(left=0.07, right=0.86, top=0.96, bottom=0.10)
            ax.legend(bbox_to_anchor=(1.00, 1.00), loc="upper left", fontsize=22, frameon=False)

            # ---- save plot ----
            plt.savefig(
                f'Figs/energy_levels_M2_no_table_t-{int(100*thetab)}_p-{int(100*phib)}.png',
                bbox_inches='tight', dpi=300
            )
            plt.show()

        # ---- save LaTeX table (ABNTEX-style, 6 columns with stacked ↑/↓) ----
        tex_path   = f'Tables/energy_levels_M2_table_t-{int(100*thetab)}_p-{int(100*phib)}.tex'
        caption    = rf"Eigenstate compositions at $\theta_B={theta_frac}\pi$, $\phi_B={phi_frac}\pi$."
        label_tex  = rf"tab:energy_levels_M2_t-{int(100*thetab)}_p-{int(100*phib)}"

        save_table_as_tex(tex_path, col_labels, rows, caption, label_tex)

        # keep only the first phi when theta is 0 or pi (=> only (0,0) and (π,0))
        if np.isclose(thetab, 0) or np.isclose(thetab, np.pi):
            break

plt.close('all')


In [ ]:
assert(1==2)

# Population Dynamics

## 14 Levels

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
def plot_bang_14_doh_vs_dua(n_exp_doh, times_doh, tis_doh, tfs_doh, 
                                      n_exp_dua, times_dua, tis_dua, tfs_dua, 
                                      b, p, t, om):
    """
    Plots population dynamics for 14 levels (7 electronic states × 2 nuclear spin states)
    comparing two datasets (Model 3 vs. Model 2).
    """
    # Define colors for electronic states (7 colors)
    e_colors = [
        "dodgerblue", "chocolate", "darkgoldenrod", 
        "mediumpurple", "mediumseagreen", "lightskyblue", "magenta"
    ]
    
    # Line styles for nuclear spin states
    n_marker = ['v', '^']
    m_style = ['-', '--']
    # Build a list of indices:
    # - For the first 5000 points, mark every 500.
    markers1 = list(range(0, 5000, 500))
    # - For the next 1000 points (5000–5999), mark every 400.
    markers2 = list(range(5000, 6000, 400))
    # - For the last 1000 points (6000–6999), mark every 100.
    markers3 = list(range(6000, 7000, 100))

    mark_indices = markers1 + markers2 + markers3
    plt.figure(figsize=(14, 8))

    # Plot Model 3 data
    for i in range(14):
        e_idx = i // 2 # Electronic state index (0-6)
        n_idx = i % 2   # Nuclear spin index (0-1)
        
        plt.plot(times_doh, n_exp_doh[i], 
                 color=e_colors[e_idx], 
                 linestyle=m_style[0],
                 marker=n_marker[n_idx],
                 markersize=8,markevery=mark_indices,
                 label=f"$n^{{doh}}_{{{e_idx+1},{n_idx}}}$")
    #for k in range(2):
    #    plt.plot(times_doh, n_exp_doh[14+k], 
    #                color='forestgreen', 
    #                linestyle='-',
    #                marker=n_marker[n_idx],
    #                markersize=4,markevery=50,
    #                linewidth=2,
    #                label=f'$n^{{doh}}_{{{{c}},{n_idx}}}')
    # Plot Model 2 data with thicker dashed lines
    for i in range(14):
        e_idx = i // 2
        n_idx = i % 2
        
        plt.plot(times_dua, n_exp_dua[i], 
                 color=e_colors[e_idx], 
                 linestyle=m_style[1],
                 marker=n_marker[n_idx],
                 markersize=8,markevery=mark_indices,
                 linewidth=3, alpha=0.7,
                 label=f"$n^{{dua}}_{{{e_idx+1},{n_idx}}}$")
    #for k in range(2):
    #    plt.plot(times_dua, n_exp_dua[14+k], 
    #                color='forestgreen', 
    #                linestyle='--',
    #                marker=n_marker[n_idx],
    #                markersize=4,markevery=90,
    #                linewidth=2,
    #                label=f'$n^{{dua}}_{{{{c}},{n_idx}}}')
    # Highlight sequence phases
    n = 0
    
    eps_y = 0.05 * (np.max(np.concatenate([n_exp_doh, n_exp_dua])) - 
                   np.min(np.concatenate([n_exp_doh, n_exp_dua])))
    eps_x = 0.05 * (np.max(np.concatenate([times_doh, times_dua])) - 
                   np.min(np.concatenate([times_doh, times_dua])))
    for i in zip(tis_doh, tfs_doh):
        if n % 3 == 0:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="palegreen", alpha=0.3,
            label="Laser ON" if n == 0 else "")
            t_laser = i[1]-i[0]
        elif (n-1) % 3 == 0:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="lightblue", alpha=0.3,
            label="Free Evolution" if n == 1 else "")
            t_free = i[1]-i[0]
        else:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="orchid", alpha=0.3,
            label="MW ON" if n == 2 else "")
            t_mw = i[1]-i[0]
        n += 1

    # Formatting
    plt.ylabel("Population",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(
        f"Comparisson between Model 3 and Model 2 Hyperfine evolution\n B={b:.2f} G, $\\theta_B$={Fraction(t/np.pi).limit_denominator()}$\\pi$, $\\phi_B$={Fraction(p/np.pi).limit_denominator()}$\\pi$, $\\omega={om:.2f}$ MHz",
        fontsize=25)
    plt.ylim(np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y)
    t_lim = (np.min(np.append(times_doh,times_dua))-eps_x, np.max(np.append(times_doh,times_dua)) + eps_x)
    plt.xlim(t_lim)
    
    # Create simplified legend
    handles = []
    # Add electronic state legend
    for idx, color in enumerate(e_colors):
        handles.append(Line2D([], [], color=color, 
                                label=f"$n_{idx+1}$"))
    #handles.append(Line2D([], [], color="forestgreen", 
    #                        label=f"$n_c$"))
    # Add nuclear spin legend
    handles.append(Line2D([], [], color='black', linestyle='-',marker=n_marker[0],
                            label="$n_i |-1/2\\rangle$"))
    handles.append(Line2D([], [], color='black', linestyle='-',marker=n_marker[1],
                            label="$n_i |+1/2\\rangle$"))
    handles.append(Line2D([], [], color='black', linestyle=m_style[0],
                            label="Model 3"))
    handles.append(Line2D([], [], color='black', linestyle=m_style[1],linewidth=3,
                            label="Model 2"))
    # Add sequence legend
    handles.append(Rectangle((0,0),1,1, fc="palegreen", alpha=0.3,
                               label="Laser ON"))
    handles.append(Rectangle((0,0),1,1, fc="lightblue", alpha=0.3,
                               label="Free Evolution"))
    handles.append(Rectangle((0,0),1,1, fc="orchid", alpha=0.3,
                               label="MW ON"))

    plt.legend(handles=handles, bbox_to_anchor=(1.05, 1), 
             loc='upper left', borderaxespad=0.,fontsize=25)
    plt.minorticks_on()
    
    plt.show()

In [ ]:
def plot_bomrs_14_doh_vs_dua(n_exp_doh, times_doh, tis_doh, tfs_doh, 
                                      n_exp_dua, times_dua, tis_dua, tfs_dua, 
                                      b, om_r):
    """
    Plots population dynamics for 14 levels (7 electronic states × 2 nuclear spin states)
    comparing two datasets (Model 3 vs. Model 2).
    """
    # Define colors for electronic states (7 colors)
    e_colors = [
        "dodgerblue", "chocolate", "darkgoldenrod", 
        "mediumpurple", "mediumseagreen", "lightskyblue", "magenta"
    ]
    
    # Line styles for nuclear spin states
    n_marker = ['v', '^']
    m_style = ['-', '--']
    # Build a list of indices:
    # - For the first 5000 points, mark every 500.
    markers1 = list(range(0, 5000, 500))
    # - For the next 1000 points (5000–5999), mark every 400.
    markers2 = list(range(5000, 6000, 400))
    # - For the last 1000 points (6000–6999), mark every 100.
    markers3 = list(range(6000, 7000, 100))

    mark_indices = markers1 + markers2 + markers3
    plt.figure(figsize=(14, 8))

    # Plot Model 3 data
    for i in range(14):
        e_idx = i // 2 # Electronic state index (0-6)
        n_idx = i % 2   # Nuclear spin index (0-1)
        
        plt.plot(times_doh, n_exp_doh[i], 
                 color=e_colors[e_idx], 
                 linestyle=m_style[0],
                 marker=n_marker[n_idx],
                 markersize=8,markevery=mark_indices,
                 label=f"$n^{{doh}}_{{{e_idx+1},{n_idx}}}$")
    #for k in range(2):
    #    plt.plot(times_doh, n_exp_doh[14+k], 
    #                color='forestgreen', 
    #                linestyle='-',
    #                marker=n_marker[n_idx],
    #                markersize=4,markevery=50,
    #                linewidth=2,
    #                label=f'$n^{{doh}}_{{{{c}},{n_idx}}}')
    # Plot Model 2 data with thicker dashed lines
    for i in range(14):
        e_idx = i // 2
        n_idx = i % 2
        
        plt.plot(times_dua, n_exp_dua[i], 
                 color=e_colors[e_idx], 
                 linestyle=m_style[1],
                 marker=n_marker[n_idx],
                 markersize=8,markevery=mark_indices,
                 linewidth=3, alpha=0.7,
                 label=f"$n^{{dua}}_{{{e_idx+1},{n_idx}}}$")
    #for k in range(2):
    #    plt.plot(times_dua, n_exp_dua[14+k], 
    #                color='forestgreen', 
    #                linestyle='--',
    #                marker=n_marker[n_idx],
    #                markersize=4,markevery=90,
    #                linewidth=2,
    #                label=f'$n^{{dua}}_{{{{c}},{n_idx}}}')
    # Highlight sequence phases
    n = 0
    
    eps_y = 0.05 * (np.max(np.concatenate([n_exp_doh, n_exp_dua])) - 
                   np.min(np.concatenate([n_exp_doh, n_exp_dua])))
    eps_x = 0.05 * (np.max(np.concatenate([times_doh, times_dua])) - 
                   np.min(np.concatenate([times_doh, times_dua])))
    for i in zip(tis_doh, tfs_doh):
        if n % 3 == 0:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="palegreen", alpha=0.3,
            label="Laser ON" if n == 0 else "")
            t_laser = i[1]-i[0]
        elif (n-1) % 3 == 0:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="lightblue", alpha=0.3,
            label="Free Evolution" if n == 1 else "")
            t_free = i[1]-i[0]
        else:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="orchid", alpha=0.3,
            label="MW ON" if n == 2 else "")
            t_mw = i[1]-i[0]
        n += 1

    # Formatting
    plt.ylabel("Population",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"Comparisson between Model 3 and Model 2 Hyperfine evolution\n B={b:.2f} G, $\\Omega_r$={om_r:.2f} MHz",fontsize=25)
    plt.ylim(np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y)
    t_lim = (np.min(np.append(times_doh,times_dua))-eps_x, np.max(np.append(times_doh,times_dua)) + eps_x)
    plt.xlim(t_lim)
    
    # Create simplified legend
    handles = []
    # Add electronic state legend
    for idx, color in enumerate(e_colors):
        handles.append(Line2D([], [], color=color, 
                                label=f"$n_{idx+1}$"))
    #handles.append(Line2D([], [], color="forestgreen", label=f"$n_c$"))
    # Add nuclear spin legend
    handles.append(Line2D([], [], color='black', linestyle='-',marker=n_marker[0],
                            label="$n_i |-1/2\\rangle$"))
    handles.append(Line2D([], [], color='black', linestyle='-',marker=n_marker[1],
                            label="$n_i |+1/2\\rangle$"))
    handles.append(Line2D([], [], color='black', linestyle=m_style[0],
                            label="Model 3"))
    handles.append(Line2D([], [], color='black', linestyle=m_style[1],linewidth=3,
                            label="Model 2"))
    # Add sequence legend
    handles.append(Rectangle((0,0),1,1, fc="palegreen", alpha=0.3,
                               label="Laser ON"))
    handles.append(Rectangle((0,0),1,1, fc="lightblue", alpha=0.3,
                               label="Free Evolution"))
    handles.append(Rectangle((0,0),1,1, fc="orchid", alpha=0.3,
                               label="MW ON"))

    plt.legend(handles=handles, bbox_to_anchor=(1.05, 1), 
             loc='upper left', borderaxespad=0.,fontsize=25)
    plt.minorticks_on()
    
    plt.show()

### Plots

In [ ]:
results_bomr_14=np.load('NumpyArrays\\results_bomr_14.npy',allow_pickle=True)

# Now plot the results:
for b, omr, Model_3_14, Model_2_14 in results_bomr_14:
    plot_bomrs_14_doh_vs_dua(Model_3_14[0], Model_3_14[1], Model_3_14[2], Model_3_14[3],
                                Model_2_14[0], Model_2_14[1], Model_2_14[2], Model_2_14[3],
                                b, omr)
plt.close('all')

In [ ]:
results_bang_14=np.load("NumpyArrays\\results_bang_14.npy",allow_pickle=True)
# For loky_pmap, you could do:
# from qutip.parallel import loky_pmap
# results = loky_pmap(run_simulation, params, progress_bar="tqdm", num_cpus=8)

# Now plot the results:
for b, p, t, om, Model_3_14, Model_2_14 in results_bang_14:
    plot_bang_14_doh_vs_dua(Model_3_14[0], Model_3_14[1], Model_3_14[2], Model_3_14[3],
                                Model_2_14[0], Model_2_14[1], Model_2_14[2], Model_2_14[3],
                                b, p, t, om)
plt.close('all')

## 7 Levels

In [ ]:
def plot_hf_vs_no_bangs(n_exp_hf,times_hf,tis_hf,tfs_hf,n_exp,times,tis,tfs,b,p,t,om):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    hf="{hf}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    #plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1], lw=5,ls='--')
    #plt.plot(times_hf, n_exp_hf[-1], label="$n^{hf}_c$", color=colors[-1], lw=5)
    for i in range(len(n_exp_hf) - 1):
        plt.plot(times_hf, n_exp_hf[i], label=f"$n^{hf}_{i+1}$", color=colors[i], lw=5)
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i], lw=5,ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp,n_exp_hf)) - np.min(np.append(n_exp,n_exp_hf)))*5e-2
    eps_x = abs(np.max(np.append(times,times_hf)) - np.min(np.append(times,times_hf)))*5e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(
        f"Comparisson between Hyperfine and No-HF evolution\n B={b:.2f} G, $\\theta_B={Fraction(t/np.pi).limit_denominator()}\\pi$,$\\phi_B={Fraction(p/np.pi).limit_denominator()}\\pi$, $\\omega={om:.2f}$ MHz",
        fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times,times_hf))-eps_x, np.max(np.append(times,times_hf)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pops-hf_v_no-b_{int(b)}-phi_{int(100*p)}-theta_{int(100*t)}-om_{int(100*om)}.png",
                bbox_inches="tight",dpi=400)
    plt.show()

In [ ]:
def plot_hf_vs_no_bomrs(n_exp_hf,times_hf,tis_hf,tfs_hf,n_exp,times,tis,tfs,b,om):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    hf="{hf}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    #plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1], lw=5,ls='--')
    #plt.plot(times_hf, n_exp_hf[-1], label="$n^{hf}_c$", color=colors[-1], lw=5)
    for i in range(len(n_exp_hf) - 1):
        plt.plot(times_hf, n_exp_hf[i], label=f"$n^{hf}_{i+1}$", color=colors[i], lw=5)
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i], lw=5,ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp,n_exp_hf)) - np.min(np.append(n_exp,n_exp_hf)))*5e-2
    eps_x = abs(np.max(np.append(times,times_hf)) - np.min(np.append(times,times_hf)))*5e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"Comparisson between Hyperfine and No-HF evolution\n B={b:.2f} G, $\\Omega_r$={om:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times,times_hf))-eps_x, np.max(np.append(times,times_hf)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pop-hf_vs_no-b_{int(b)}_om_{int(100*om)}.png",bbox_inches="tight",dpi=400)
    plt.show()

In [ ]:
def plot_bang_doh_vs_dua(n_exp_doh,times_doh,tis_doh,tfs_doh,n_exp_dua,times_dua,tis_dua,tfs_dua,b,p,t,om):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    doh="{doh}"
    dua="{dua}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    #plt.plot(times_doh, n_exp_doh[-1], label="$n^{doh}_c$", color=colors[-1], lw=5)
    #plt.plot(times_dua, n_exp_dua[-1], label=f"$n^{dua}_c$", color=colors[-1], lw=5,ls='--')
    for i in range(len(n_exp_doh) - 1):
        plt.plot(times_doh, n_exp_doh[i], label=f"$n^{doh}_{i+1}$", color=colors[i], lw=5)
    for i in range(len(n_exp_dua) - 1):
        plt.plot(times_dua, n_exp_dua[i], label=f"$n^{dua}_{i+1}$", color=colors[i], lw=5,ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp_doh,n_exp_dua)) - np.min(np.append(n_exp_doh,n_exp_dua)))*5e-2
    eps_x = abs(np.max(np.append(times_doh,times_dua)) - np.min(np.append(times_doh,times_dua)))*5e-2
    for i in zip(tis_doh, tfs_doh):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(
        f"Comparisson between Model 3 and Model 2 Hyperfine evolution\n B={b:.2f} G, $\\theta_B$={Fraction(t/np.pi).limit_denominator()}$\\pi$, $\\phi_B$={Fraction(p/np.pi).limit_denominator()}$\\pi$, $\\omega={om:.2f}$ MHz",
        fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times_doh,times_dua))-eps_x, np.max(np.append(times_doh,times_dua)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pop-doh-v-dua-b-{int(b)}-phi-{int(100*p)}-theta-{int(100*t)}-om-{int(100*om)}.png",
                bbox_inches='tight',dpi=400)
    plt.show()

In [ ]:
def plot_bomr_doh_vs_dua(n_exp_doh,times_doh,tis_doh,tfs_doh,n_exp_dua,times_dua,tis_dua,tfs_dua,b,om_r):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    doh="{doh}"
    dua="{dua}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    #plt.plot(times_doh, n_exp_doh[-1], label="$n^{doh}_c$", color=colors[-1], lw=5)
    #plt.plot(times_dua, n_exp_dua[-1], label=f"$n^{dua}_c$", color=colors[-1], lw=5,ls='--')
    for i in range(len(n_exp_doh) - 1):
        plt.plot(times_doh, n_exp_doh[i], label=f"$n^{doh}_{i+1}$", color=colors[i], lw=5)
    for i in range(len(n_exp_dua) - 1):
        plt.plot(times_dua, n_exp_dua[i], label=f"$n^{dua}_{i+1}$", color=colors[i], lw=5,ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp_doh,n_exp_dua)) - np.min(np.append(n_exp_doh,n_exp_dua)))*5e-2
    eps_x = abs(np.max(np.append(times_doh,times_dua)) - np.min(np.append(times_doh,times_dua)))*5e-2
    for i in zip(tis_doh, tfs_doh):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"Comparisson between Model 3 and Model 2 Hyperfine evolution\n B={b:.2f} G, $\\Omega_r$={om_r:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times_doh,times_dua))-eps_x, np.max(np.append(times_doh,times_dua)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pop-doh-v-dua-B_{int(b)}-omr_{int(100*om_r)}.png",bbox_inches='tight',dpi=400)
    plt.show()

### Plots

In [ ]:
results=np.load("NumpyArrays\\results_bomr.npy",allow_pickle=True)

# Example usage:
# Suppose you have data1 and data2, each with length 300
# And you define three regions:
regions = [(0, 5000), (5000, 6000), (6000, 7000)]
labels  = ["Laser", "Free", "MW"]

# Then call with your actual data:
# summary = compare_regions(data1, data2, regions, labels)


# Example:
# print_summary_table(summary)
# Now plot the results:
for b, omr, Model_3, Model_2, norm in results:
    plot_bomr_doh_vs_dua(Model_3[0], Model_3[1], Model_3[2], Model_3[3],Model_2[0], Model_2[1], Model_2[2], Model_2[3],b, omr)
    # compare deviations nexps of model 3 vs model 2
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    print("M3-M2 $Delta n_i")
    plt.figure(figsize=(14, 8))
    for i in range(len(Model_3[0])-1):
        labels  = [f"Laser(n_{i+1})", f"Free(n_{i+1})", f"MW(n_{i+1})"]
        summary = compare_regions(Model_3[0][i], Model_2[0][i], regions, labels)
        print_summary_table(summary)
        plt.plot(Model_3[1], Model_3[0][i]-Model_2[0][i], label=rf"$\Delta n_{i}$", color=colors[i], lw=5)
    labels  = ["Laser(n_c)", "Free(n_c)", "MW(n_c)"]
    summary = compare_regions(Model_3[0][-1], Model_2[0][-1], regions, labels)
    print_summary_table(summary)
    plt.plot(Model_3[1], Model_3[0][-1]-Model_2[0][-1], label=r"$\Delta n_c$", color=colors[-1], lw=5)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    plot_hf_vs_no_bomrs(Model_3[0], Model_3[1], Model_3[2], Model_3[3],norm[0], norm[1], norm[2], norm[3],b, omr)
    print("M3-N $Delta n_i")
    plt.figure(figsize=(14, 8))
    for i in range(len(norm[0])-1):
        labels  = [f"Laser(n_{i+1})", f"Free(n_{i+1})", f"MW(n_{i+1})"]
        summary = compare_regions(Model_3[0][i], norm[0][i], regions, labels)
        print_summary_table(summary)
        plt.plot(Model_3[1], Model_3[0][i]-norm[0][i], label=rf"$\Delta n_{i}$", color=colors[i], lw=5)
    labels  = ["Laser(n_c)", "Free(n_c)", "MW(n_c)"]
    summary = compare_regions(Model_3[0][-1], norm[0][-1], regions, labels)
    print_summary_table(summary)
    plt.plot(Model_3[1], Model_3[0][-1]-norm[0][-1], label=r"$\Delta n_c$", color=colors[-1], lw=5)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()

#### Bang

In [ ]:
results_bang=np.load("NumpyArrays\\results_bang.npy",allow_pickle=True)

# Now plot the results:
for b, p, t, om, Model_3, Model_2, norm in results_bang:
    plot_bang_doh_vs_dua(Model_3[0], Model_3[1], Model_3[2], Model_3[3],
                                Model_2[0], Model_2[1], Model_2[2], Model_2[3],
                                b, p, t, om)
    
    print("M3-M2 $Delta n_i")
    plt.figure(figsize=(14, 8))
    for i in range(len(Model_3[0]-1)):
        labels  = [f"Laser(n_{i+1})", f"Free(n_{i+1})", f"MW(n_{i+1})"]
        summary = compare_regions(Model_3[0][i], Model_2[0][i], regions, labels)
        print_summary_table(summary)
        plt.plot(Model_3[1], Model_3[0][i]-Model_2[0][i], label=rf"$\Delta n_{i}$", color=colors[i], lw=5)
    labels  = ["Laser(n_c)", "Free(n_c)", "MW(n_c)"]
    summary = compare_regions(Model_3[0][i], Model_2[0][i], regions, labels)
    print_summary_table(summary)
    plt.plot(Model_3[1], Model_3[0][i]-Model_2[0][i], label=r"$\Delta n_c$", color=colors[-1], lw=5)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    plot_hf_vs_no_bangs(Model_3[0], Model_3[1], Model_3[2], Model_3[3],
                               norm[0], norm[1], norm[2], norm[3],
                               b, p, t, om)
    print("M3-N $Delta n_i")
    plt.figure(figsize=(14, 8))
    for i in range(len(norm[0]-1)):
        labels  = [f"Laser(n_{i+1})", f"Free(n_{i+1})", f"MW(n_{i+1})"]
        summary = compare_regions(Model_3[0][i], norm[0][i], regions, labels)
        print_summary_table(summary)
        plt.plot(Model_3[1], Model_3[0][i]-norm[0][i], label=rf"$\Delta n_{i}$", color=colors[i], lw=5)
    labels  = ["Laser(n_c)", "Free(n_c)", "MW(n_c)"]
    summary = compare_regions(Model_3[0][i], norm[0][i], regions, labels)
    print_summary_table(summary)
    plt.plot(Model_3[1], Model_3[0][i]-norm[0][i], label=r"$\Delta n_c$", color=colors[-1], lw=5)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
plt.close('all')

# Photoluminescence (PL)

## Magaletti PL

In [ ]:
result_pls_w = np.load("NumpyArrays\\results_pl_w.npy",allow_pickle=True)
dynam=np.array([])
o=0
for dyn,w_p,rest in zip(result_pls_w[0::3],result_pls_w[1::3],result_pls_w[2::3]):
    b=B0(100,0,0)
    om_r=15.7
    c=0
    print(dyn)
    for res in rest:
        #for j in range(len(res[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(res)):
        #        nn_exp = np.append(nn_exp, res[i].expect[j])
        #    if j == 0:
        #        n_exp_t = np.array([nn_exp])
        #    else:
        #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        #    # Gather the times for the plots
        N=[n4&nit[0]*nit[0].dag(),n5&nit[0]*nit[0].dag(),n6&nit[0]*nit[0].dag(),#type:ignore
           n4&nit[1]*nit[1].dag(),n5&nit[1]*nit[1].dag(),n6&nit[1]*nit[1].dag(),#type:ignore
           n4&IdN15,n5&IdN15,n6&IdN15]
        n_exp_t = np.array([np.concatenate([qt.expect(M,ress.states) for ress in res]) for M in N])
        times_t = np.array([])
        for i in range(len(res)):
            times_t = np.append(times_t, res[i].times)
        pl=np.array([])
        pl=np.add(n_exp_t[0],n_exp_t[1])
        pl=np.add(pl,n_exp_t[2])
        pl=np.add(pl,n_exp_t[3])
        pl=np.add(pl,n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        pl1=np.array([])
        pl1=np.add(n_exp_t[6],n_exp_t[7])
        pl1=np.add(pl1,n_exp_t[8])
        if c==0:
            pls1 = np.array([pl1])
            pls = np.array([pl])
            c+=1
        else:
            pls = np.append(pls,[pl],axis=0)
            pls1 = np.append(pls1,[pl1],axis=0)
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    if o==0:
        pls_0=np.array([pls1[0]])
        pls_1=np.array([pls1[2]])
        o+=1
    else:
        pls_0=np.append(pls_0,[pls1[0]],axis=0)
        pls_1=np.append(pls_1,[pls1[2]],axis=0)
    dynam=np.append(dynam,dyn)
    for pl in pls:
    #    plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a],ls='--',lw=5 label=f"PL no-trace init state=${state[a]}$")
        a+=1
    for pl in pls1:    
        plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a-3],lw=5, label=f"PL init state=${state[a-3]}$")
        a+=1
    
    eps_y = abs(np.min(pls1) - np.max(pls1))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"{dyn} \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz, $W_p$={w_p:.2f} ",fontsize=25)
    
    # set the time limits you want to show in the plot
    t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
    plt.xlim(t_lim)
    plt.ylim((np.min(pls1) - eps_y, np.max(pls1) + eps_y))
    plt.minorticks_on()
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pl-{dyn[:3].lower()}-b-{int(b[2])}-w-{int(100*w_p)}.png",bbox_inches='tight',
                dpi=400)
    plt.show()
ws=np.linspace(0.5, 4, 8)
print(pls_0.shape)
plt.figure(figsize=(14, 8))
for pl,w in zip(pls_0[0::2],ws):
    plt.plot(times_t[-5200:]-11,pl[-5200:], lw=5, label=f"$W_p={w}$")
    a+=1


eps_y = abs(np.min(pls_0) - np.max(pls_0))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL (a.u.)",fontsize=25)
plt.xlabel(r"Time ($\mu$s)",fontsize=25)
plt.title(f"{dynam[0]} \n PL init state=$|0\\rangle$ \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz",fontsize=25)

# set the time limits you want to show in the plot
t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
plt.xlim(t_lim)
plt.ylim((np.min(pls_0) - eps_y, np.max(pls_0) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
plt.savefig(f"Figs\\pl-{dynam[0][:3].lower()}-s0-b-{int(b[2])}-ws-omr-{int(100*om_r)}.png",
            bbox_inches='tight',dpi=400)
plt.show()

plt.figure(figsize=(14, 8))
for pl,w in zip(pls_0[1::2],ws):
    plt.plot(times_t[-5200:]-11,pl[-5200:], lw=5, label=f"$W_p={w}$")
    a+=1


eps_y = abs(np.min(pls_0) - np.max(pls_1))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL (a.u.)",fontsize=25)
plt.xlabel(r"Time ($\mu$s)",fontsize=25)
plt.title(f"{dynam[1]} \n PL init state=$|0\\rangle$ \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz",fontsize=25)

# set the time limits you want to show in the plot
t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
plt.xlim(t_lim)
plt.ylim((np.min(pls_0) - eps_y, np.max(pls_0) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
plt.savefig(f"Figs\\pl-{dynam[1][:3].lower()}-s0-b-{int(b[2])}-ws-omr-{int(100*om_r)}.png",
            bbox_inches='tight',dpi=400)
plt.show()

plt.figure(figsize=(14, 8))
for pl,w in zip(pls_1[0::2],ws):
    plt.plot(times_t[-5200:]-11,pl[-5200:], lw=5, label=f"$W_p={w}$")
    a+=1


eps_y = abs(np.min(pls_0) - np.max(pls_0))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL (a.u.)",fontsize=25)
plt.xlabel(r"Time ($\mu$s)",fontsize=25)
plt.title(f"{dynam[0]} \n PL init state=$|1\\rangle$ \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz",fontsize=25)

# set the time limits you want to show in the plot
t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
plt.xlim(t_lim)
plt.ylim((np.min(pls_0) - eps_y, np.max(pls_0) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
plt.savefig(f"Figs\\pl-{dynam[0][:3].lower()}-s1-b-{int(b[2])}-ws-omr-{int(100*om_r)}.png",
            bbox_inches='tight',dpi=400)
plt.show()

plt.figure(figsize=(14, 8))
for pl,w in zip(pls_1[1::2],ws):
    plt.plot(times_t[-5200:]-11,pl[-5200:], lw=5, label=f"$W_p={w}$")
    a+=1


eps_y = abs(np.min(pls_0) - np.max(pls_0))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL (a.u.)",fontsize=25)
plt.xlabel(r"Time ($\mu$s)",fontsize=25)
plt.title(f"{dynam[1]} \n PL init state=$|1\\rangle$ \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz",fontsize=25)

# set the time limits you want to show in the plot
t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
plt.xlim(t_lim)
plt.ylim((np.min(pls_0) - eps_y, np.max(pls_0) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
plt.savefig(f"Figs\\pl-{dynam[1][:3].lower()}-s1-b-{int(b[2])}-ws-omr-{int(100*om_r)}.png",
            bbox_inches='tight',dpi=400)
plt.show()
plt.close('all')

### Magnetic x Omr

In [ ]:
result_pls_bomr=np.load("NumpyArrays\\results_pl_bomr.npy",allow_pickle=True)  
for dyn,b0,om_r,rest in zip(result_pls_bomr[0::4],result_pls_bomr[1::4],result_pls_bomr[2::4],result_pls_bomr[3::4]):
    b=B0(b0,0,0)
    w_p=1.9
    c=0
    for res in rest:
        #for j in range(len(res[0].expect)):
        #        nn_exp = np.array([])
        #        for i in range(len(res)):
        #            nn_exp = np.append(nn_exp, res[i].expect[j])
        #        if j == 0:
        #            n_exp_t = np.array([nn_exp])
        #        else:
        #            n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        #    # Gather the times for the plots
        N=[n4&nit[0]*nit[0].dag(),n5&nit[0]*nit[0].dag(),n6&nit[0]*nit[0].dag(),#type:ignore
           n4&nit[1]*nit[1].dag(),n5&nit[1]*nit[1].dag(),n6&nit[1]*nit[1].dag(),#type:ignore
           n4&IdN15,n5&IdN15,n6&IdN15]
        n_exp_t = np.array([np.concatenate([qt.expect(M,ress.states) for ress in res]) for M in N])
        times_t = np.array([])
        for i in range(len(res)):
            times_t = np.append(times_t, res[i].times)
        pl=np.array([])
        pl=np.add(n_exp_t[0],n_exp_t[1])
        pl=np.add(pl,n_exp_t[2])
        pl=np.add(pl,n_exp_t[3])
        pl=np.add(pl,n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        pl1=np.array([])
        pl1=np.add(n_exp_t[6],n_exp_t[7])
        pl1=np.add(pl1,n_exp_t[8])
        if c==0:
            pls1 = np.array([pl1])
            pls = np.array([pl])
            c+=1
        else:
            pls = np.append(pls,[pl],axis=0)
            pls1 = np.append(pls1,[pl1],axis=0)
        # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls:
        #plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a],ls='--' label=f"PL init state=${state[a]}$")
        a+=1
    for pl in pls1:    
        plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a-3],lw=5, label=f"PL1 init state=${state[a-3]}$")
        a+=1
    
    eps_y = abs(np.min(pls1) - np.max(pls1))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"{dyn} \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz, $W_p$={w_p:.2f} ",fontsize=25)
    
    # set the time limits you want to show in the plot
    t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
    plt.xlim(t_lim)
    plt.ylim((np.min(pls1) - eps_y, np.max(pls1) + eps_y))
    plt.minorticks_on()
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pl-{dyn[:3].lower()}-b-{int(b[2])}-omr-{int(100*om_r)}.png",bbox_inches='tight',
                dpi=400)
    plt.show()
plt.close('all')

## Hirose PL

In [ ]:
ks=[[66.0,0.0,57.0,1.0,0.7],
    [77.0,0.0,30.0,3.3,0.0],
    [62.7,12.97,80.0,3.45,1.08],
    [63.2,10.8,60.7,0.8,0.4],
    [67.4,9.9,96.6,4.83,1.055],
    [64.0,11.8,79.8,5.6,0.0]]

In [ ]:
result_pls_w = np.load("NumpyArrays\\results_pl_w.npy",allow_pickle=True)
dynam=np.array([])
o=0
for dyn,w_p,rest in zip(result_pls_w[0::3],result_pls_w[1::3],result_pls_w[2::3]):
    b=B0(100,0,0)
    om_r=15.7
    c=0
    for res in rest:
        #for j in range(len(res[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(res)):
        #        nn_exp = np.append(nn_exp, res[i].expect[j])
        #    if j == 0:
        #        n_exp_t = np.array([nn_exp])
        #    else:
        #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        #    # Gather the times for the plots
        N=[n4&nit[0]*nit[0].dag(),n5&nit[0]*nit[0].dag(),n6&nit[0]*nit[0].dag(),#type:ignore
           n4&nit[1]*nit[1].dag(),n5&nit[1]*nit[1].dag(),n6&nit[1]*nit[1].dag(),#type:ignore
           n4&IdN15,n5&IdN15,n6&IdN15]
        n_exp_t = np.array([np.concatenate([qt.expect(M,ress.states) for ress in res]) for M in N])
        times_t = np.array([])
        for i in range(len(res)):
            times_t = np.append(times_t, res[i].times)
        pl=np.array([])
        pl=np.add(n_exp_t[0],n_exp_t[1])
        pl=np.add(pl,n_exp_t[2])
        pl=np.add(pl,n_exp_t[3])
        pl=np.add(pl,n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        pl1=np.array([])
        pl1=np.add(ks[5][0]*(1+2*0.01)*n_exp_t[6]/(ks[5][0]+ks[5][1]),ks[5][0]*(1+2*0.01)*n_exp_t[7]/(ks[5][0]+ks[5][2]))
        pl1=np.add(pl1,ks[5][0]*(1+2*0.01)*n_exp_t[8]/(ks[5][0]+ks[5][2]))
        if c==0:
            pls1 = np.array([pl1])
            pls = np.array([pl])
            c+=1
        else:
            pls = np.append(pls,[pl],axis=0)
            pls1 = np.append(pls1,[pl1],axis=0)
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    if o==0:
        pls_0=np.array([pls1[0]])
        pls_1=np.array([pls1[2]])
        o+=1
    else:
        pls_0=np.append(pls_0,[pls1[0]],axis=0)
        pls_1=np.append(pls_1,[pls1[2]],axis=0)
    dynam=np.append(dynam,dyn)
    for pl in pls:
        plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a], lw=5, label=f"PL Magaletti \nInit state=${state[a]}$")
        a+=1
    for pl in pls1:    
        plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a-3],ls="--",lw=5, label=f"PL Hirose \nInit state=${state[a-3]}$")
        a+=1
    
    eps_y = abs(np.min(np.append(pls1,pls)) - np.max(np.append(pls1,pls)))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"{dyn} \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz, $W_p$={w_p:.2f} ",fontsize=25)
    
    # set the time limits you want to show in the plot
    t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(pls1,pls)) - eps_y, np.max(np.append(pls1,pls)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pl-hirose-{dyn[:3].lower()}-b-{int(b[2])}-w-{int(100*w_p)}.png",bbox_inches='tight',
                dpi=400)
    plt.show()
plt.close('all')

### Magnetic x Omr

In [ ]:
result_pls_bomr=np.load("NumpyArrays\\results_pl_bomr.npy",allow_pickle=True)  
for dyn,b0,om_r,rest in zip(result_pls_bomr[0::4],result_pls_bomr[1::4],result_pls_bomr[2::4],result_pls_bomr[3::4]):
    b=B0(b0,0,0)
    w_p=1.9
    c=0
    for res in rest:
        #for j in range(len(res[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(res)):
        #        nn_exp = np.append(nn_exp, res[i].expect[j])
        #    if j == 0:
        #        n_exp_t = np.array([nn_exp])
        #    else:
        #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        #    # Gather the times for the plots
        N=[n4&nit[0]*nit[0].dag(),n5&nit[0]*nit[0].dag(),n6&nit[0]*nit[0].dag(),#type:ignore
           n4&nit[1]*nit[1].dag(),n5&nit[1]*nit[1].dag(),n6&nit[1]*nit[1].dag(),#type:ignore
           n4&IdN15,n5&IdN15,n6&IdN15]
        n_exp_t = np.array([np.concatenate([qt.expect(M,ress.states) for ress in res]) for M in N])
            # Gather the times for the plots
        times_t = np.array([])
        for i in range(len(res)):
            times_t = np.append(times_t, res[i].times)
        pl=np.array([])
        pl=np.add(n_exp_t[0],n_exp_t[1])
        pl=np.add(pl,n_exp_t[2])
        pl=np.add(pl,n_exp_t[3])
        pl=np.add(pl,n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        pl1=np.array([])
        pl1=np.add(ks[2][0]*n_exp_t[6]/(ks[2][0]+ks[2][1]),ks[2][0]*n_exp_t[7]/(ks[2][0]+ks[2][2]))
        pl1=np.add(pl1,ks[2][0]*n_exp_t[8]/(ks[2][0]+ks[2][2]))
        if c==0:
            pls1 = np.array([pl1])
            pls = np.array([pl])
            c+=1
        else:
            pls = np.append(pls,[pl],axis=0)
            pls1 = np.append(pls1,[pl1],axis=0)
        # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls:
        plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a],lw=5, label=f"PL Magaletti \nInit state=${state[a]}$")
        a+=1
    for pl in pls1:    
        plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a-3],lw=5, ls="--", label=f"PL Hirose \nInit state=${state[a-3]}$")
        a+=1
    
    eps_y = abs(np.min(np.append(pls1,pls)) - np.max(np.append(pls1,pls)))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"{dyn} \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz, $W_p$={w_p:.2f} ",fontsize=25)
    
    # set the time limits you want to show in the plot
    t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(pls1,pls)) - eps_y, np.max(np.append(pls1,pls)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pl-hirose-{dyn[:3].lower()}-b-{int(b[2])}-omr-{int(100*om_r)}.png",
                bbox_inches='tight',dpi=400)
    plt.show()
plt.close('all')

In [ ]:
#assert(1==2)

# ODMR

In [ ]:
n_p_odmr=1.5
n_p_ramsey=1.5
w_p_odmr=n_p_odmr*1.9
w_p_ramsey=n_p_ramsey*1.9
om_r_odmr=15.7

In [ ]:
for b_odmr in [0,100,500,1000]:
    flur_codm,ws_codm =(
        np.load(f'NumpyArrays\\flur_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_codm,ws_doh_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_codm,ws_dua_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_podm,ws_podm =(
        np.load(f'NumpyArrays\\flur_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_podm,ws_doh_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_podm,ws_dua_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    # CW - ODMR
    ep=0.02
    norm_flur=max_reg(flur_codm)
    norm_flur_doh_hf=max_reg(flur_doh_hf_codm)-ep
    norm_flur_dua_hf=max_reg(flur_dua_hf_codm)-2*ep
    plt.figure(figsize=(14, 8))
    if len(flur_codm)>30 and not np.isclose(b_odmr,0):
        y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='red',fontsize=25)
    plt.title(f"CW-ODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    plt.plot(ws_codm,norm_flur,label="No-HF",color="green")
    plt.plot(ws_doh_hf_codm,norm_flur_doh_hf,label="HF,Model 3",ls="--",color='blue')
    plt.plot(ws_dua_hf_codm,norm_flur_dua_hf,label="HF,Model 2",ls="-.",lw=5,color="purple")
    H=H_doh_hf(B0(b_odmr,0,0),0)[0]+H_doh_hf(B0(b_odmr,0,0),0)[1]
    h=qt.Qobj(H.full()[:12,:12])
    energies=h.eigenenergies()
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    
    plt.minorticks_on()
    plt.savefig(f'Figs\\cwODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    # PuODMR
    norm_flur=max_reg(flur_podm)
    norm_flur_doh_hf=max_reg(flur_doh_hf_podm)-ep
    norm_flur_dua_hf=max_reg(flur_dua_hf_podm)-2*ep
    plt.figure(figsize=(14, 8))
    if len(flur_podm)>30 and not np.isclose(b_odmr,0):
        y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='r',fontsize=25)
    plt.plot(ws_podm,norm_flur,label="No-HF",lw=5,color="green")
    plt.title(f"PuODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    plt.plot(ws_doh_hf_podm,norm_flur_doh_hf,label="HF,Model 3",lw=5,ls="--",color='purple')
    plt.plot(ws_dua_hf_podm,norm_flur_dua_hf,label="HF,Model 2",ls="-.",lw=5,color="blue")
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            print(f"E={energy}")
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.minorticks_on()
    
    plt.savefig(f'Figs\\puODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    # Comparison
    plt.figure(figsize=(14, 8))
    plt.title(f"B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    if not np.isclose(b_odmr,0):
        y,x1,x2=min(max_reg(flur_codm)),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='r',fontsize=25)
    plt.plot(ws_codm,max_reg(flur_codm),color='green',label="no-HF,CW-ODMR")
    plt.plot(ws_podm,max_reg(flur_podm),color='lime',label="no-HF,PuODMR")
    plt.plot(ws_doh_hf_codm,max_reg(flur_doh_hf_codm)-ep,color='purple',label="HF,CW-ODMR\nModel 3",ls="--")
    plt.plot(ws_doh_hf_podm,max_reg(flur_doh_hf_podm)-ep,color='mediumpurple',label="HF,PuODMR\nModel 3",ls="--")
    plt.plot(ws_dua_hf_codm,max_reg(flur_dua_hf_codm)-2*ep,color='blue',label="HF,CW-ODMR\nModel 2",ls="-.",lw=5)
    plt.plot(ws_dua_hf_podm,max_reg(flur_dua_hf_podm)-2*ep,color='steelblue',label="HF,PuODMR\nModel 2",ls="-.",lw=5)
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.minorticks_on()
    
    plt.savefig(f'Figs\\ODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()

## Omega T2

In [ ]:
n_p_odmr=1.5
n_p_ramsey=1.5
w_p_odmr=n_p_odmr*1.9
w_p_ramsey=n_p_ramsey*1.9
om_r_odmr=np.pi/t2_gs

In [ ]:
for b_odmr in [0,100,518]:
    flur_codm,ws_codm =(
        np.load(f'NumpyArrays\\flur_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_codm,ws_doh_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_codm,ws_dua_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_podm,ws_podm =(
        np.load(f'NumpyArrays\\flur_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_podm,ws_doh_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_podm,ws_dua_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    ts_podm=np.load(f'NumpyArrays\\ts_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ts_doh_hf_podm=np.load(f'NumpyArrays\\ts_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ts_dua_hf_podm=np.load(f'NumpyArrays\\ts_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_podm=np.load(f'NumpyArrays\\ns_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_doh_hf_podm=np.load (f'NumpyArrays\\ns_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_dua_hf_podm=np.load (f'NumpyArrays\\ns_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    #res_codm=np.load(f'NumpyArrays\\res_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_doh_hf_codm=np.load(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_dua_hf_codm=np.load(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_podm=np.load(f'NumpyArrays\\res_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_doh_hf_podm=np.load (f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_dua_hf_podm=np.load (f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    # CW - ODMR
    ep=0.02
    norm_flur=max_reg(flur_codm)
    norm_flur_doh_hf=max_reg(flur_doh_hf_codm)-ep
    norm_flur_dua_hf=max_reg(flur_dua_hf_codm)-2*ep
    plt.figure(figsize=(14, 8))
    if len(flur_codm)>30 and not np.isclose(b_odmr,0):
        y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='red',fontsize=25)
    plt.title(f"CW-ODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    plt.plot(ws_codm,norm_flur,label="No-HF",lw=5,color="green")
    plt.plot(ws_doh_hf_codm,norm_flur_doh_hf,label="HF,Model 3",ls="--",lw=5,color='purple')
    plt.plot(ws_dua_hf_codm,norm_flur_dua_hf,label="HF,Model 2",ls="-.",lw=5,color="blue")
    H=H_doh_hf(B0(b_odmr,0,0),0)[0]+H_doh_hf(B0(b_odmr,0,0),0)[1]
    h=qt.Qobj(H.full()[:12,:12])
    energies=h.eigenenergies()
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    
    plt.minorticks_on()
    plt.savefig(f'Figs\\cwODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    # PuODMR
    norm_flur=max_reg(flur_podm)
    norm_flur_doh_hf=max_reg(flur_doh_hf_podm)-ep
    norm_flur_dua_hf=max_reg(flur_dua_hf_podm)-2*ep
    plt.figure(figsize=(14, 8))
    if len(flur_podm)>30 and not np.isclose(b_odmr,0):
        y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='r',fontsize=25)
    plt.plot(ws_podm,norm_flur,label="No-HF",lw=5,color="green")
    plt.title(f"PuODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    plt.plot(ws_doh_hf_podm,norm_flur_doh_hf,label="HF,Model 3",ls="--",lw=5,color='purple')
    plt.plot(ws_dua_hf_podm,norm_flur_dua_hf,label="HF,Model 2",ls="-.",lw=5,color="blue")
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            print(f"E={energy}")
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.minorticks_on()
    
    plt.savefig(f'Figs\\puODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    # Comparison
    plt.figure(figsize=(14, 8))
    plt.title(f"B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    if not np.isclose(b_odmr,0):
        y,x1,x2=min(max_reg(flur_codm)),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='r',fontsize=25)
    plt.plot(ws_codm,max_reg(flur_codm),lw=5,color='green',label="no-HF,CW-ODMR")
    plt.plot(ws_podm,max_reg(flur_podm),lw=5,color='lime',label="no-HF,PuODMR")
    plt.plot(ws_doh_hf_codm,max_reg(flur_doh_hf_codm)-ep,lw=5,color='purple',label="HF,CW-ODMR\nModel 3",ls="--")
    plt.plot(ws_doh_hf_podm,max_reg(flur_doh_hf_podm)-ep,lw=5,color='mediumpurple',label="HF,PuODMR\nModel 3",ls="--")
    plt.plot(ws_dua_hf_codm,max_reg(flur_dua_hf_codm)-2*ep,color='blue',label="HF,CW-ODMR\nModel 2",ls="-.",lw=5)
    plt.plot(ws_dua_hf_podm,max_reg(flur_dua_hf_podm)-2*ep,color='steelblue',label="HF,PuODMR\nModel 2",ls="-.",lw=5)
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.minorticks_on()
    
    plt.savefig(f'Figs\\ODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
plt.close('all')

## Omega 2 T2

In [ ]:
n_p_odmr=1.5
n_p_ramsey=1.5
w_p_odmr=n_p_odmr*1.9
w_p_ramsey=n_p_ramsey*1.9
om_r_odmr=2*np.pi/t2_gs

In [ ]:
for b_odmr in [0,500]:
    flur_codm,ws_codm =(
        np.load(f'NumpyArrays\\flur_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_codm,ws_doh_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_codm,ws_dua_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_podm,ws_podm =(
        np.load(f'NumpyArrays\\flur_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_podm,ws_doh_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_podm,ws_dua_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    ts_podm=np.load(f'NumpyArrays\\ts_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ts_doh_hf_podm=np.load(f'NumpyArrays\\ts_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ts_dua_hf_podm=np.load(f'NumpyArrays\\ts_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_podm=np.load(f'NumpyArrays\\ns_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_doh_hf_podm=np.load (f'NumpyArrays\\ns_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_dua_hf_podm=np.load (f'NumpyArrays\\ns_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    #res_codm=np.load(f'NumpyArrays\\res_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_doh_hf_codm=np.load(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_dua_hf_codm=np.load(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_podm=np.load(f'NumpyArrays\\res_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_doh_hf_podm=np.load (f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_dua_hf_podm=np.load (f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    # CW - ODMR
    ep=0.02
    norm_flur=max_reg(flur_codm)
    norm_flur_doh_hf=max_reg(flur_doh_hf_codm)-ep
    norm_flur_dua_hf=max_reg(flur_dua_hf_codm)-2*ep
    plt.figure(figsize=(14, 8))
    if len(flur_codm)>30 and not np.isclose(b_odmr,0):
        y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='red',fontsize=25)
    plt.title(f"CW-ODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    plt.plot(ws_codm,norm_flur,label="No-HF",color="green",lw=5)
    plt.plot(ws_doh_hf_codm,norm_flur_doh_hf,label="HF,Model 3",ls="--",color='purple',lw=5)
    plt.plot(ws_dua_hf_codm,norm_flur_dua_hf,label="HF,Model 2",ls="-.",lw=5,color="blue")
    H=H_doh_hf(B0(b_odmr,0,0),0)[0]+H_doh_hf(B0(b_odmr,0,0),0)[1]
    h=qt.Qobj(H.full()[:12,:12])
    energies=h.eigenenergies()
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    
    plt.minorticks_on()
    plt.savefig(f'Figs\\cwODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    # PuODMR
    norm_flur=max_reg(flur_podm)
    norm_flur_doh_hf=max_reg(flur_doh_hf_podm)-ep
    norm_flur_dua_hf=max_reg(flur_dua_hf_podm)-2*ep
    plt.figure(figsize=(14, 8))
    if len(flur_podm)>30 and not np.isclose(b_odmr,0):
        y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='r',fontsize=25)
    plt.plot(ws_podm,norm_flur,label="No-HF",color="green",lw=5)
    plt.title(f"PuODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    plt.plot(ws_doh_hf_podm,norm_flur_doh_hf,label="HF,Model 3",ls="--",color='purple',lw=5)
    plt.plot(ws_dua_hf_podm,norm_flur_dua_hf,label="HF,Model 2",ls="-.",lw=5,color="blue")
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            print(f"E={energy}")
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.minorticks_on()
    
    plt.savefig(f'Figs\\puODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',bbox_inches='tight',dpi=400)
    plt.show()
    # Comparison
    plt.figure(figsize=(14, 8))
    plt.title(f"B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    if not np.isclose(b_odmr,0):
        y,x1,x2=min(max_reg(flur_codm)),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='r',fontsize=25)
    plt.plot(ws_codm,max_reg(flur_codm),color='green',lw=5,label="no-HF,CW-ODMR")
    plt.plot(ws_podm,max_reg(flur_podm),color='lime',lw=5,label="no-HF,PuODMR")
    plt.plot(ws_doh_hf_codm,max_reg(flur_doh_hf_codm)-ep,color='purple',lw=5,label="HF,CW-ODMR\nModel 3",ls="--")
    plt.plot(ws_doh_hf_podm,max_reg(flur_doh_hf_podm)-ep,color='mediumpurple',lw=5,label="HF,PuODMR\nModel 3",ls="--")
    plt.plot(ws_dua_hf_codm,max_reg(flur_dua_hf_codm)-2*ep,color='blue',lw=5,label="HF,CW-ODMR\nModel 2",ls="-.")
    plt.plot(ws_dua_hf_podm,max_reg(flur_dua_hf_podm)-2*ep,color='steelblue',lw=5,label="HF,PuODMR\nModel 2",ls="-.")
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.minorticks_on()
    
    plt.savefig(f"Figs\\ODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png",bbox_inches='tight',dpi=400)
    plt.show()
plt.close('all')

## E_ES 70

In [ ]:
n_p_odmr=1
w_p_odmr=n_p_odmr*1.9

In [ ]:
for b_odmr in [0,150,510]:#,1020]:
    if b_odmr==0:
        om_r_odmr=15.5
    else:
        om_r_odmr=15.7
    flur_codm,ws_codm =(
        np.load(f'NumpyArrays\\flur_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_codm,ws_doh_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_codm,ws_dua_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_podm,ws_podm =(
        np.load(f'NumpyArrays\\flur_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_podm,ws_doh_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_podm,ws_dua_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    ts_podm=np.load(f'NumpyArrays\\ts_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ts_doh_hf_podm=np.load(f'NumpyArrays\\ts_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ts_dua_hf_podm=np.load(f'NumpyArrays\\ts_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_podm=np.load(f'NumpyArrays\\ns_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_doh_hf_podm=np.load (f'NumpyArrays\\ns_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    ns_dua_hf_podm=np.load (f'NumpyArrays\\ns_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    #res_codm=np.load(f'NumpyArrays\\res_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_doh_hf_codm=np.load(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_dua_hf_codm=np.load(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_podm=np.load(f'NumpyArrays\\res_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_doh_hf_podm=np.load (f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    #res_dua_hf_podm=np.load (f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',allow_pickle=True)
    # CW - ODMR
    ep=0.02
    norm_flur=max_reg(flur_codm)
    norm_flur_doh_hf=max_reg(flur_doh_hf_codm)-ep
    norm_flur_dua_hf=max_reg(flur_dua_hf_codm)-2*ep
    plt.figure(figsize=(14, 8))
    if len(flur_codm)>30 and not np.isclose(b_odmr,0):
        y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='red',fontsize=25)
    plt.title(f"CW-ODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    plt.plot(ws_codm,norm_flur,label="No-HF",color="green",lw=5)
    plt.plot(ws_doh_hf_codm,norm_flur_doh_hf,label="HF,Model 3",ls="--",color='purple',lw=5)
    plt.plot(ws_dua_hf_codm,norm_flur_dua_hf,label="HF,Model 2",ls="-.",lw=5,color="blue")
    H=H_doh_hf(B0(b_odmr,0,0),0)[0]+H_doh_hf(B0(b_odmr,0,0),0)[1]
    h=qt.Qobj(H.full()[:12,:12])
    energies=h.eigenenergies()
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    
    plt.minorticks_on()
    plt.savefig(f'Figs\\cwODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',bbox_inches='tight',dpi=400)
    plt.show()
    # PuODMR
    norm_flur=max_reg(flur_podm)
    norm_flur_doh_hf=max_reg(flur_doh_hf_podm)-ep
    norm_flur_dua_hf=max_reg(flur_dua_hf_podm)-2*ep
    plt.figure(figsize=(14, 8))
    if len(flur_podm)>30 and not np.isclose(b_odmr,0):
        y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                                ha='center', va='bottom',color='r',fontsize=25)
    plt.plot(ws_podm,norm_flur,label="No-HF",color="green",lw=5)
    plt.title(f"PuODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    plt.plot(ws_doh_hf_podm,norm_flur_doh_hf,label="HF,Model 3",ls="--",color='purple',lw=5)
    plt.plot(ws_dua_hf_podm,norm_flur_dua_hf,label="HF,Model 2",ls="-.",lw=5,color="blue")
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            print(f"E={energy}")
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.minorticks_on()
    
    plt.savefig(f'Figs\\puODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    # Comparison
    plt.figure(figsize=(14, 8))
    plt.title(f"B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    plt.xlabel("Frequency (MHz)",fontsize=25)
    plt.ylabel("PL (a.u.)",fontsize=25)
    if not np.isclose(b_odmr,0):
        y,x1,x2=min(max_reg(flur_codm)),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        plt.hlines(y,x1,x2,"r","--")
        plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),
                     textcoords='offset points',ha='center', va='bottom',color='r',fontsize=25)
    plt.plot(ws_codm,max_reg(flur_codm),color='green',lw=5,label="no-HF,CW-ODMR")
    plt.plot(ws_podm,max_reg(flur_podm),color='lime',lw=5,label="no-HF,PuODMR")
    plt.plot(ws_doh_hf_codm,max_reg(flur_doh_hf_codm)-ep,color='purple',label="HF,CW-ODMR\nModel 3",ls="--")
    plt.plot(ws_doh_hf_podm,max_reg(flur_doh_hf_podm)-ep,color='mediumpurple',lw=5,label="HF,PuODMR\nModel 3",ls="--")
    plt.plot(ws_dua_hf_codm,max_reg(flur_dua_hf_codm)-2*ep,color='blue',lw=5,label="HF,CW-ODMR\nModel 2",ls="-.")
    plt.plot(ws_dua_hf_podm,max_reg(flur_dua_hf_podm)-2*ep,color='steelblue',lw=5,label="HF,PuODMR\nModel 2",ls="-.")
    m=0
    d=0
    for energy in energies:
        if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
            if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
                plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
                m+=1
            else:
                plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
                d+=1
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.minorticks_on()
    
    plt.savefig(f'Figs\\ODMR_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    ## FFT panels (one spectrum per trace)
    #fig_fft, ax_fft = plt.subplots(figsize=(14, 8))
    #plot_fft(ws_codm, (flur_codmr-s0)/(s0-s1), ax=ax_fft, label='no-HF')
    #plot_fft(ws_dua_hf_ramsey, max_reg((flu_ramsey_dua-s0_dua)/(s0_dua-s1_dua)),
    #         ax=ax_fft, label='HF Model 2')
    #plot_fft(ts_doh_hf_ramsey, max_reg((flu_ramsey_doh-s0_doh)/(s0_doh-s1_doh)),
    #         ax=ax_fft, label='HF Model 3')
    #plt.show()
    #Padding for fit (Proved unecessary just needed to change the fit functions)
    size = len(ws_codm) // 2 if b_odmr != 0 else len(ws_codm)
    size_p = len(ws_podm) // 2 if b_odmr != 0 else len(ws_podm)
    #pad = 300 if b_odmr==0 else 200

    # -------- codm before --------
    ws_codm_b = ws_codm[:size].copy()
    #w_b = np.arange(ws_codm_b[0] - pad * (ws_codm_b[1] - ws_codm_b[0]) / 2, ws_codm_b[0], (ws_codm_b[1] - ws_codm_b[0]) / 2)
    #w_ba = np.arange(ws_codm_b[-1], ws_codm_b[-1] + pad * (ws_codm_b[1] - ws_codm_b[0]) / 2, (ws_codm_b[1] - ws_codm_b[0]) / 2)
    #ws_codm_bef = np.concatenate([w_b, ws_codm_b, w_ba])
    #flur_codm_bef = np.concatenate([np.ones(pad), max_reg(flur_codm)[:size], np.ones(pad)])
    ws_codm_bef=ws_codm_b
    flur_codm_bef =max_reg(flur_codm)[:size]
    if b_odmr != 0:
        # -------- codm after --------
        ws_codm_a = ws_codm[size:].copy()
        #w_a = np.arange(ws_codm_a[0] - pad * (ws_codm_a[1] - ws_codm_a[0]) / 2, ws_codm_a[0], 
        #                (ws_codm_a[1] - ws_codm_a[0]) / 2)
        #w_ab = np.arange(ws_codm_a[-1], ws_codm_a[-1] + pad * (ws_codm_a[1] - ws_codm_a[0]) / 2, 
        #                 (ws_codm_a[1] - ws_codm_a[0]) / 2)
        #ws_codm_aft = np.concatenate([w_a, ws_codm_a, w_ab])
        #flur_codm_aft = np.concatenate([np.ones(pad), max_reg(flur_codm)[size:], np.ones(pad)])
        ws_codm_aft = ws_codm_a
        flur_codm_aft = max_reg(flur_codm)[size:]
    # -------- podm before --------
    ws_podm_b = ws_podm[:size_p]
    #w_pb = np.arange(ws_podm_b[0] - pad * (ws_podm_b[1] - ws_podm_b[0]) / 2, ws_podm_b[0], (ws_podm_b[1] - ws_podm_b[0]) / 2)
    #w_pba = np.arange(ws_podm_b[-1], ws_podm_b[-1] + pad * (ws_podm_b[1] - ws_podm_b[0]) / 2, 
    #                  (ws_podm_b[1] - ws_podm_b[0]) / 2)
    #ws_podm_bef = np.concatenate([w_pb, ws_podm_b, w_pba])
    #flur_podm_bef = np.concatenate([np.ones(pad), max_reg(flur_podm)[:size_p], np.ones(pad)])
    ws_podm_bef = ws_podm_b
    flur_podm_bef = max_reg(flur_podm)[:size_p]

    if b_odmr != 0:
        # -------- podm after --------
        ws_podm_a = ws_podm[size_p:]
        #w_pa = np.arange(ws_podm_a[0] - pad * (ws_podm_a[1] - ws_podm_a[0]) / 2, ws_podm_a[0], 
        #                 (ws_podm_a[1] - ws_podm_a[0]) / 2)
        #w_pab = np.arange(ws_podm_a[-1], ws_podm_a[-1] + pad * (ws_podm_a[1] - ws_podm_a[0]) / 2, 
        #                  (ws_podm_a[1] - ws_podm_a[0]) / 2)
        #ws_podm_aft = np.concatenate([w_pa, ws_podm_a, w_pab])
        #flur_podm_aft = np.concatenate([np.ones(pad), max_reg(flur_podm)[size_p:], np.ones(pad)])
        ws_podm_aft = ws_podm_a
        flur_podm_aft = max_reg(flur_podm)[size_p:]

    # -------- dua_hf_codm before --------
    ws_dua_hf_codm_b = ws_dua_hf_codm[:size]
    #w_dhb = np.arange(ws_dua_hf_codm_b[0] - pad * (ws_dua_hf_codm_b[1] - ws_dua_hf_codm_b[0]) / 2, ws_dua_hf_codm_b[0], 
    #                  (ws_dua_hf_codm_b[1] - ws_dua_hf_codm_b[0]) / 2)
    #w_dhba = np.arange(ws_dua_hf_codm_b[-1], ws_dua_hf_codm_b[-1] + pad * (ws_dua_hf_codm_b[1] - ws_dua_hf_codm_b[0]) / 2, 
    #                   (ws_dua_hf_codm_b[1] - ws_dua_hf_codm_b[0]) / 2)
    #ws_dua_hf_codm_bef = np.concatenate([w_dhb, ws_dua_hf_codm_b, w_dhba])
    #flur_dua_hf_codm_bef = np.concatenate([np.ones(pad), max_reg(flur_dua_hf_codm)[:size], np.ones(pad)])
    ws_dua_hf_codm_bef =  ws_dua_hf_codm_b
    flur_dua_hf_codm_bef = max_reg(flur_dua_hf_codm)[:size]

    if b_odmr != 0:
        ws_dua_hf_codm_a = ws_dua_hf_codm[size:]
        #w_dha = np.arange(ws_dua_hf_codm_a[0] - pad * (ws_dua_hf_codm_a[1] - ws_dua_hf_codm_a[0]) / 2, ws_dua_hf_codm_a[0], 
        #                  (ws_dua_hf_codm_a[1] - ws_dua_hf_codm_a[0]) / 2)
        #w_dhab = np.arange(ws_dua_hf_codm_a[-1], ws_dua_hf_codm_a[-1] + pad * (ws_dua_hf_codm_a[1] - ws_dua_hf_codm_a[0]) / 2, 
        #                   (ws_dua_hf_codm_a[1] - ws_dua_hf_codm_a[0]) / 2)
        #ws_dua_hf_codm_aft = np.concatenate([w_dha, ws_dua_hf_codm_a, w_dhab])
        #flur_dua_hf_codm_aft = np.concatenate([np.ones(pad), max_reg(flur_dua_hf_codm)[size:], np.ones(pad)])
        ws_dua_hf_codm_aft = ws_dua_hf_codm_a
        flur_dua_hf_codm_aft = max_reg(flur_dua_hf_codm)[size:]

    # -------- dua_hf_podm before --------
    ws_dua_hf_podm_b = ws_dua_hf_podm[:size_p]
    #w_dpb = np.arange(ws_dua_hf_podm_b[0] - pad * (ws_dua_hf_podm_b[1] - ws_dua_hf_podm_b[0]) / 2, ws_dua_hf_podm_b[0], 
    #                  (ws_dua_hf_podm_b[1] - ws_dua_hf_podm_b[0]) / 2)
    #w_dpba = np.arange(ws_dua_hf_podm_b[-1], ws_dua_hf_podm_b[-1] + pad * (ws_dua_hf_podm_b[1] - ws_dua_hf_podm_b[0]) / 2,
    #                   (ws_dua_hf_podm_b[1] - ws_dua_hf_podm_b[0]) / 2)
    #ws_dua_hf_podm_bef = np.concatenate([w_dpb, ws_dua_hf_podm_b, w_dpba])
    #flur_dua_hf_podm_bef = np.concatenate([np.ones(pad), max_reg(flur_dua_hf_podm)[:size_p], np.ones(pad)])
    ws_dua_hf_podm_bef =  ws_dua_hf_podm_b
    flur_dua_hf_podm_bef = max_reg(flur_dua_hf_podm)[:size_p]

    if b_odmr != 0:
        ws_dua_hf_podm_a = ws_dua_hf_podm[size_p:]
        #w_dpa = np.arange(ws_dua_hf_podm_a[0] - pad * (ws_dua_hf_podm_a[1] - ws_dua_hf_podm_a[0]) / 2, ws_dua_hf_podm_a[0],
        #                  (ws_dua_hf_podm_a[1] - ws_dua_hf_podm_a[0]) / 2)
        #w_dpab = np.arange(ws_dua_hf_podm_a[-1], ws_dua_hf_podm_a[-1] + pad * (ws_dua_hf_podm_a[1] - ws_dua_hf_podm_a[0]) / 2,
        #                   (ws_dua_hf_podm_a[1] - ws_dua_hf_podm_a[0]) / 2)
        #ws_dua_hf_podm_aft = np.concatenate([w_dpa, ws_dua_hf_podm_a, w_dpab])
        #flur_dua_hf_podm_aft = np.concatenate([np.ones(pad), max_reg(flur_dua_hf_podm)[size_p:], np.ones(pad)])
        ws_dua_hf_podm_aft = ws_dua_hf_podm_a
        flur_dua_hf_podm_aft =max_reg(flur_dua_hf_podm)[size_p:]

    # -------- doh_hf_codm --------
    ws_doh_hf_codm_b = ws_doh_hf_codm[:size]
    #w_dhb = np.arange(ws_doh_hf_codm_b[0] - pad * (ws_doh_hf_codm_b[1] - ws_doh_hf_codm_b[0]) / 2, ws_doh_hf_codm_b[0],
    #                  (ws_doh_hf_codm_b[1] - ws_doh_hf_codm_b[0]) / 2)
    #w_dhba = np.arange(ws_doh_hf_codm_b[-1], ws_doh_hf_codm_b[-1] + pad * (ws_doh_hf_codm_b[1] - ws_doh_hf_codm_b[0]) / 2,
    #                   (ws_doh_hf_codm_b[1] - ws_doh_hf_codm_b[0]) / 2)
    #ws_doh_hf_codm_bef = np.concatenate([w_dhb, ws_doh_hf_codm_b, w_dhba])
    #flur_doh_hf_codm_bef = np.concatenate([np.ones(pad), max_reg(flur_doh_hf_codm)[:size], np.ones(pad)])
    ws_doh_hf_codm_bef = ws_doh_hf_codm_b
    flur_doh_hf_codm_bef = max_reg(flur_doh_hf_codm)[:size]

    if b_odmr != 0:
        ws_doh_hf_codm_a = ws_doh_hf_codm[size:]
        #w_dha = np.arange(ws_doh_hf_codm_a[0] - pad * (ws_doh_hf_codm_a[1] - ws_doh_hf_codm_a[0]) / 2, ws_doh_hf_codm_a[0], 
        #                  (ws_doh_hf_codm_a[1] - ws_doh_hf_codm_a[0]) / 2)
        #w_dhab = np.arange(ws_doh_hf_codm_a[-1], ws_doh_hf_codm_a[-1] + pad * (ws_doh_hf_codm_a[1] - ws_doh_hf_codm_a[0]) / 2,
        #                   (ws_doh_hf_codm_a[1] - ws_doh_hf_codm_a[0]) / 2)
        #ws_doh_hf_codm_aft = np.concatenate([w_dha, ws_doh_hf_codm_a, w_dhab])
        #flur_doh_hf_codm_aft = np.concatenate([np.ones(pad), max_reg(flur_doh_hf_codm)[size:], np.ones(pad)])
        ws_doh_hf_codm_aft = ws_doh_hf_codm_a
        flur_doh_hf_codm_aft = max_reg(flur_doh_hf_codm)[size:]

    # -------- doh_hf_podm --------
    ws_doh_hf_podm_b = ws_doh_hf_podm[:size_p]
    #w_dpb = np.arange(ws_doh_hf_podm_b[0] - pad * (ws_doh_hf_podm_b[1] - ws_doh_hf_podm_b[0]) / 2, ws_doh_hf_podm_b[0],
    #                  (ws_doh_hf_podm_b[1] - ws_doh_hf_podm_b[0]) / 2)
    #w_dpba = np.arange(ws_doh_hf_podm_b[-1], ws_doh_hf_podm_b[-1] + pad * (ws_doh_hf_podm_b[1] - ws_doh_hf_podm_b[0]) / 2,
    #                   (ws_doh_hf_podm_b[1] - ws_doh_hf_podm_b[0]) / 2)
    #ws_doh_hf_podm_bef = np.concatenate([w_dpb, ws_doh_hf_podm_b, w_dpba])
    #flur_doh_hf_podm_bef = np.concatenate([np.ones(pad), max_reg(flur_doh_hf_podm)[:size_p], np.ones(pad)])
    ws_doh_hf_podm_bef = ws_doh_hf_podm_b
    flur_doh_hf_podm_bef = max_reg(flur_doh_hf_podm)[:size_p]

    if b_odmr != 0:
        ws_doh_hf_podm_a = ws_doh_hf_podm[size_p:]
        #w_dpa = np.arange(ws_doh_hf_podm_a[0] - pad * (ws_doh_hf_podm_a[1] - ws_doh_hf_podm_a[0]) / 2, ws_doh_hf_podm_a[0],
        #                  (ws_doh_hf_podm_a[1] - ws_doh_hf_podm_a[0]) / 2)
        #w_dpab = np.arange(ws_doh_hf_podm_a[-1], ws_doh_hf_podm_a[-1] + pad * (ws_doh_hf_podm_a[1] - ws_doh_hf_podm_a[0]) / 2,
        #                       (ws_doh_hf_podm_a[1] - ws_doh_hf_podm_a[0]) / 2)
        #ws_doh_hf_podm_aft = np.concatenate([w_dpa, ws_doh_hf_podm_a, w_dpab])
        #flur_doh_hf_podm_aft = np.concatenate([np.ones(pad), max_reg(flur_doh_hf_podm)[size_p:], np.ones(pad)])
        ws_doh_hf_podm_aft = ws_doh_hf_podm_a
        flur_doh_hf_podm_aft = max_reg(flur_doh_hf_podm)[size_p:]

    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Lorentzian Fit CW-ODMR",fontsize=25)
    fit_plot_data(lorent,ws_codm_bef,flur_codm_bef,
                  ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,10,1,1], 
                  ax=ax_fit,label="p1 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr !=0:
        fit_plot_data(lorent,ws_codm_aft,flur_codm_aft,
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,10,1,1], 
                      ax=ax_fit,label="p2 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\cwODMR_lorent_no_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Lorentzian Fit CW-ODMR",fontsize=25)
    fit_plot_data(lorent,ws_dua_hf_codm_bef,flur_dua_hf_codm_bef,
                  ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,10,1,1],
                  ax=ax_fit,label="p1 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(lorent,ws_dua_hf_codm_aft,flur_dua_hf_codm_aft,
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,10,1,1],
                      ax=ax_fit,label="p2 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\cwODMR_lorent_M2_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Lorentzian Fit CW-ODMR",fontsize=25)
    fit_plot_data(lorent,ws_doh_hf_codm_bef,flur_doh_hf_codm_bef,
                  ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,10,1,1],
                  ax=ax_fit,label="p1 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(lorent,ws_doh_hf_codm_aft,flur_doh_hf_codm_aft,
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,10,1,1],
                      ax=ax_fit,label="p2 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\cwODMR_lorent_M3_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Lorentzian Fit PulODMR",fontsize=25)
    fit_plot_data(lorent,ws_podm_bef,flur_podm_bef,
                  ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,1,1,1], 
                  ax=ax_fit,label="p1 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(lorent,ws_podm_aft,flur_podm_aft,
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,1,1,1],
                      ax=ax_fit,label="p2 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\pulODMR_lorent_no_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Lorentzian Fit PulODMR",fontsize=25)
    fit_plot_data(lorent,ws_dua_hf_podm_bef,flur_dua_hf_podm_bef,
                  ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,1,1,1],
                  ax=ax_fit,label="p1 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(lorent,ws_dua_hf_podm_aft,flur_dua_hf_podm_aft,
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,1,1,1],
                      ax=ax_fit,label="p2 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\pulODMR_lorent_M2_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Lorentzian Fit PulODMR",fontsize=25)
    fit_plot_data(lorent,ws_doh_hf_podm_bef, flur_doh_hf_podm_bef,
                  ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,1,1,1],
                  ax=ax_fit,label="p1 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(lorent,ws_doh_hf_podm_aft, flur_doh_hf_podm_aft,
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,1,1,1],
                      ax=ax_fit,label="p2 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\pulODMR_lorent_M3_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Generalized Gaussian Fit CW-ODMR",fontsize=25)
    fit_plot_data(gen_gauss,ws_codm_bef,flur_codm_bef,
                  ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1],
                  ax=ax_fit,label="p1 No-HF", yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(gen_gauss,ws_codm_aft,flur_codm_aft,
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1],
                      ax=ax_fit,label="p2 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #x_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\cwODMR_gauss_no_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Generalized Gaussian Fit CW-ODMR",fontsize=25)
    fit_plot_data(gen_gauss,ws_dua_hf_codm_bef,flur_dua_hf_codm_bef,
                  ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1],
                  ax=ax_fit,label="p1 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(gen_gauss,ws_dua_hf_codm_aft,flur_dua_hf_codm_aft,
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1],
                      ax=ax_fit,label="p2 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\cwODMR_gauss_M2_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Generalized Gaussian Fit CW-ODMR",fontsize=25)
    fit_plot_data(gen_gauss,ws_doh_hf_codm_bef,flur_doh_hf_codm_bef,
                  ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1],
                  ax=ax_fit,label="p1 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(gen_gauss,ws_doh_hf_codm_aft,flur_doh_hf_codm_aft,
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1],
                      ax=ax_fit,label="p2 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\cwODMR_gauss_M3_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    #ax_fit.set_title("Generalized Gaussian Fit PulODMR",fontsize=25)
    fit_plot_data(gen_gauss,ws_podm_bef,flur_podm_bef,
                  ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1], 
                  ax=ax_fit,label="p1 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(gen_gauss,ws_podm_aft,flur_podm_aft,
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1],
                      ax=ax_fit,label="p2 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\pulODMR_gauss_no_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Generalized Gaussian Fit PulODMR",fontsize=25)
    fit_plot_data(gen_gauss,ws_dua_hf_podm_bef,flur_dua_hf_podm_bef,
                  ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1],
                  ax=ax_fit,label="p1 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(gen_gauss,ws_dua_hf_podm_aft,flur_dua_hf_podm_aft,
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1],
                      ax=ax_fit,label="p2 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\pulODMR_gauss_M2_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
    ax_fit.set_title("Generalized Gaussian Fit PulODMR",fontsize=25)
    fit_plot_data(gen_gauss,ws_doh_hf_podm_bef, flur_doh_hf_podm_bef,
                  ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1],
                  ax=ax_fit,label="p1 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    if b_odmr!=0:
        fit_plot_data(gen_gauss,ws_doh_hf_podm_aft, flur_doh_hf_podm_aft,
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1],
                      ax=ax_fit,label="p2 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
    #ax_fit.set_xlim(D_gs-mu_e*b_odmr-100,D_gs+mu_e*b_odmr+100)
    plt.savefig(f'Figs\\pulODMR_gauss_M3_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                bbox_inches='tight',dpi=400)
    plt.show()
    
    # Caculating PL normalized by S0S1
    if b_odmr!=0:
        ## -------- codm before --------
        #ws_codm_b = ws_codm[:size].copy()
        #ws_codm_bef=ws_codm_b
        #flur_codm_bef =max_reg(flur_codm)[:size]
        ## -------- codm after --------
        #ws_codm_a = ws_codm[size:].copy()
        #ws_codm_aft = ws_codm_a
        #flur_codm_aft = max_reg(flur_codm)[size:]
        ## -------- podm before --------
        #ws_podm_b = ws_podm[:size_p]
        #ws_podm_bef = ws_podm_b
        #flur_podm_bef = max_reg(flur_podm)[:size_p]
#
        ## -------- podm after --------
        #ws_podm_a = ws_podm[size_p:]
        #ws_podm_aft = ws_podm_a
        #flur_podm_aft = max_reg(flur_podm)[size_p:]
#
        ## -------- dua_hf_codm before --------
        #ws_dua_hf_codm_b = ws_dua_hf_codm[:size]
        #ws_dua_hf_codm_bef =  ws_dua_hf_codm_b
        #flur_dua_hf_codm_bef = max_reg(flur_dua_hf_codm)[:size]
#
        #ws_dua_hf_codm_a = ws_dua_hf_codm[size:]
        #ws_dua_hf_codm_aft = ws_dua_hf_codm_a
        #flur_dua_hf_codm_aft = max_reg(flur_dua_hf_codm)[size:]
#
        ## -------- dua_hf_podm before --------
        #ws_dua_hf_podm_b = ws_dua_hf_podm[:size_p]
        #ws_dua_hf_podm_bef =  ws_dua_hf_podm_b
        #flur_dua_hf_podm_bef = max_reg(flur_dua_hf_podm)[:size_p]
#
        #ws_dua_hf_podm_a = ws_dua_hf_podm[size_p:]
        #ws_dua_hf_podm_aft = ws_dua_hf_podm_a
        #flur_dua_hf_podm_aft =max_reg(flur_dua_hf_podm)[size_p:]
#
        ## -------- doh_hf_codm --------
        #ws_doh_hf_codm_b = ws_doh_hf_codm[:size]
        #ws_doh_hf_codm_bef = ws_doh_hf_codm_b
        #flur_doh_hf_codm_bef = max_reg(flur_doh_hf_codm)[:size]
#
        #ws_doh_hf_codm_a = ws_doh_hf_codm[size:]
        #ws_doh_hf_codm_aft = ws_doh_hf_codm_a
        #flur_doh_hf_codm_aft = max_reg(flur_doh_hf_codm)[size:]
#       
        ## -------- doh_hf_podm --------
        #ws_doh_hf_podm_b = ws_doh_hf_podm[:size_p]
        #ws_doh_hf_podm_bef = ws_doh_hf_podm_b
        #flur_doh_hf_podm_bef = max_reg(flur_doh_hf_podm)[:size_p]
        #
        #ws_doh_hf_podm_a = ws_doh_hf_podm[size_p:]
        #ws_doh_hf_podm_aft = ws_doh_hf_podm_a
        #flur_doh_hf_podm_aft = max_reg(flur_doh_hf_podm)[size_p:]
        #init_state=(n1+n2+n3)/3
        #cw_base=dynamics_no(10, 
        #                   init_state,
        #                   b=B0(b_odmr,0,0),
        #                   om_r=om_r_odmr,
        #                   om=0.0,
        #                   w_p=w_p_odmr/n_p_odmr,
        #                   ti=0.0, 
        #                   mode="Laser", 
        #                   progress_bar="OFF")
        #init_state=((n1+n2+n3)&IdN15)/6
        #cw_base_dua=dynamics_dua_hf(10, 
        #                   init_state,
        #                   b=B0(b_odmr,0,0),
        #                   om_r=om_r_odmr,
        #                   om=0.0,
        #                   w_p=w_p_odmr/n_p_odmr,
        #                   ti=0.0, 
        #                   mode="Laser", 
        #                   progress_bar="OFF")
        #cw_base_doh=dynamics_doh_hf(10, 
        #                   init_state,
        #                   b=B0(b_odmr,0,0),
        #                   om_r=om_r_odmr,
        #                   om=0.0,
        #                   w_p=w_p_odmr/n_p_odmr,
        #                   ti=0.0, 
        #                   mode="Laser", 
        #                   progress_bar="OFF")
        
        
        read_time=3
        init_state=1/3*(n1+n2+n3)
        exps=(n4+n5+n6)
        S=contrast(dynamics_no,init_state,exps,b_odmr,w_p_odmr/n_p_odmr,w_p_odmr,om_r_odmr,readtime=read_time)
        s0,s1=S[4],S[5] # 0,1 - 2,3 are non integrated read and full(10 \mu s) time, 
                        # respectively, 4,5 - 6,7 are the integrate in the same time spans 
        #print(f"SNR_noHF(B={b},tR={read_time}):{(s0-s1)/np.sqrt(0.5*(s0+s1)):.4g}")
        init_state=1/6*((n1+n2+n3)&IdN15)
        exps=((n4+n5+n6)&IdN15)
        S=contrast(dynamics_doh_hf,init_state,exps,b_odmr,w_p_odmr/n_p_odmr,w_p_odmr,om_r_odmr,readtime=read_time)
        s0_doh,s1_doh=S[4],S[5]
        #print(f"SNR_doh(B={b},tR={read_time}):{(s0_doh-s1_doh)/np.sqrt(0.5*(s0_doh+s1_doh)):.4g}")
        S=contrast(dynamics_dua_hf,init_state,exps,b_odmr,w_p_odmr/n_p_odmr,w_p_odmr,om_r_odmr,readtime=read_time)
        s0_dua,s1_dua=S[4],S[5]
        #print(f"SNR_dua(B={b},tR={read_time}):{(s0_dua-s1_dua)/np.sqrt(0.5*(s0_dua+s1_dua)):.4g}")
        # Comparison
        plt.figure(figsize=(14, 8))
        plt.title(f"B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz (S-S1)/(S0-S1)",fontsize=25)
        plt.xlabel("Frequency (MHz)",fontsize=25)
        plt.ylabel("PL (a.u.)",fontsize=25)
        #if not np.isclose(b_odmr,0):
        #    y,x1,x2=min(max_reg(flur_codm)),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
        #    plt.hlines(y,x1,x2,"r","--")
        #    plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
        #                            ha='center', va='bottom',color='r',fontsize=25)
        plt.plot(ws_codm,max_reg((flur_codm-s0)/(s0-s1)),color='green',lw=5,label="no-HF,CW-ODMR")
        plt.plot(ws_podm,max_reg((flur_podm-s0)/(s0-s1)),color='lime',lw=5,label="no-HF,PuODMR")
        plt.plot(ws_doh_hf_codm,max_reg((flur_doh_hf_codm-s1_doh)/(s0_doh-s1_doh))-ep,color='purple',
                 label="HF,CW-ODMR\nModel 3",ls="--")
        plt.plot(ws_doh_hf_podm,max_reg((flur_doh_hf_podm-s1_doh)/(s0_doh-s1_doh))-ep,color='mediumpurple',lw=5,
                 label="HF,PuODMR\nModel 3",ls="--")
        plt.plot(ws_dua_hf_codm,max_reg((flur_dua_hf_codm-s1_dua)/(s0_dua-s1_dua))-2*ep,color='blue',lw=5,
                 label="HF,CW-ODMR\nModel 2",ls="-.")
        plt.plot(ws_dua_hf_podm,max_reg((flur_dua_hf_podm-s1_dua)/(s0_dua-s1_dua))-2*ep,color='steelblue',lw=5,
                 label="HF,PuODMR\nModel 2",ls="-.")
        m=0
        d=0
        #for energy in energies:
        #    if np.abs(energy)>=D_gs-mu_e*b_odmr-2/5*mu_e*b_odmr-2*a_gs[0] and np.abs(energy)<=D_gs+mu_e*b_odmr+2*a_gs[0]:
        #        if np.isclose(energy,D_gs+mu_e*b_odmr,atol=a_gs[0]) or np.isclose(energy,D_gs-mu_e*b_odmr,atol=a_gs[0]):
        #            plt.vlines(energy,.98,1.005,ls='dotted',label="Energy of GS \n$m_s=\\pm1$" if m==0 else "",color="magenta")
        #            m+=1
        #        else:
        #            plt.vlines(np.abs(energy),.98,1.005,ls='dotted',label="Other eigen energies" if d==0 else"",color="darkgoldenrod")
        #            d+=1
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
        plt.minorticks_on()
        plt.savefig(f'Figs\\ODMR_s0s1_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Lorentzian Fit CW-ODMR",fontsize=25)
        fit_plot_data(lorent,ws_codm[:size], max_reg((flur_codm-s1)/(s0-s1))[:size],
                      ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,10,1,1],
                      ax=ax_fit,label="p1 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(lorent,ws_codm[size:], max_reg((flur_codm-s1)/(s0-s1))[size:],
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,10,1,1],
                      ax=ax_fit,label="p2 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\CWODMR_s0s1_no_lorent_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Lorentzian Fit CW-ODMR",fontsize=25)
        fit_plot_data(lorent,ws_dua_hf_codm[:size], max_reg((flur_dua_hf_codm-s1_dua)/(s0_dua-s1_dua))[:size],
                      ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,10,1,1], 
                      ax=ax_fit,label="p1 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(lorent,ws_dua_hf_codm[size:], max_reg((flur_dua_hf_codm-s1_dua)/(s0_dua-s1_dua))[size:],
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,10,1,1], 
                      ax=ax_fit,label="p2 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\CWODMR_s0s1_M2_lorent_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Lorentzian Fit CW-ODMR",fontsize=25)
        fit_plot_data(lorent,ws_doh_hf_codm[:size], max_reg((flur_doh_hf_codm-s1_doh)/(s0_doh-s1_doh))[:size],
                      ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,10,1,1], 
                      ax=ax_fit,label="p1 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(lorent,ws_doh_hf_codm[size:], max_reg((flur_doh_hf_codm-s1_doh)/(s0_doh-s1_doh))[size:],
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,10,1,1], 
                      ax=ax_fit,label="p2 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\CWODMR_s0s1_M3_lorent_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Lorentzian Fit PulODMR",fontsize=25)
        fit_plot_data(lorent,ws_podm[:size_p], max_reg((flur_podm-s1)/(s0-s1))[:size_p],
                      ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,1,1,1],
                      ax=ax_fit,label="p1 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(lorent,ws_podm[size_p:], max_reg((flur_podm-s1)/(s0-s1))[size_p:],
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,1,1,1],
                      ax=ax_fit,label="p2 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\PulODMR_s0s1_no_lorent_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Lorentzian Fit PulODMR",fontsize=25)
        fit_plot_data(lorent,ws_dua_hf_podm[:size_p], max_reg((flur_dua_hf_podm-s1_dua)/(s0_dua-s1_dua))[:size_p],
                      ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,1,1,1], 
                      ax=ax_fit,label="p1 HF Model 2", yaxis_label="PL (a.u.)", xaxis_label="Frequency (MHz)")
        fit_plot_data(lorent,ws_dua_hf_podm[size_p:], max_reg((flur_dua_hf_podm-s1_dua)/(s0_dua-s1_dua))[size_p:],
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,1,1,1], 
                      ax=ax_fit,label="p2 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\PulODMR_s0s1_M2_lorent_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Lorentzian Fit PulODMR",fontsize=25)
        fit_plot_data(lorent,ws_doh_hf_podm[:size_p], max_reg((flur_doh_hf_podm-s1_doh)/(s0_doh-s1_doh))[:size_p],
                      ["x0","gamma","A","D"],[D_gs-mu_e*b_odmr,1,1,1], 
                      ax=ax_fit,label="p1 HF Model 3", yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(lorent,ws_doh_hf_podm[size_p:], max_reg((flur_doh_hf_podm-s1_doh)/(s0_doh-s1_doh))[size_p:],
                      ["x0","gamma","A","D"],[D_gs+mu_e*b_odmr,1,1,1], 
                      ax=ax_fit,label="p2 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\PulODMR_s0s1_M3_lorent_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Generalized Gaussian Fit CW-ODMR",fontsize=25)
        fit_plot_data(gen_gauss,ws_codm[:size], max_reg((flur_codm-s1)/(s0-s1))[:size],
                      ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit,label="p1 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(gen_gauss,ws_codm[size:], max_reg((flur_codm-s1)/(s0-s1))[size:],
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,.05,1,1],
                      ax=ax_fit,label="p2 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\CWODMR_s0s1_no_gauss_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Generalized Gaussian Fit CW-ODMR",fontsize=25)
        fit_plot_data(gen_gauss,ws_dua_hf_codm[:size], max_reg((flur_dua_hf_codm-s1_dua)/(s0_dua-s1_dua))[:size],
                      ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit,label="p1 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(gen_gauss,ws_dua_hf_codm[size:], max_reg((flur_dua_hf_codm-s1_dua)/(s0_dua-s1_dua))[size:],
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1],
                      ax=ax_fit,label="p2 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\CWODMR_s0s1_M2_gauss_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Generalized Gaussian Fit CW-ODMR",fontsize=25)
        fit_plot_data(gen_gauss,ws_doh_hf_codm[:size], max_reg((flur_doh_hf_codm-s1_doh)/(s0_doh-s1_doh))[:size],
                      ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit, label="p1 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(gen_gauss,ws_doh_hf_codm[size:], max_reg((flur_doh_hf_codm-s1_doh)/(s0_doh-s1_doh))[size:],
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit,label="p2 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\CWODMR_s0s1_M3_gauss_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Generalized Gaussian Fit PulODMR",fontsize=25)
        fit_plot_data(gen_gauss,ws_podm[:size_p], max_reg((flur_podm-s1)/(s0-s1))[:size_p],
                      ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit,label="p1 No-HF",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(gen_gauss,ws_podm[size_p:], max_reg((flur_podm-s1)/(s0-s1))[size_p:],
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit,label="p2 No-HF", yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\PulODMR_s0s1_no_gauss_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Generalized Gaussian Fit PulODMR",fontsize=25)
        fit_plot_data(gen_gauss,ws_dua_hf_podm[:size_p], max_reg((flur_dua_hf_podm-s1_dua)/(s0_dua-s1_dua))[:size_p],
                      ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit, label="p1 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(gen_gauss,ws_dua_hf_podm[size_p:], max_reg((flur_dua_hf_podm-s1_dua)/(s0_dua-s1_dua))[size_p:],
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit,label="p2 HF Model 2",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\PulODMR_s0s1_M2_gauss_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()
        
        fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
        ax_fit.set_title("Generalized Gaussian Fit PulODMR",fontsize=25)
        fit_plot_data(gen_gauss,ws_doh_hf_podm[:size_p], max_reg((flur_doh_hf_podm-s1_doh)/(s0_doh-s1_doh))[:size_p],
                      ["mu","alph","bet","A","D"],[D_gs-mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit,label="p1 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        fit_plot_data(gen_gauss,ws_doh_hf_podm[size_p:], max_reg((flur_doh_hf_podm-s1_doh)/(s0_doh-s1_doh))[size_p:],
                      ["mu","alph","bet","A","D"],[D_gs+mu_e*b_odmr,1,0.5,1,1], 
                      ax=ax_fit,label="p2 HF Model 3",yaxis_label="PL (a.u.)",xaxis_label="Frequency (MHz)")
        plt.savefig(f'Figs\\PulODMR_s0s1_M3_gauss_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.png',
                    bbox_inches='tight',dpi=400)
        plt.show()

# Ramsey

In [ ]:
n_p_ramsey=1.5
w_p_ramsey=n_p_ramsey*1.9
om_r_pi=np.pi/t2_gs
oms_r=np.array([om_r_pi,2*om_r_pi,10*om_r_pi])
bs=np.array([0,100,510,1020])
for omr in tqdm(oms_r,desc='Rabi Frequency',position=0):
    for b in tqdm(bs,desc='Magnectic Field',position=1):
        b0=B0(b,0.0,0.0)
        # Load no_HF Ramsey
        flur_ramsey,times_ramsey,ns_ramsey,ts_ramsey=(
            np.load(f'NumpyArrays\\flur_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\times_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\ns_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\ts_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            )
        # Load Model 3 ramsey
        flur_doh_hf_ramsey,times_doh_hf_ramsey,ns_doh_hf_ramsey,ts_doh_hf_ramsey=(
            np.load(f'NumpyArrays\\flur_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\times_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\ns_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\ts_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            )
        # Load Model 2 Ramsey
        flur_dua_hf_ramsey,times_dua_hf_ramsey,ns_dua_hf_ramsey,ts_dua_hf_ramsey=(                
            np.load(f'NumpyArrays\\flur_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\times_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\ns_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            np.load(f'NumpyArrays\\ts_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.npy'),
            )
        flur_ramsey=np.reshape(flur_ramsey,len(flur_ramsey))
        flur_doh_hf_ramsey=np.reshape(flur_doh_hf_ramsey,len(flur_doh_hf_ramsey))
        flur_dua_hf_ramsey=np.reshape(flur_dua_hf_ramsey,len(flur_dua_hf_ramsey))
        # Load no_HF Ramsey
        flur_ramsey_510,times_ramsey_510,ns_ramsey_510,ts_ramsey_510=(
            np.load(f'NumpyArrays\\flur_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\times_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\ns_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\ts_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            )
        # Load Model 3 ramsey
        flur_doh_hf_ramsey_510,times_doh_hf_ramsey_510,ns_doh_hf_ramsey_510,ts_doh_hf_ramsey_510=(
            np.load(f'NumpyArrays\\flur_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\times_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\ns_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\ts_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            )
        # Load Model 2 Ramsey
        flur_dua_hf_ramsey_510,times_dua_hf_ramsey_510,ns_dua_hf_ramsey_510,ts_dua_hf_ramsey_510=(                
            np.load(f'NumpyArrays\\flur_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\times_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\ns_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            np.load(f'NumpyArrays\\ts_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_5_tf_10.npy'),
            )
        flur_ramsey_510=np.reshape(flur_ramsey_510,len(flur_ramsey_510))
        flur_doh_hf_ramsey_510=np.reshape(flur_doh_hf_ramsey_510,len(flur_doh_hf_ramsey_510))
        flur_dua_hf_ramsey_510=np.reshape(flur_dua_hf_ramsey_510,len(flur_dua_hf_ramsey_510))
        # Put together the 0 to 5 wait time to the 5 to 10 wait time
        fls_ramsey_0_10=np.concatenate((flur_ramsey,flur_ramsey_510))
        fls_dua_hf_ramsey_0_10=np.concatenate((flur_dua_hf_ramsey,flur_dua_hf_ramsey_510))
        fls_doh_hf_ramsey_0_10=np.concatenate((flur_doh_hf_ramsey,flur_doh_hf_ramsey_510))
        ts_ramsey_0_10=np.concatenate((ts_ramsey,ts_ramsey_510))
        ts_dua_hf_ramsey_0_10=np.concatenate((ts_dua_hf_ramsey,ts_dua_hf_ramsey_510))
        ts_doh_hf_ramsey_0_10=np.concatenate((ts_doh_hf_ramsey,ts_doh_hf_ramsey_510))
        
        all_ns = [*ns_ramsey, *ns_ramsey_510]
        all_ns_dua = [*ns_dua_hf_ramsey, *ns_dua_hf_ramsey_510]
        all_ns_doh = [*ns_doh_hf_ramsey, *ns_doh_hf_ramsey_510]
        all_times = [*times_ramsey, *times_ramsey_510]
        all_times_dua = [*times_dua_hf_ramsey, *times_dua_hf_ramsey_510]
        all_times_doh = [*times_doh_hf_ramsey, *times_doh_hf_ramsey_510]
        
        read_time=3.0

        init_state=1/3*(n1+n2+n3)
        exps=(n4+n5+n6)
        S=contrast(dynamics_no,init_state,exps,b,w_p_ramsey/n_p_ramsey,w_p_ramsey,omr,readtime=read_time)
        s0,s1=S[4],S[5] # 0,1 - 2,3 are non integrated read and full(10 \mu s) time, 
                        # respectively, 4,5 - 6,7 are the integrate in the same time spans 
        print(f"SNR_noHF(B={b},tR={read_time}):{(s0-s1)/np.sqrt(0.5*(s0+s1)):.4g}")
        init_state=1/6*((n1+n2+n3)&IdN15)
        exps=((n4+n5+n6)&IdN15)
        S=contrast(dynamics_doh_hf,init_state,exps,b,w_p_ramsey/n_p_ramsey,w_p_ramsey,omr,readtime=read_time)
        s0_doh,s1_doh=S[4],S[5]
        print(f"SNR_doh(B={b},tR={read_time}):{(s0_doh-s1_doh)/np.sqrt(0.5*(s0_doh+s1_doh)):.4g}")
        S=contrast(dynamics_dua_hf,init_state,exps,b,w_p_ramsey/n_p_ramsey,w_p_ramsey,omr,readtime=read_time)
        s0_dua,s1_dua=S[4],S[5]
        print(f"SNR_dua(B={b},tR={read_time}):{(s0_dua-s1_dua)/np.sqrt(0.5*(s0_dua+s1_dua)):.4g}")
        
        if b==0:
            wind_time=read_time
            
            flu_ramsey_0=np.array([])
            for ns_temp,times_temp in zip(all_ns,all_times):
                wind=int(5000*wind_time/10)
                fl=np.zeros(wind,dtype=np.complex128)
                c=0
                for i, n in enumerate(ns_temp):
                    if i > 2 and i < 6: #select population of states 4,5,6 for the readout time selected by wind param
                        fl+=n[-5000:-5000+wind]
                flu_ramsey_0=np.append(flu_ramsey_0,scp.integrate.simpson(fl,times_temp[-5000:-5000+wind]))
            sig_0=(flu_ramsey_0-s1)/(s0-s1)
            
            flu_ramsey_dua_0=np.array([])
            for ns_temp,times_temps in zip(all_ns_dua,all_times_dua):
                wind=int(5000*wind_time/10)
                fl=np.zeros(wind,dtype=np.complex128)
                for i, n in enumerate(ns_temp):
                    if i > 2 and i < 6: #select population of states 4,5,6 for the readout time selected by wind param
                        fl+=n[-5000:-5000+wind]
                flu_ramsey_dua_0=np.append(flu_ramsey_dua_0,scp.integrate.simpson(fl,times_temp[-5000:-5000+wind]))
            sig_dua_0=(flu_ramsey_dua_0-s1_dua)/(s0_dua-s1_dua)
            
            flu_ramsey_doh_0=np.array([])
            for ns_temp,times_temp in zip(all_ns_doh,all_times_doh):
                wind=int(5000*wind_time/10)
                fl=np.zeros(wind,dtype=np.complex128)
                for i, n in enumerate(ns_temp):
                    if i > 2 and i < 6: #select population of states 4,5,6 for the readout time selected by wind param
                        fl+=n[-5000:-5000+wind]       
                flu_ramsey_doh_0=np.append(flu_ramsey_doh_0,scp.integrate.simpson(fl,times_temp[-5000:-5000+wind]))
            sig_doh_0=(flu_ramsey_doh_0-s1_doh)/(s0_doh-s1_doh)
        else:
            print(f"----------------------B={b:.2f} G--Om_R={omr:.2f}--------------------------------") 
                   
            wind_time=read_time
            
            plt.figure(figsize=(14, 8))
            plt.title(f"Ramsey\n B={b:.2f} G, $\\Omega_r$={omr:.2f} MHz\n(S-S1)/(S0-S1) z-score",fontsize=25)
            
            plt.plot(ts_ramsey_0_10,stand(sig_0),"k-",lw=3,alpha=0.5,label='B(0 G) no-HF')
            plt.plot(ts_dua_hf_ramsey_0_10,stand(sig_dua_0),"k-.",lw=3,alpha=0.5,label="B(0 G) HF Model 2")
            plt.plot(ts_doh_hf_ramsey_0_10,stand(sig_doh_0),"k--",lw=3,alpha=0.5,label="B(0 G) HF Model 3")
            
            flu_ramsey=np.array([])
            for ns_temp,times_temp in zip(all_ns,all_times):
                wind=int(5000*wind_time/10)
                fl=np.zeros(wind,dtype=np.complex128)
                c=0
                for i, n in enumerate(ns_temp):
                    if i > 2 and i < 6: #select population of states 4,5,6 for the readout time selected by wind param
                        fl+=n[-5000:-5000+wind]
                flu_ramsey=np.append(flu_ramsey,scp.integrate.simpson(fl,times_temp[-5000:-5000+wind]))
            sig=(flu_ramsey-s1)/(s0-s1)
            plt.plot(ts_ramsey_0_10,stand(sig),lw=5,label='no-HF')
            
            flu_ramsey_dua=np.array([])
            for ns_temp,times_temp in zip(all_ns_dua,all_times_dua):
                wind=int(5000*wind_time/10)
                fl=np.zeros(wind,dtype=np.complex128)
                for i, n in enumerate(ns_temp):
                    if i > 2 and i < 6: #select population of states 4,5,6 for the readout time selected by wind param
                        fl+=n[-5000:-5000+wind]
                flu_ramsey_dua=np.append(flu_ramsey_dua,scp.integrate.simpson(fl,times_temp[-5000:-5000+wind]))
            sig_dua=(flu_ramsey_dua-s1_dua)/(s0_dua-s1_dua)
            plt.plot(ts_dua_hf_ramsey_0_10,stand(sig_dua),lw=5,ls="-.",label="HF Model 2")
            
            flu_ramsey_doh=np.array([])
            for ns_temp,times_temp in zip(all_ns_doh,all_times_doh):
                wind=int(5000*wind_time/10)
                fl=np.zeros(wind,dtype=np.complex128)
                for i, n in enumerate(ns_temp):
                    if i > 2 and i < 6: #select population of states 4,5,6 for the readout time selected by wind param
                        fl+=n[-5000:-5000+wind]
                flu_ramsey_doh=np.append(flu_ramsey_doh,scp.integrate.simpson(fl,times_temp[-5000:-5000+wind]))
            sig_doh=(flu_ramsey_doh-s1_doh)/(s0_doh-s1_doh)
            plt.plot(ts_doh_hf_ramsey_0_10,stand(sig_doh),lw=5,ls="--",label="HF Model 3")
            
            plt.minorticks_on()
            plt.ylabel("PL (a.u.)",fontsize=25)
            plt.xlabel('$\\tau (\\mu s)$',fontsize=25)
            plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
            plt.savefig(f'Figs\\Ramsey_s0s1_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            
            print(f"----------------------Fit----S0S1----------------------------")
            w1=D_gs-mu_e*b
            w0=D_gs
            print(f"w1**2/(w1**2+w0**2)={w1**2/(w1**2+w0**2):.2f}")
            fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
            _,f_params_no=fit_plot_data(expo,ts_ramsey_0_10, sig,
                          ["A","tau","D"],
                          [w1**2/(w1**2+w0**2),t2_gs,(np.max(sig)+np.min(sig)).real/2],  
                          ax=ax_fit, label="No-HF",yaxis_label="PL (a.u.)",xaxis_label='$\\tau (\\mu s)$')
            plt.savefig(f'Figs\\Ramsey_fit_s0s1_no_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            
            fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
            _,f_params_dua=fit_plot_data(damp_cos,ts_dua_hf_ramsey_0_10, sig_dua,
                          ["A","om","phi","tau","D"],
                          [1,omr,np.pi/2,t2_gs,
                           (np.max(sig_dua)+np.min(sig_dua)).real/2], 
                          ax=ax_fit, label="HF Model 2",yaxis_label="PL (a.u.)",xaxis_label='$\\tau (\\mu s)$')
            plt.savefig(f'Figs\\Ramsey_fit_s0s1_m2_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            
            print(f"from fit M2 B_0={f_params_dua['om'][0]/(2*np.pi*mu_e):.6g} ± {f_params_dua['om'][1]/(2*np.pi*mu_e):.8g}")
            print(f"from fit M2 t*_2={f_params_dua['tau'][0]:.6g} ± {f_params_dua['tau'][1]:.8g}")
            fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
            _,f_params_doh=fit_plot_data(damp_cos,ts_doh_hf_ramsey_0_10, sig_doh,
                          ["A","om","phi","tau","D"],
                          [1,omr,np.pi/2,t2_gs,
                           (np.max(sig_doh)+np.min(sig_doh)).real/2], 
                          ax=ax_fit, label="HF Model 3",yaxis_label="PL (a.u.)",xaxis_label='$\\tau (\\mu s)$')
            plt.savefig(f'Figs\\Ramsey_fit_s0s1_m3_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            print(f"from fit M3 B_0={f_params_doh['om'][0]/(2*np.pi*mu_e):.6g} ± {f_params_doh['om'][1]/(2*np.pi*mu_e):.8g}")
            print(f"from fit M3 t*_2={f_params_doh['tau'][0]:.6g} ± {f_params_doh['tau'][1]:.8g}")
            print(f"----------------------FFT----S0S1----------------------------")
            # FFT panels (one spectrum per trace)
            fig_fft, ax_fft = plt.subplots(figsize=(14, 8))
            #fit_params = [val[0] for val in f_params_no.values()]
            #t=np.linspace(0,10,500)
            #plot_fft(t, expo(t,*fit_params), 
            #         ax=ax_fft, label='no-HF')
            #fit_params = [val[0] for val in f_params_dua.values()]
            #t=np.linspace(0,10,500)
            #plot_fft(t, damp_sin(t,*fit_params),
            #         ax=ax_fft, label='HF Model 2')
            #fit_params = [val[0] for val in f_params_doh.values()]
            #t=np.linspace(0,10,500)
            #plot_fft(t, damp_sin(t,*fit_params),
            #         ax=ax_fft, label='HF Model 3')
            
            plot_fft(ts_ramsey_0_10, sig, 
                     ax=ax_fft, label='no-HF')
            plot_fft(ts_dua_hf_ramsey_0_10, sig_dua,
                     ax=ax_fft, label='HF Model 2')
            plot_fft(ts_doh_hf_ramsey_0_10, sig_doh,
                     ax=ax_fft, label='HF Model 3')
            plt.savefig(f'Figs\\Ramsey_fft_s0s1_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()


            print(f"----------------------Z-Score--------------------------------")
            plt.figure(figsize=(14, 8))
            plt.title(f"Ramsey\n B={b:.2f} G, $\\Omega_r$={omr:.2f} MHz\nZ-Score Norm",fontsize=25)
            
            plt.plot(ts_ramsey_0_10,stand(flu_ramsey_0),"k-",lw=3,alpha=0.5,label='B(0 G) no-HF')
            plt.plot(ts_dua_hf_ramsey_0_10,stand(flu_ramsey_dua_0),"k-.",
                     lw=3,alpha=0.5,label="B(0 G) HF Model 2")
            plt.plot(ts_doh_hf_ramsey_0_10,stand(flu_ramsey_doh_0),"k--",
                     lw=3,alpha=0.5,label="B(0 G) HF Model 3")
            
            plt.plot(ts_ramsey_0_10,stand(flu_ramsey),label='no-HF')
            plt.plot(ts_dua_hf_ramsey_0_10,stand(flu_ramsey_dua),ls="-.",label="HF Model 2")
            plt.plot(ts_doh_hf_ramsey_0_10,stand(flu_ramsey_doh),ls="--",label="HF Model 3")
            plt.minorticks_on()
            plt.ylabel("PL (a.u.)",fontsize=25)
            plt.xlabel('$\\tau (\\mu s)$',fontsize=25)
            plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
            plt.savefig(f'Figs\\Ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            print(f"----------------------FIT----------------------------")
            fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
            _,f_params_no=fit_plot_data(expo,ts_ramsey_0_10, flu_ramsey,
                          ["A","tau","D"],
                          [w1**2/(w1**2+w0**2),t2_gs,
                           (np.max(flu_ramsey).real+np.min(flu_ramsey).real)/2],
                          ax=ax_fit, label="No-HF",yaxis_label="PL (a.u.)",xaxis_label='$\\tau (\\mu s)$')
            plt.savefig(f'Figs\\Ramsey_fit_no_hf_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()

            fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
            _,f_params_dua=fit_plot_data(damp_cos,ts_dua_hf_ramsey_0_10, flu_ramsey_dua,
                          ["A","om","phi","tau","D"],
                          [1,omr,np.pi/2,t2_gs,
                           (np.max(flu_ramsey_dua)+np.min(flu_ramsey_dua)).real/2],
                          ax=ax_fit, label="HF Model 2",yaxis_label="PL (a.u.)",xaxis_label='$\\tau (\\mu s)$')
            plt.savefig(f'Figs\\Ramsey_fit_m2_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            print(f"from fit M2 B_0={f_params_dua['om'][0]/(2*np.pi*mu_e):.6g} ± {f_params_dua['om'][1]/(2*np.pi*mu_e):.8g}")
            print(f"from fit M2 t*_2={f_params_dua['tau'][0]:.6g} ± {f_params_dua['tau'][1]:.8g}")
            fig_fit,ax_fit=plt.subplots(figsize=(14, 8))
            _,f_params_doh=fit_plot_data(damp_cos,ts_doh_hf_ramsey_0_10, flu_ramsey_doh,
                          ["A","om","phi","tau","D"],
                          [1,omr,np.pi/2,t2_gs,
                           (np.max(flu_ramsey_doh)+np.min(flu_ramsey_doh)).real/2],
                          ax=ax_fit, label="HF Model 3",yaxis_label="PL (a.u.)",xaxis_label='$\\tau (\\mu s)$')
            plt.savefig(f'Figs\\Ramsey_fit_m3_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            print(f"from fit M3 B_0={f_params_doh['om'][0]/(2*np.pi*mu_e):.6g} ± {f_params_doh['om'][1]/(2*np.pi*mu_e):.8g}")
            print(f"from fit M3 t*_2={f_params_doh['tau'][0]:.6g} ± {f_params_doh['tau'][1]:.8g}")
            print(f"----------------------FFT--------------------------------")
            # FFT panels (one spectrum per trace)
            fig_fft, ax_fft = plt.subplots(figsize=(14, 8))
            #fit_params = [val[0] for val in f_params_no.values()]
            #t=np.linspace(0,10,500)
            #plot_fft(t, expo(t,*fit_params), 
            #         ax=ax_fft, label='no-HF')
            #fit_params = [val[0] for val in f_params_dua.values()]
            #t=np.linspace(0,10,500)
            #plot_fft(t, damp_sin(t,*fit_params),
            #         ax=ax_fft, label='HF Model 2')
            #fit_params = [val[0] for val in f_params_doh.values()]
            #t=np.linspace(0,10,500)
            #plot_fft(t, damp_sin(t,*fit_params),
            #         ax=ax_fft, label='HF Model 3')
            
            plot_fft(ts_ramsey_0_10, flu_ramsey,
                     ax=ax_fft, label='no-HF')
            plot_fft(ts_dua_hf_ramsey_0_10, flu_ramsey_dua,
                     ax=ax_fft, label='HF Model 2')
            plot_fft(ts_doh_hf_ramsey_0_10, flu_ramsey_doh,
                     ax=ax_fft, label='HF Model 3')
            plt.savefig(f'Figs\\Ramsey_fft_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            print(f"----------------------OTHER NORMS----------------------------")
            plt.figure(figsize=(14, 8))
            plt.title(f"Ramsey\n B={b:.2f} G, $\\Omega_r$={omr:.2f} MHz\nL2 Norm",fontsize=25)
            plt.plot(ts_ramsey_0_10,l2_norm(flu_ramsey),label='no-HF')
            plt.plot(ts_dua_hf_ramsey_0_10,l2_norm(flu_ramsey_dua),ls="-.",label="HF Duarte")
            plt.plot(ts_doh_hf_ramsey_0_10,l2_norm(flu_ramsey_doh),ls="--",label="HF Doherty")
            plt.minorticks_on()
            plt.ylabel("PL (a.u.)",fontsize=25)
            plt.xlabel('$\\tau (\\mu s)$',fontsize=25)
            plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
            plt.savefig(f'Figs\\Ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_l2.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            
            plt.figure(figsize=(14, 8))
            plt.title(f"Ramsey\n B={b:.2f} G, $\\Omega_r$={omr:.2f} MHz\nMax Abs Norm",fontsize=25)
            plt.plot(ts_ramsey_0_10,max_reg(flu_ramsey),label='no-HF')
            plt.plot(ts_dua_hf_ramsey_0_10,max_reg(flu_ramsey_dua),ls="-.",label="HF Duarte")
            plt.plot(ts_doh_hf_ramsey_0_10,max_reg(flu_ramsey_doh),ls="--",label="HF Doherty")
            plt.minorticks_on()
            plt.ylabel("PL (a.u.)",fontsize=25)
            plt.xlabel('$\\tau (\\mu s)$',fontsize=25)
            plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
            plt.savefig(f'Figs\\Ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_max_abs.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            
            plt.figure(figsize=(14, 8))
            plt.title(f"Ramsey\n B={b:.2f} G, $\\Omega_r$={omr:.2f} MHz\nMin Max Norm",fontsize=25)
            plt.plot(ts_ramsey_0_10,normaliz(flu_ramsey),label='no-HF')
            plt.plot(ts_dua_hf_ramsey_0_10,normaliz(flu_ramsey_dua),ls="-.",label="HF Duarte")
            plt.plot(ts_doh_hf_ramsey_0_10,normaliz(flu_ramsey_doh),ls="--",label="HF Doherty")
            plt.minorticks_on()
            plt.ylabel("PL (a.u.)",fontsize=25)
            plt.xlabel('$\\tau (\\mu s)$',fontsize=25)
            plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
            plt.savefig(f'Figs\\Ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_min_max.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            
            print("--------------------flu vs flur----------------------------")
            plt.figure(figsize=(14, 8))
            plt.title(f"Ramsey\n B={b:.2f} G, $\\Omega_r$={omr:.2f} MHz",fontsize=25)
            plt.plot(ts_ramsey_0_10,flu_ramsey,label='no-HF')
            plt.plot(ts_dua_hf_ramsey_0_10,flu_ramsey_dua,ls="-.",label="HF Duarte")
            plt.plot(ts_doh_hf_ramsey_0_10,flu_ramsey_doh,ls="--",label="HF Doherty")
            plt.minorticks_on()
            plt.ylabel("PL (a.u.)",fontsize=25)
            plt.xlabel('$\\tau (\\mu s)$',fontsize=25)
            plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
            plt.savefig(f'Figs\\Ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_flu.png',
                        bbox_inches='tight',dpi=400)
            plt.show()
            
            plt.figure(figsize=(14, 8))
            plt.title(f"Abubu Ramsey\n B={b:.2f} G, $\\Omega_r$={omr:.2f} MHz",fontsize=25)
            plt.plot(ts_ramsey_0_10,fls_ramsey_0_10,label='no-HF')
            plt.plot(ts_dua_hf_ramsey_0_10,fls_dua_hf_ramsey_0_10,ls="-.",label="HF Duarte")
            plt.plot(ts_doh_hf_ramsey_0_10,fls_doh_hf_ramsey_0_10,ls="--",label="HF Doherty")
            plt.minorticks_on()
            plt.ylabel("PL (a.u.)",fontsize=25)
            plt.xlabel('$\\tau (\\mu s)$',fontsize=25)
            plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
            plt.savefig(f'Figs\\Ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_flur.png',
                        bbox_inches='tight',dpi=400)
            plt.show()

# SNR Opt

In [ ]:
if run_opt:
    snr_no=[]
    snr_du=[]
    snr_do=[]
    b=100
    omr=15.8
    wI=2
    wR=2
    Ts=np.linspace(0.5,4,35)
    for tr in tqdm(Ts):
        init_state=1/3*(n1+n2+n3)
        exps=(n4+n5+n6)
        S=contrast(dynamics_no,init_state,exps,b,wI,wR,omr,readtime=tr)
        s0,s1=S[4],S[5] # 0,1 - 2,3 are non integrated read and full(10 \mu s) time, 
                        # respectively, 4,5 - 6,7 are the integrate in the same time spans 
        snr_no.append((s0-s1)/np.sqrt(0.5*(s0+s1)))
        init_state=1/6*((n1+n2+n3)&IdN15)
        exps=((n4+n5+n6)&IdN15)
        S=contrast(dynamics_doh_hf,init_state,exps,b,wI,wR,omr,readtime=tr)
        s0_doh,s1_doh=S[4],S[5]
        snr_do.append((s0_doh-s1_doh)/np.sqrt(0.5*(s0_doh+s1_doh)))
        S=contrast(dynamics_dua_hf,init_state,exps,b,wI,wR,omr,readtime=tr)
        s0_dua,s1_dua=S[4],S[5]
        snr_du.append((s0_dua-s1_dua)/np.sqrt(0.5*(s0_dua+s1_dua)))
    plt.figure(figsize=(14, 8))
    plt.plot(Ts,snr_no,label="NoHF")
    plt.plot(Ts,snr_du,label="Model 2")
    plt.plot(Ts,snr_do,label="Model 3")
    plt.minorticks_on()
    plt.ylabel("SNR (s0-s1)/$\\sqrt{0.5(s0+s1)}$ (a.u.)",fontsize=25)
    plt.xlabel('$\\tau_\\mathrm{R} (\\mu s)$',fontsize=25)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    print(Ts[np.argmax(snr_no)],Ts[np.argmax(snr_du)],Ts[np.argmax(snr_do)])
    

In [ ]:
if run_opt:
    snr_no_wR=[]
    snr_du_wR=[]
    snr_do_wR=[]
    b=100
    omr=15.8
    Ws=np.linspace(0.5,5.5,50)
    tr_op=3#Ts[np.argmax(snr_no)]
    for wp in tqdm(Ws):
        init_state=1/3*(n1+n2+n3)
        exps=(n4+n5+n6)
        S=contrast(dynamics_no,init_state,exps,b,wI,wp,omr,readtime=tr_op)
        s0,s1=S[4],S[5] # 0,1 - 2,3 are non integrated read and full(10 \mu s) time, 
                        # respectively, 4,5 - 6,7 are the integrate in the same time spans 
        snr_no_wR.append((s0-s1)/np.sqrt(0.5*(s0+s1)))
        init_state=1/6*((n1+n2+n3)&IdN15)
        exps=((n4+n5+n6)&IdN15)
        S=contrast(dynamics_doh_hf,init_state,exps,b,wI,wp,omr,readtime=tr_op)
        s0_doh,s1_doh=S[4],S[5]
        snr_do_wR.append((s0_doh-s1_doh)/np.sqrt(0.5*(s0_doh+s1_doh)))
        S=contrast(dynamics_dua_hf,init_state,exps,b,wI,wp,omr,readtime=tr_op)
        s0_dua,s1_dua=S[4],S[5]
        snr_du_wR.append((s0_dua-s1_dua)/np.sqrt(0.5*(s0_dua+s1_dua)))
    plt.figure(figsize=(14, 8))
    plt.plot(Ws,snr_no_wR,label="NoHF")
    plt.plot(Ws,snr_du_wR,label="Model 2")
    plt.plot(Ws,snr_do_wR,label="Model 3")
    plt.minorticks_on()
    plt.ylabel("SNR (s0-s1)/$\\sqrt{0.5(s0+s1)}$ (a.u.)",fontsize=25)
    plt.xlabel('$W^R_\\mathrm{P}$ (MHz)',fontsize=25)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    print(Ws[np.argmax(snr_no_wR)],Ws[np.argmax(snr_du_wR)],Ws[np.argmax(snr_do_wR)])

In [ ]:
if run_opt:
    snr_no_omr=[]
    snr_du_omr=[]
    snr_do_omr=[]
    b=100
    Oms=np.linspace(4,22,100)
    w_op=Ws[np.argmax(snr_no_wR)]
    tr_op=Ts[np.argmax(snr_no)]
    for om in tqdm(Oms):
        init_state=1/3*(n1+n2+n3)
        exps=(n4+n5+n6)
        S=contrast(dynamics_no,init_state,exps,b,wI,w_op,om,readtime=tr_op)
        s0,s1=S[4],S[5] # 0,1 - 2,3 are non integrated read and full(10 \mu s) time, 
                        # respectively, 4,5 - 6,7 are the integrate in the same time spans 
        snr_no_omr.append((s0-s1)/np.sqrt(0.5*(s0+s1)))
        init_state=1/6*((n1+n2+n3)&IdN15)
        exps=((n4+n5+n6)&IdN15)
        S=contrast(dynamics_doh_hf,init_state,exps,b,wI,w_op,om,readtime=tr_op)
        s0_doh,s1_doh=S[4],S[5]
        snr_do_omr.append((s0_doh-s1_doh)/np.sqrt(0.5*(s0_doh+s1_doh)))
        S=contrast(dynamics_dua_hf,init_state,exps,b,wI,w_op,om,readtime=tr_op)
        s0_dua,s1_dua=S[4],S[5]
        snr_du_omr.append((s0_dua-s1_dua)/np.sqrt(0.5*(s0_dua+s1_dua)))
    plt.figure(figsize=(14, 8))
    plt.plot(Oms,snr_no_omr,label="NoHF")
    plt.plot(Oms,snr_du_omr,label="Model 2")
    plt.plot(Oms,snr_do_omr,label="Model 3")
    plt.minorticks_on()
    plt.ylabel("SNR (s0-s1)/$\\sqrt{0.5(s0+s1)}$ (a.u.)",fontsize=25)
    plt.xlabel('$\\Omega_\\mathrm{R}$ (MHz)',fontsize=25)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    print(Oms[np.argmax(snr_no_omr)],Oms[np.argmax(snr_du_omr)],Oms[np.argmax(snr_do_omr)])

In [ ]:
if run_opt:
    snr_no_tR=[]
    snr_du_tR=[]
    snr_do_tR=[]
    b=100
    omr_op=Oms[np.argmax(snr_no_omr)]
    wR_op=Ws[np.argmax(snr_no_wR)]
    Ts=np.linspace(0.5,4,35)
    wI=2
    for tr in tqdm(Ts):
        init_state=1/3*(n1+n2+n3)
        exps=(n4+n5+n6)
        S=contrast(dynamics_no,init_state,exps,b,wI,wR_op,omr_op,readtime=tr)
        s0,s1=S[4],S[5] # 0,1 - 2,3 are non integrated read and full(10 \mu s) time, 
                        # respectively, 4,5 - 6,7 are the integrate in the same time spans 
        snr_no_tR.append((s0-s1)/np.sqrt(0.5*(s0+s1)))
        init_state=1/6*((n1+n2+n3)&IdN15)
        exps=((n4+n5+n6)&IdN15)
        S=contrast(dynamics_doh_hf,init_state,exps,b,wI,wR_op,omr_op,readtime=tr)
        s0_doh,s1_doh=S[4],S[5]
        snr_do_tR.append((s0_doh-s1_doh)/np.sqrt(0.5*(s0_doh+s1_doh)))
        S=contrast(dynamics_dua_hf,init_state,exps,b,wI,wR_op,omr_op,readtime=tr)
        s0_dua,s1_dua=S[4],S[5]
        snr_du_tR.append((s0_dua-s1_dua)/np.sqrt(0.5*(s0_dua+s1_dua)))
    plt.figure(figsize=(14, 8))
    plt.plot(Ts,snr_no_tR,label="NoHF")
    plt.plot(Ts,snr_du_tR,label="Model 2")
    plt.plot(Ts,snr_do_tR,label="Model 3")
    plt.minorticks_on()
    plt.ylabel("SNR (s0-s1)/$\\sqrt{0.5(s0+s1)}$ (a.u.)",fontsize=25)
    plt.xlabel('$\\tau_\\mathrm{R} (\\mu s)$',fontsize=25)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    print(Ts[np.argmax(snr_no_tR)],Ts[np.argmax(snr_du_tR)],Ts[np.argmax(snr_do_tR)])

In [ ]:
if run_opt:
    snr_no_wI=[]
    snr_du_wI=[]
    snr_do_wI=[]
    b=100
    omr_op=Oms[np.argmax(snr_no_omr)]
    wR_op=Ws[np.argmax(snr_no_wR)]
    tr_op=Ts[np.argmax(snr_no_tR)]
    Ws=np.linspace(0.5,5.5,50)
    for wp in tqdm(Ws):
        init_state=1/3*(n1+n2+n3)
        exps=(n4+n5+n6)
        S=contrast(dynamics_no,init_state,exps,b,wp,wR_op,omr_op,readtime=tr_op)
        s0,s1=S[4],S[5] # 0,1 - 2,3 are non integrated read and full(10 \mu s) time, 
                        # respectively, 4,5 - 6,7 are the integrate in the same time spans 
        snr_no_wI.append((s0-s1)/np.sqrt(0.5*(s0+s1)))
        init_state=1/6*((n1+n2+n3)&IdN15)
        exps=((n4+n5+n6)&IdN15)
        S=contrast(dynamics_doh_hf,init_state,exps,b,wp,wR_op,omr_op,readtime=tr_op)
        s0_doh,s1_doh=S[4],S[5]
        snr_do_wI.append((s0_doh-s1_doh)/np.sqrt(0.5*(s0_doh+s1_doh)))
        S=contrast(dynamics_dua_hf,init_state,exps,b,wp,wR_op,omr_op,readtime=tr_op)
        s0_dua,s1_dua=S[4],S[5]
        snr_du_wI.append((s0_dua-s1_dua)/np.sqrt(0.5*(s0_dua+s1_dua)))
    plt.figure(figsize=(14, 8))
    plt.plot(Ws,snr_no_wI,label="NoHF")
    plt.plot(Ws,snr_du_wI,label="Model 2")
    plt.plot(Ws,snr_do_wI,label="Model 3")
    plt.minorticks_on()
    plt.ylabel("SNR (s0-s1)/$\\sqrt{0.5(s0+s1)}$ (a.u.)",fontsize=25)
    plt.xlabel('$W^I_\\mathrm{P}$ (a.u.)',fontsize=25)
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    print(Ws[np.argmax(snr_no_wI)],Ws[np.argmax(snr_du_wI)],Ws[np.argmax(snr_do_wI)])

In [ ]:
def run_seq_w(model,k,init_state,w):
    results=[]
    Om_r = 15.8
    W_p = w
    bs=B0(100,0.0,0)
    omega = D_gs-mu_e*bs[2]
    _,result=model(10, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=0.0,k_index=k, mode="Laser", progress_bar="OFF")
    results.append(result)
    init_state=result.states[-1]
    ti=result.times[-1]
    _,result=model(2, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=ti, k_index=k, mode="Free", progress_bar="OFF")
    results.append(result)
    init_state=result.states[-1]
    ti=result.times[-1]
    _,result=model(5, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=ti, k_index=k, mode="MW", progress_bar="OFF")
    results.append(result)
    init_state=result.states[-1]
    ti=result.times[-1]
    _,result=model(5, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=ti, k_index=k, mode="Laser", progress_bar="OFF")
    results.append(result)
    times=np.concatenate([r.times for r in results])
    return results,times

In [ ]:
#for w in tqdm(np.linspace(1,10,10)):
#    colors = [
#            "dodgerblue",
#            "chocolate",
#            "darkgoldenrod",
#            "mediumpurple",
#            "mediumseagreen",
#            "lightskyblue",
#            "magenta",
#            "forestgreen",
#        ]
#    no,t_no=run_seq_w(dynamics_no,2,(n1+n2+n3)/3,w)
#    do,t_do=run_seq_w(dynamics_doh_hf,2,((n1+n2+n3)&IdN15)/6,w)
#    exp_ops_no=[n1,n2,n3,n4,n5,n6,n7,nc]
#    exp_ops_do=[n1&IdN15,n2&IdN15,n3&IdN15,n4&IdN15,n5&IdN15,n6&IdN15,n7&IdN15,nc&IdN15]
#        # build a list of psi arrays, one per operator
#    psi_no = [np.concatenate([qt.expect(op, R.states) for R in no]) for op in exp_ops_no]
#        # build a list of psi arrays, one per operator
#    psi_do = [np.concatenate([qt.expect(op, R.states) for R in do]) for op in exp_ops_do]
#    plt.figure(figsize=(14, 8))
#    plt.title(f"Populations for \n $W_P=${w:.4g}",fontsize=25)
#    for i,e_p in enumerate(psi_do):
#        plt.plot(t_do,e_p,label=f"$n^{{hf}}_{i+1}$" if i<7 else "$n^{{hf}}_c$",lw=5,color=colors[i])
#    for i,e_p in enumerate(psi_no):
#        plt.plot(t_no,e_p,label=f"$n_{i+1}$" if i<7 else "$n_c$",lw=5,ls="--",color=colors[i])
#    plt.minorticks_on()
#    plt.ylabel("Population (a.u.)",fontsize=25)
#    plt.xlabel('Times ($\\mu$s)',fontsize=25)
#    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
#    plt.show()

# PL vs B

In [ ]:
# -------------------------------------------------------------------
# 1) Configuration matching save loop
# -------------------------------------------------------------------
final_b   = 1300
delt_b=5
ang_degs  = [0, 0.5, 1, 4, 8, 10]
# -------------------------------------------------------------------
# 2) Container for all loaded data
# -------------------------------------------------------------------
all_data = {}
# -------------------------------------------------------------------
# 3) Iterate over angles and reload files
# -------------------------------------------------------------------
for ang_deg in ang_degs:
    # match the save naming convention
    adeg = int(10 * ang_deg)
    key  = f"{ang_deg}"
    all_data[key] = {}
    # --- 3.1 Load final_states arrays ---
    all_data[key]['final_states']    = np.load(
        f'NumpyArrays/final_states_PL_v_B_{final_b}_ang_deg_{adeg}.npy',
        allow_pickle=True
    )
    all_data[key]['final_states_hf'] = np.load(
        f'NumpyArrays/final_states_hf_PL_v_B_{final_b}_ang_deg_{adeg}.npy',
        allow_pickle=True
    )
    
colors = [
            "dodgerblue",
            "chocolate",
            "darkgoldenrod",
            "mediumpurple",
            "mediumseagreen",
            "magenta",
        ]
fig, ax = plt.subplots(figsize=(14, 8))
# No‑HF 
for idx, ang in enumerate(ang_degs):
    key = f"{ang}"
    states = all_data[key]['final_states']
    y = [qt.expect(n4 + n5 + n6, state) for state in states]
    ax.plot(range(0,final_b,delt_b), y,ls="-",alpha=0.6,color=colors[idx])
ax.set_xlabel("B (G)",fontsize=25)
ax.set_ylabel("PL (a.u.)",fontsize=25)
# HF M3
for idx, ang in enumerate(ang_degs):
    key = f"{ang}"
    states_hf = all_data[key]['final_states_hf']
    yhf = [qt.expect((n4 + n5 + n6) & IdN15, state) for state in states_hf]
    ax.plot(range(0,final_b,delt_b), yhf,ls="--",color=colors[idx],marker="o",markevery=20,markersize=10)
# Create legend
handles = []
# Add angle legend
for idx, ang in enumerate(ang_degs):
    handles.append(mpl.lines.Line2D([], [], label=rf"$\theta_B=$ {ang}$^\circ$",color=colors[idx])) # type: ignore

# Add model legend
handles.append(mpl.lines.Line2D([], [], color='black', linestyle='-',alpha=0.5, # type: ignore
                        label="No-HF"))
handles.append(mpl.lines.Line2D([], [], color='black', linestyle='--',marker="o",markersize=10, # type: ignore
                        label="Model 3"))

ax.legend(handles=handles, bbox_to_anchor=(1.05, 1), 
             loc='upper left', borderaxespad=0.,fontsize=25)
ax.set_title("Photoluminescence (PL) vs Magnetic Field (B)",fontsize=25)
plt.savefig(f"Figs\\pl-v-b-angs.png",bbox_inches='tight',dpi=400)
#fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# -------------------------------------------------------------------
# 1) Configuration matching save loop
# -------------------------------------------------------------------
final_b   = 1300
delt_b=5
ang_degs  = [0, 0.5, 1]
# -------------------------------------------------------------------
# 2) Container for all loaded data
# -------------------------------------------------------------------
all_data = {}
# -------------------------------------------------------------------
# 3) Iterate over angles and reload files
# -------------------------------------------------------------------
for ang_deg in ang_degs:
    # match the save naming convention
    adeg = int(10 * ang_deg)
    key  = f"{ang_deg}"
    all_data[key] = {}
    # --- 3.1 Load final_states arrays ---
    all_data[key]['final_states']    = np.load(
        f'NumpyArrays/final_states_PL_v_B_{final_b}_ang_deg_{adeg}.npy',
        allow_pickle=True
    )
    all_data[key]['final_states_hf'] = np.load(
        f'NumpyArrays/final_states_hf_PL_v_B_{final_b}_ang_deg_{adeg}.npy',
        allow_pickle=True
    )
    
colors = [
            "dodgerblue",
            "chocolate",
            "darkgoldenrod",
            "mediumpurple",
            "mediumseagreen",
            "magenta",
        ]
fig, ax = plt.subplots(figsize=(14, 8))
# No‑HF 
for idx, ang in enumerate(ang_degs):
    key = f"{ang}"
    states = all_data[key]['final_states']
    y = [qt.expect(n4 + n5 + n6, state) for state in states]
    ax.plot(range(0,final_b,delt_b)[:len(y)//2], y[:len(y)//2],ls="-",alpha=0.6,color=colors[idx])
ax.set_xlabel("B (G)",fontsize=25)
ax.set_ylabel("PL (a.u.)",fontsize=25)
# HF M3
for idx, ang in enumerate(ang_degs):
    key = f"{ang}"
    states_hf = all_data[key]['final_states_hf']
    yhf = [qt.expect((n4 + n5 + n6) & IdN15, state) for state in states_hf]
    ax.plot(range(0,final_b,delt_b)[:len(yhf)//2], yhf[:len(yhf)//2],ls="--",color=colors[idx],marker="o",markevery=20,markersize=10)
# Create legend
handles = []
# Add angle legend
for idx, ang in enumerate(ang_degs):
    handles.append(mpl.lines.Line2D([], [], label=rf"$\theta_B=$ {ang}$^\circ$",color=colors[idx])) # type: ignore

# Add model legend
handles.append(mpl.lines.Line2D([], [], color='black', linestyle='-',alpha=0.5, # type: ignore
                        label="No-HF"))
handles.append(mpl.lines.Line2D([], [], color='black', linestyle='--',marker="o",markersize=10, # type: ignore
                        label="Model 3"))

ax.legend(handles=handles, bbox_to_anchor=(1.05, 1), 
             loc='upper left', borderaxespad=0.,fontsize=25)
ax.set_title("Photoluminescence (PL) vs Magnetic Field (B)",fontsize=25)
plt.savefig(f"Figs\\pl-v-b-angs_1-0-half.png",bbox_inches='tight',dpi=400)
#fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

In [ ]:
# -------------------------------------------------------------------
# 1) Configuration matching save loop
# -------------------------------------------------------------------
final_b   = 1300
delt_b=5
ang_degs  = [0, 0.5, 1]
# -------------------------------------------------------------------
# 2) Container for all loaded data
# -------------------------------------------------------------------
all_data = {}
# -------------------------------------------------------------------
# 3) Iterate over angles and reload files
# -------------------------------------------------------------------
for ang_deg in ang_degs:
    # match the save naming convention
    adeg = int(10 * ang_deg)
    key  = f"{ang_deg}"
    all_data[key] = {}
    # --- 3.1 Load final_states arrays ---
    all_data[key]['final_states']    = np.load(
        f'NumpyArrays/final_states_PL_v_B_{final_b}_ang_deg_{adeg}.npy',
        allow_pickle=True
    )
    all_data[key]['final_states_hf'] = np.load(
        f'NumpyArrays/final_states_hf_PL_v_B_{final_b}_ang_deg_{adeg}.npy',
        allow_pickle=True
    )
    
colors = [
            "dodgerblue",
            "chocolate",
            "darkgoldenrod",
            "mediumpurple",
            "mediumseagreen",
            "magenta",
        ]
fig, ax = plt.subplots(figsize=(14, 8))
# No‑HF 
for idx, ang in enumerate(ang_degs):
    key = f"{ang}"
    states = all_data[key]['final_states']
    y = [qt.expect(n4 + n5 + n6, state) for state in states]
    ax.plot(range(0,final_b,delt_b)[len(y)//2:], y[len(y)//2:],ls="-",alpha=0.6,color=colors[idx])
ax.set_xlabel("B (G)",fontsize=25)
ax.set_ylabel("PL (a.u.)",fontsize=25)
# HF M3
for idx, ang in enumerate(ang_degs):
    key = f"{ang}"
    states_hf = all_data[key]['final_states_hf']
    yhf = [qt.expect((n4 + n5 + n6) & IdN15, state) for state in states_hf]
    ax.plot(range(0,final_b,delt_b)[len(yhf)//2:], yhf[len(yhf)//2:],ls="--",color=colors[idx],marker="o",markevery=20,markersize=10)
# Create legend
handles = []
# Add angle legend
for idx, ang in enumerate(ang_degs):
    handles.append(mpl.lines.Line2D([], [], label=rf"$\theta_B=$ {ang}$^\circ$",color=colors[idx])) # type: ignore

# Add model legend
handles.append(mpl.lines.Line2D([], [], color='black', linestyle='-',alpha=0.5, # type: ignore
                        label="No-HF"))
handles.append(mpl.lines.Line2D([], [], color='black', linestyle='--',marker="o",markersize=10, # type: ignore
                        label="Model 3"))

ax.legend(handles=handles, bbox_to_anchor=(1.05, 1), 
             loc='upper left', borderaxespad=0.,fontsize=25)
ax.set_title("Photoluminescence (PL) vs Magnetic Field (B)",fontsize=25)
plt.savefig(f"Figs\\pl-v-b-angs_1-half.png",bbox_inches='tight',dpi=400)
#fig.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Comparison of Transition Rates

In [ ]:
ks=[[66.0,0.0,57.0,1.0,0.7],
    [77.0,0.0,30.0,3.3,0.0],
    [62.7,12.97,80.0,3.45,1.08],
    [63.2,10.8,60.7,0.8,0.4],
    [67.4,9.9,96.6,4.83,1.055],
    [64.0,11.8,79.8,5.6,0.0]]


In [ ]:
def run_seq(model,k,init_state):
    results=[]
    Om_r = 15.7
    W_p = 2
    bs=B0(100,0.0,0)
    omega = D_gs-mu_e*bs[2]
    _,result=model(10, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=0.0,k_index=k, mode="Laser", progress_bar="OFF")
    results.append(result)
    init_state=result.states[-1]
    ti=result.times[-1]
    _,result=model(2, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=ti, k_index=k, mode="Free", progress_bar="OFF")
    results.append(result)
    init_state=result.states[-1]
    ti=result.times[-1]
    _,result=model(5, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=ti, k_index=k, mode="MW", progress_bar="OFF")
    results.append(result)
    init_state=result.states[-1]
    ti=result.times[-1]
    _,result=model(5, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=ti, k_index=k, mode="Laser", progress_bar="OFF")
    results.append(result)
    times=np.concatenate([r.times for r in results])
    return results,times

In [ ]:
if run_ks:
    results_ks_no = []
    results_ks_doh = []
    results_ks_dua = []
    ns=[n1,n2,n3,n4,n5,n6,n7,nc]
    for k in range(len(ks)):
        colors = [
            "dodgerblue",
            "chocolate",
            "darkgoldenrod",
            "mediumpurple",
            "mediumseagreen",
            "lightskyblue",
            "magenta",
            "forestgreen",
        ]
        results_no,times_no=run_seq(dynamics_no,k,(n1+n2+n3)/3)
        results_doh,times_doh=run_seq(dynamics_doh_hf,k,((n1+n2+n3)&IdN15)/6)
        results_dua,times_dua=run_seq(dynamics_dua_hf,k,((n1+n2+n3)&IdN15)/6)
        # Plotting
        results_ks_no.append((k,results_no))
        results_ks_doh.append((k,results_doh))
        results_ks_dua.append((k,results_dua))
    np.save(f'NumpyArrays\\results_ks_no.npy',np.array(results_ks_no,dtype=object),allow_pickle=True)
    np.save(f'NumpyArrays\\results_ks_doh.npy',np.array(results_ks_doh,dtype=object),allow_pickle=True)
    np.save(f'NumpyArrays\\results_ks_dua.npy',np.array(results_ks_dua,dtype=object),allow_pickle=True)
else:
    results_ks_no=np.load(f'NumpyArrays\\results_ks_no.npy',allow_pickle=True)
    results_ks_doh=np.load(f'NumpyArrays\\results_ks_doh.npy',allow_pickle=True)
    results_ks_dua=np.load(f'NumpyArrays\\results_ks_dua.npy',allow_pickle=True)

def unpack(rs):
    return {k: res_list for k, res_list in rs}

data_no  = unpack(results_ks_no)
data_doh = unpack(results_ks_doh)
data_dua = unpack(results_ks_dua)

ns = [n1, n2, n3, n4, n5, n6, n7, nc]
ns_hf = [n1 & IdN15, n2 & IdN15, n3 & IdN15, n4 & IdN15, n5 & IdN15, n6 & IdN15, n7 & IdN15, nc & IdN15]
markers = ['o','s','^','v','P','X','D','*']  # up to 8 distinct marks
palette = [
    "dodgerblue","chocolate","darkgoldenrod","mediumpurple",
    "mediumseagreen","lightskyblue","magenta","forestgreen"
]
# -------------------------------------------------------------------
# Plotting helper with optional zoom
# -------------------------------------------------------------------
def plot_regime(data, title, exp_ops, zoom_pts=None, highlight_k=None, colors=palette):
    """
    data: dict k→list of Results
    zoom_pts: int or None. if set, makes a second 'zoom' figure of t[:zoom_pts].
    highlight_k: which k to highlight with colors; others get gray.
    colors: list of length len(ns) of colors for highlight_k curves.
    """
    def _do_plot(ax, t, psi_dict, colors=colors):
        st=[1,2,3,4,5,6,7,'c']
        for k, psi_all in psi_dict.items():
            for i, psi in enumerate(psi_all):
                if k == highlight_k:
                    c = colors[i]
                    lw = 6.0
                    alpha = 1.0
                    m = markers[k % len(markers)]
                    label = f'$K_{{{k+1}}}$, n$_{st[i]}$'
                else:
                    c = 'gray'
                    lw = 4.0
                    alpha = 0.4
                    m = markers[k % len(markers)]
                    label = f'$K_{{{k+1}}}$' if i == 0 else None  # only label first operator
                ax.plot(
                    t, psi,
                    color=c, lw=lw, alpha=alpha,
                    linestyle='-',
                    marker=m, markevery=500,markersize=10,
                    label=label
                )
        ax.set_xlabel('Time',fontsize=25)
        ax.set_ylabel('Population',fontsize=25)
        ax.set_title(title + (" (zoom)" if ax.get_title().endswith("(zoom)") else ""),fontsize=25)
        ax.grid(True, linestyle=':', alpha=0.6)
        if highlight_k is not None:
            ax.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)

    # build full traces
    psi_full = {}
    for k, results in data.items():
        t_full = np.concatenate([r.times for r in results])
        # build a list of psi arrays, one per operator
        psi_full[k] = [
            np.concatenate([qt.expect(op, R.states) for R in results])
            for op in exp_ops
        ]

    # 1) full plot
    fig, ax = plt.subplots(figsize=(14, 8))
    _do_plot(ax, t_full, psi_full)
    plt.show()

    # 2) zoom plot, if requested
    if zoom_pts is not None:
        fig, ax = plt.subplots(figsize=(14, 8))
        t_zoom = t_full[:zoom_pts]
        psi_zoom = {
            k: [psi[:zoom_pts] for psi in psi_list]
            for k, psi_list in psi_full.items()
        }
        _do_plot(ax, t_zoom, psi_zoom)
        plt.show()


# -------------------------------------------------------------------
# Now call once per regime, highlighting K=2 and zooming first 100 pts
# -------------------------------------------------------------------

plot_regime(data_no,
            title="No‑HF: population vs time for varying K",
            exp_ops=ns,
            zoom_pts=2000,
            highlight_k=2,
            colors=palette)

plot_regime(data_doh,
            title="Model 3: population vs time for varying K",
            exp_ops=ns_hf,
            zoom_pts=2000,
            highlight_k=2,
            colors=palette)

plot_regime(data_dua,
            title="Model 2: population vs time for varying K",
            exp_ops=ns_hf,
            zoom_pts=2000,
            highlight_k=2,
            colors=palette) 

# Other comparisson tables

## PL no HF Pure v Prepared

In [ ]:
pure_no_hf=[]
for b_pl in [100,510]:
    bs=B0(b_pl,0.0,0.0)
    Om_r = 15.7
    W_p = 2
    om=D_gs-mu_e*bs[2]
    c = 0
    print("Starting calculations, this may take a while...")
    for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
        init_state = n1 
        if dt_mw==0.5*np.pi/Om_r:
            init_state = (n1 + n2 + ground[1]*ground[2].dag() + ground[2]*ground[1].dag())/2 #type:ignore
        if dt_mw==np.pi/Om_r:   
            init_state = n2

        # Array to collect the result Class
        results_t = np.array([])

        # Initial time of the evolution
        ti = 0.0

        # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
        tis_t = []
        tfs_t = []

        # Set the number of times to repeat the Sequence
        n = 1
        # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
        dt_laser = 10.0
        dt_free = 1.0
        # This loop is defined to reproduce the Sequence of Magalleti el al
        for i in tqdm(range(n),position=1,leave=False):
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics_no based on the mode choosen
            tf, result = dynamics_no(dt_laser,
                                     init_state,
                                     b=bs, 
                                     om_r=Om_r,
                                     om=om,
                                     w_p=W_p,
                                     ti=ti,
                                     mode="Laser",
                                     progress_bar="ON",
                                     i=i) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the Sequence
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
        times_t = np.concatenate([res.times for res in results_t])
        N = [n1, n2, n3, n4, n5, n6, n7, nc]
        n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
        S = [sx_gs, sy_gs, sz_gs, sx_es, sy_es, sz_es]
        s_exp_t = np.array([np.concatenate([qt.expect(M, res.states)  for res in results_t]) for M in S])
        # Gather the expectation values from the results array
        #for j in range(len(results_t[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(results_t)):
        #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
        #    if j == 0:
        #        n_exp_t = np.array([nn_exp])
        #    else:
        #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        # Gather the times for the plots
        #times_t = np.array([])
        #for i in range(len(results_t)):
        #    times_t = np.append(times_t, results_t[i].times)
        #if len(n_exp)>8:
        #    s_exp_t=n_exp_t[len(n_exp_t)-6:]
        #    n_exp_t=n_exp_t[:len(n_exp_t)-6]
        pl=np.array([])
        pl=np.add(n_exp_t[3],n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        if c==0:
            pls_pure = np.array([pl])
            c+=1
        else:
            pls_pure = np.append(pls_pure,[pl],axis=0)
    print(f"Calculations are finished!! See the plots below")
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls_pure:
        plt.plot(times_t[0:],pl[0:], lw=5, color=colors[a], label=f"init state=${state[a]}$")
        a+=1
    times_t_pure=times_t[0:].copy()
    eps_y = abs(np.min(pls_pure) - np.max(pls_pure))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"Pure states PL for B={b_pl} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (times_t[0]-eps_x, times_t[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(pls_pure) - eps_y, np.max(pls_pure) + eps_y))
    plt.minorticks_on()

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    pure_no_hf.append((b_pl,pls_pure,times_t_pure))

In [ ]:
prep_no_hf=[]
for b_pl in [100,510]:
    bs=B0(b_pl,0.0,0.0)
    Om_r = 15.7
    W_p = 2
    om=D_gs-mu_e*bs[2]
    c = 0
    print("Starting calculations, this may take a while...")
    for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
        init_state = 1 / 3 * (n1 + n2 + n3)

        # Array to collect the result Class
        results_t = np.array([])

        # Initial time of the evolution
        ti = 0.0

        # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
        tis_t = []
        tfs_t = []

        # Set the number of times to repeat the Sequence
        n = 1
        # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
        dt_laser = 10.0
        dt_free = 1.0
        # This loop is defined to reproduce the Sequence of Magalleti el al
        for i in tqdm(range(n),position=1,leave=False):
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_no(dt_laser,
                                     init_state,
                                     b=bs, 
                                     om_r=Om_r,
                                     om=om,
                                     w_p=W_p,
                                     ti=ti,
                                     mode="Laser",
                                     progress_bar="ON",
                                     i=i) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the Sequence
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_no(dt_free,
                                     init_state,
                                     b=bs, 
                                     om_r=Om_r,
                                     om=om,
                                     w_p=W_p,
                                     ti=ti,
                                     mode="Free",
                                     progress_bar="ON",
                                     i=i) # type:ignore
            # Store the Result class for MW on evolution
            results_t = np.append(results_t, result) # type: ignore 
            # Save the times for coloring areas of the plot
            tis_t.append(ti)
            tfs_t.append(tf)
            # Make initial time to be the end time of the last part of the Sequence
            ti = tf
            # Make the last state  of this evolution step be the initial states of the next iteration
            init_state = result.states[-1]
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_no(dt_mw, 
                                     init_state,
                                     b=bs, 
                                     om_r=Om_r,
                                     om=om,
                                     w_p=W_p, 
                                     ti=ti, 
                                     mode="MW", 
                                     progress_bar="ON", 
                                     i=i) # type:ignore
            # Store the Result class for MW on evolution
            results_t = np.append(results_t, result) # type: ignore
            # Save the times for coloring areas of the plot
            tis_t.append(ti)
            tfs_t.append(tf)
            # Make initial time to be the end time of the last part of the Sequence
            ti = tf
            # Make the last state  of this evolution step be the initial states of the next iteration
            init_state = result.states[-1]
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_no(dt_laser,
                                     init_state,
                                     b=bs, 
                                     om_r=Om_r,
                                     om=om,
                                     w_p=W_p,
                                     ti=ti,
                                     mode="Laser",
                                     progress_bar="ON",
                                     i=i+1) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the Sequence
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
        times_t = np.concatenate([res.times for res in results_t])
        N = [n1, n2, n3, n4, n5, n6, n7, nc]
        n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
        S = [sx_gs, sy_gs, sz_gs, sx_es, sy_es, sz_es]
        s_exp_t = np.array([np.concatenate([qt.expect(M, res.states)  for res in results_t]) for M in S])
        # Gather the expectation values from the results array
        #for j in range(len(results_t[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(results_t)):
        #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
        #    if j == 0:
        #        n_exp_t = np.array([nn_exp])
        #    else:
        #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        ## Gather the times for the plots
        #times_t = np.array([])
        #for i in range(len(results_t)):
        #    times_t = np.append(times_t, results_t[i].times)
        #if len(n_exp)>8:
        #    s_exp_t=n_exp_t[len(n_exp_t)-6:]
        #    n_exp_t=n_exp_t[:len(n_exp_t)-6]
        pl=np.array([])
        pl=np.add(n_exp_t[3],n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        if c==0:
            pls = np.array([pl])
            c+=1
        else:
            pls = np.append(pls,[pl],axis=0)
    print(f"Calculations are finished!! See the plots below")
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls:
        plt.plot(times_t[-5200:],pl[-5200:], lw=5, color=colors[a], label=f"init state=${state[a]}$")
        a+=1

    eps_y = abs(np.min(pls) - np.max(pls))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"B={b_pl:.2f} G, $\\Omega_r$={Om_r:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (times_t[-5200]-eps_x, times_t[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(pls) - eps_y, np.max(pls) + eps_y))
    plt.minorticks_on()

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    prep_no_hf.append((b_pl,pls,times_t))

In [ ]:
for (b_pls,pls_pure,times_t_pure),(b_pl,pls,times_t) in zip(pure_no_hf,prep_no_hf):
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls_pure:
        plt.plot(times_t_pure[0:],pl[0:], lw=5, ls='--', color=colors[a], label=f"init Pure state=${state[a]}$")
        a+=1
    a=0
    for pl in pls:
        plt.plot(times_t[-5200:]-11.2,pl[-5200:], lw=5, color=colors[a], label=f"init state=${state[a]}$")
        a+=1
    eps_y = abs(np.min(pls_pure) - np.max(pls_pure))*5e-2
    eps_x = abs(np.min(times_t[-5200:]) - np.max(times_t[-5200:]))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"B={b_pls:.2f} G, $\\Omega_r$={Om_r:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(pls_pure) - eps_y, np.max(pls_pure) + eps_y))
    plt.minorticks_on()

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pl-no-pure-v-prep-b-{int(b_pls)}-omr-{int(100*Om_r)}.png",bbox_inches='tight',
                dpi=400)
    plt.show()
    a=0
    print(f"B={b_pl:.2f} G, $\\Omega_r$={Om_r:.2f} MHz")
    for pl_p,pl_prep in zip(pls_pure,pls):
        summary = compare_regions(pl_p, pl_prep, [(-5000,-2500)], [state[a]])
        if a==0:
            pl_0 = pl_prep
        if a==2:
            pl_1 = pl_prep
        a+=1
        print_summary_table(summary)
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'|1\rangle']
    for pl in [pl_0,pl_1]:
        plt.plot(times_t[-5200:]-11.2,pl[-5200:]/np.max(pl_0), lw=5, color=colors[a], label=f"init state=${state[a]}$")
        a+=1
    plt.plot(times_t[-5200:]-11.2,pl_0[-5200:]/np.max(pl_0)-pl_1[-5200:]/np.max(pl_0), 
             lw=5, color=colors[-1], label=f"Contrast")
    print(f"Max contrast(%): {100*np.max(pl_0[-5200:]/np.max(pl_0)-pl_1[-5200:]/np.max(pl_0))}")
    eps_y = abs(np.min(np.append(pl_0/np.max(pl_0),pl_1/np.max(pl_0))) - np.max(np.append(pl_0/np.max(pl_0),
                                                                                          pl_1/np.max(pl_0))))*5e-2
    eps_x = abs(np.min(times_t[-5200:]) - np.max(times_t[-5200:]))*5e-2
    plt.ylabel(f"%PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"B={b_pls:.2f} G, $\\Omega_r$={Om_r:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(pl_0/np.max(pl_0),pl_1/np.max(pl_0))) - eps_y, 
              np.max(np.append(pl_0/np.max(pl_0),pl_1/np.max(pl_0))) + eps_y))
    plt.minorticks_on()

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pl-no-prep-contrast-b-{int(b_pls)}-omr-{int(100*Om_r)}.png",bbox_inches='tight',
                dpi=400)
    plt.show()


## PL HF Pure v Prepared

In [ ]:
pure=[]
for b_pl in [100,510]:
    bs=B0(b_pl,0.0,0.0)
    Om_r = 15.7
    W_p = 2
    om=D_gs-mu_e*bs[2]
    c = 0
    print("Starting calculations, this may take a while...")
    for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
        init_state = n1&IdN15/2 
        if dt_mw==0.5*np.pi/Om_r:
            init_state = ((n1 + n2 + ground[1]*ground[2].dag() + ground[2]*ground[1].dag())/2)&IdN15/2 #type:ignore
        if dt_mw==np.pi/Om_r:   
            init_state = n2&IdN15/2

        # Array to collect the result Class
        results_t = np.array([])

        # Initial time of the evolution
        ti = 0.0

        # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
        tis_t = []
        tfs_t = []

        # Set the number of times to repeat the Sequence
        n = 1
        # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
        dt_laser = 10.0
        dt_free = 1.0
        # This loop is defined to reproduce the Sequence of Magalleti el al
        for i in tqdm(range(n),position=1,leave=False):
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_doh_hf(
                dt_laser,
                init_state,
                b=bs, 
                om_r=Om_r,
                om=om,
                w_p=W_p,
                ti=ti,
                mode="Laser",
                progress_bar="ON",
                i=i,
            ) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the Sequence
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
        times_t_pure = np.concatenate([res.times for res in results_t])
        N = [n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
                n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
                n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
                n4&IdN15,
                n5&IdN15,
                n6&IdN15]
        n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
        ## Gather the expectation values from the results array
        #for j in range(len(results_t[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(results_t)):
        #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
        #    if j == 0:
        #        n_exp_t = np.array([nn_exp])
        #    else:
        #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        # Gather the times for the plots
        #times_t_pure = np.array([])
        #for i in range(len(results_t)):
        #    times_t_pure = np.append(times_t_pure, results_t[i].times)
        pl=np.array([])
        pl=np.add(n_exp_t[0],n_exp_t[1])
        pl=np.add(pl,n_exp_t[2])
        pl=np.add(pl,n_exp_t[3])
        pl=np.add(pl,n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        pl1=np.array([])
        pl1=np.add(n_exp_t[6],n_exp_t[7])
        pl1=np.add(pl1,n_exp_t[8])
        if c==0:
            pls1_pure = np.array([pl1])
            pls_pure = np.array([pl])
            c+=1
        else:
            pls_pure = np.append(pls_pure,[pl],axis=0)
            pls1_pure = np.append(pls1_pure,[pl1],axis=0)
    print(f"Calculations are finished!! See the plots below")
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls_pure:
        #plt.plot(times_t_pure,pl, color=colors[a],ls="--", label=f"PL no trace init state=${state[a]}$")
        a+=1
    for pl in pls1_pure:    
        plt.plot(times_t_pure,pl, color=colors[a-3],lw=5,label=f"PL init state=${state[a-3]}$")
        a+=1

    eps_y = abs(np.min(pls1_pure) - np.max(pls1_pure))*5e-2
    eps_x = abs(np.min(times_t_pure) - np.max(times_t_pure))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"Pure states PL for B={b_pl:.2f} G, $\\Omega_r$={Om_r:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(pls1_pure) - eps_y, np.max(pls1_pure) + eps_y))
    plt.minorticks_on()

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    pure.append((b_pl,pls1_pure,times_t_pure))

In [ ]:
prep=[]
for b_pl in [100,510]:
    bs=B0(b_pl,0.0,0.0)
    Om_r = 15.7
    W_p = 2
    om=D_gs-mu_e*bs[2]
    c = 0
    print("Starting calculations, this may take a while...")
    for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
        init_state = ((n1+n2+n3)&IdN15)/6

        # Array to collect the result Class
        results_t = np.array([])

        # Initial time of the evolution
        ti = 0.0

        # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
        tis_t = []
        tfs_t = []

        # Set the number of times to repeat the Sequence
        n = 1
        # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
        dt_laser = 10.0
        dt_free = 1.0
        # This loop is defined to reproduce the Sequence of Magalleti el al
        for i in tqdm(range(n),position=1,leave=False):
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_doh_hf(dt_laser,
                                         init_state,
                                         b=bs, 
                                         om_r=Om_r,
                                         om=om,
                                         w_p=W_p,
                                         ti=ti,
                                         mode="Laser",
                                         progress_bar="ON",
                                         i=i) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the Sequence
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_doh_hf(dt_free,
                                         init_state,
                                         b=bs, 
                                         om_r=Om_r,
                                         om=om,
                                         w_p=W_p,
                                         ti=ti,
                                         mode="Free",
                                         progress_bar="ON",
                                         i=i) # type:ignore
            # Store the Result class for MW on evolution
            results_t = np.append(results_t, result) # type: ignore 
            # Save the times for coloring areas of the plot
            tis_t.append(ti)
            tfs_t.append(tf)
            # Make initial time to be the end time of the last part of the Sequence
            ti = tf
            # Make the last state  of this evolution step be the initial states of the next iteration
            init_state = result.states[-1]
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_doh_hf(dt_mw,
                                         init_state,
                                         b=bs, 
                                         om_r=Om_r,
                                         om=om,
                                         w_p=W_p,
                                         ti=ti,
                                         mode="MW",
                                         progress_bar="ON",
                                         i=i) # type:ignore
            # Store the Result class for MW on evolution
            results_t = np.append(results_t, result) # type: ignore
            # Save the times for coloring areas of the plot
            tis_t.append(ti)
            tfs_t.append(tf)
            # Make initial time to be the end time of the last part of the Sequence
            ti = tf
            # Make the last state  of this evolution step be the initial states of the next iteration
            init_state = result.states[-1]
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = dynamics_doh_hf(dt_laser,
                                         init_state,
                                         b=bs, 
                                         om_r=Om_r,
                                         om=om,
                                         w_p=W_p,
                                         ti=ti,
                                         mode="Laser",
                                         progress_bar="ON",
                                         i=i+1) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the Sequence
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
        times_t = np.concatenate([res.times for res in results_t])
        N = [n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
                n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
                n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
                n4&IdN15,
                n5&IdN15,
                n6&IdN15]
        n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
        # Gather the expectation values from the results array
        #for j in range(len(results_t[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(results_t)):
        #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
        #    if j == 0:
        #        n_exp_t = np.array([nn_exp])
        #    else:
        #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        ## Gather the times for the plots
        #times_t = np.array([])
        #for i in range(len(results_t)):
        #    times_t = np.append(times_t, results_t[i].times)
        pl=np.array([])
        pl=np.add(n_exp_t[0],n_exp_t[1])
        pl=np.add(pl,n_exp_t[2])
        pl=np.add(pl,n_exp_t[3])
        pl=np.add(pl,n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        pl1=np.array([])
        pl1=np.add(n_exp_t[6],n_exp_t[7])
        pl1=np.add(pl1,n_exp_t[8])
        if c==0:
            pls1 = np.array([pl1])
            pls = np.array([pl])
            c+=1
        else:
            pls = np.append(pls,[pl],axis=0)
            pls1 = np.append(pls1,[pl1],axis=0)
    print(f"Calculations are finished!! See the plots below")
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls:
        #plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a], ls='--', label=f"PL no trace init state=${state[a]}$")
        a+=1
    for pl in pls1:    
        plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a-3], lw=5,label=f"PL init state=${state[a-3]}$")
        a+=1

    eps_y = abs(np.min(pls1) - np.max(pls1))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"B={b_pl:.2f} G, $\\Omega_r$={Om_r:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
    plt.xlim(t_lim)
    plt.ylim((np.min(pls1) - eps_y, np.max(pls1) + eps_y))
    plt.minorticks_on()

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()
    prep.append((b_pl,pls1,times_t))

In [ ]:
for (b_pls,pls1_pure,times_t_pure),(b_pl,pls1,times_t) in zip(pure,prep):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls1_pure:
        plt.plot(times_t_pure,pl, ls='--', lw=5, color=colors[a], label=f"init Pure state=${state[a]}$")
        a+=1
    for pl in pls1:
        plt.plot(times_t[-5200:]-11.2,pl[-5200:], lw=5, color=colors[a-3], label=f"init state=${state[a-3]}$")
        a+=1
    eps_y = abs(np.min(pls1_pure) - np.max(pls1_pure))*5e-2
    eps_x = abs(np.min(times_t[-5200:]) - np.max(times_t[-5200:]))*5e-2
    plt.ylabel(f"PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"B={b_pls:.2f} G, $\\Omega_r$={Om_r:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(pls1_pure) - eps_y, np.max(pls1_pure) + eps_y))
    plt.minorticks_on()
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pl-hf-pure-v-prep-b-{int(b_pls)}-omr-{int(100*Om_r)}.png",bbox_inches='tight',
                dpi=400)
    plt.show()
    a=0
    print(f"B={b_pls:.2f} G, $\\Omega_r$={Om_r:.2f} MHz")
    for pl_p,pl_prep in zip(pls1_pure,pls1):
        summary = compare_regions(pl_p, pl_prep, [(-5000,-2500)], [state[a]])
        if a==0:
            pl_0 = pl_prep
        if a==2:
            pl_1 = pl_prep
        a+=1
        print_summary_table(summary)
    plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'|1\rangle']
    for pl in [pl_0,pl_1]:
        plt.plot(times_t[-5200:]-11.2,pl[-5200:]/np.max(pl_0), lw=5, color=colors[a], label=f"init state=${state[a]}$")
        a+=1
    plt.plot(times_t[-5200:]-11.2,pl_0[-5200:]/np.max(pl_0)-pl_1[-5200:]/np.max(pl_0), 
             lw=5, color=colors[-1], label=f"Contrast")
    print(f"Max contrast(%): {100*np.max(pl_0[-5200:]/np.max(pl_0)-pl_1[-5200:]/np.max(pl_0))}")
    eps_y = abs(np.min(np.append(pl_0/np.max(pl_0),pl_1/np.max(pl_0))) - np.max(np.append(pl_0/np.max(pl_0),
                                                                                          pl_1/np.max(pl_0))))*5e-2
    eps_x = abs(np.min(times_t[-5200:]) - np.max(times_t[-5200:]))*5e-2
    plt.ylabel(f"%PL (a.u.)",fontsize=25)
    plt.xlabel(r"Time ($\mu$s)",fontsize=25)
    plt.title(f"B={b_pls:.2f} G, $\\Omega_r$={Om_r:.2f} MHz",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(pl_0/np.max(pl_0),pl_1/np.max(pl_0))) - eps_y, 
              np.max(np.append(pl_0/np.max(pl_0),pl_1/np.max(pl_0))) + eps_y))
    plt.minorticks_on()

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.savefig(f"Figs\\pl-hf-prep-contrast-b-{int(b_pls)}-omr-{int(100*Om_r)}.png",bbox_inches='tight',dpi=400)
    plt.show()


# Population Magaletti

In [ ]:
W_p = 1.9
# I had to modify this two value to repruduce the results of the paper
Om_r = 15.7
omega = 0.1

In [ ]:
def plot_continuous(n_exp,times,tis,tfs):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i])
    plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1])

    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps = 5*1e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(n_exp) - eps, np.max(n_exp) + eps],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.75 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.75 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
            else:
                plt.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.75 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)

    # set the time limits you want to show in the plot
    t_lim = (-5 * eps, tfs[-1] + 5 * eps)
    plt.xlim(t_lim)
    if tfs[-1]<30:
        plt.xticks(np.arange(0,tfs[-1]+1,1))
    plt.minorticks_on()
    plt.ylim((np.min(n_exp) - eps, np.max(n_exp) + eps))
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

In [ ]:
def plot_continuous_5_rep_v_1(n_exp,times,tis,tfs,n_reps):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    # Defines figure size
    fig,axs=plt.subplots(2,1,figsize=(14, 10),gridspec_kw={'height_ratios': [1, 2]})
    # Plot the population of each state
    for i in range(len(n_exp) - 1):
        axs[0].plot(times, n_exp[i], color=colors[i],lw=3)
    axs[0].plot(times, n_exp[-1], color=colors[-1],lw=3)
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    eps = 5*1e-2
    # set the time limit and n_is for the bottom plot 
    n_exp_1,times_1=n_exp[:,:len(times)//n_reps+1000], times[:len(times)//n_reps+1000]
    t_lim_1 = (-5 * eps, times_1[-1] + 5 * eps)
    for ax in axs:
        n = 0
        for i in zip(tis, tfs):
            if n % 3 == 0:
                ax.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    t_lim_1[1]-5*eps if ax==axs[1] and n==3 else i[1], # set the max range of 2nd green colored region of bottom plot
                    color="palegreen",
                    alpha=0.3+0.55 ** (int(n / 3) + 1),
                    label=f"Laser ON" if ax==axs[0] and n<3 else "",
                ) 
            else:
                if (n - 1) % 3 == 0:
                    ax.fill_betweenx(
                        [np.min(n_exp) - eps, np.max(n_exp) + eps],
                        i[0],
                        i[1],
                        color="lightblue",
                        alpha=0.3+0.55 ** (int(n / 3) + 1),
                        label=f"MS depl" if ax==axs[0] and n<3 else "",
                    )
                else:
                    ax.fill_betweenx(
                        [np.min(n_exp) - eps, np.max(n_exp) + eps],
                        i[0],
                        i[1],
                        color="orchid",
                        alpha=0.3+0.55 ** (int(n / 3) + 1),
                        label=f"MW ON" if ax==axs[0] and n<3 else "",
                    )
            n += 1
    fig.text(0.02,0.5,f"Population",va='center',rotation='vertical',fontsize=25)
    axs[1].set_xlabel(r"Time ($\mu$s)",fontsize=25)

    # set the time limits you want to show in the plot
    t_lim = (-0.5-5 * eps, tfs[-1] + 5 * eps+1)
    axs[0].set_xlim(t_lim)
    for ax in axs:
        ax.minorticks_on()
        ax.set_ylim((np.min(n_exp) - eps, np.max(n_exp) + eps))
    
    axs[1].set_xticks(np.arange(0,t_lim[1],1))
    axs[1].set_xlim(t_lim_1)
    for i in range(len(n_exp) - 1):
        axs[1].plot(times_1, n_exp_1[i], label=f"$n_{i+1}$", color=colors[i], lw=5)
    axs[1].plot(times_1, n_exp_1[-1],label=f"$n_c$", color=colors[-1], lw=5)
    axs[1].legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    axs[0].legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()

In [ ]:
# Initial state where the ground state is evenly populated
init_state = 1 / 3 * (n1 + n2 + n3)

# Array to collect the result Class
results = np.array([])

# Initial time of the evolution
ti = 0.0

# Store the times of activation and deativation of laser and microwaves for region highlight in plotting
tis_mg = []
tfs_mg = []

# Set the number of times to repeat the protocol
n_reps = 5
# Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
dt_laser = 10.0
dt_free = 1.0
dt_mw = 1.0
print("Starting calculations, this may take a while...")
# This loop is defined to reproduce the protocol of Magalleti el al
for i in tqdm(range(n_reps)):
    tis_mg.append(ti)
    # Here Laser is ON
    # Run the dynamics based on the mode choosen
    tf, result = dynamics_mg(
        dt_laser,
        init_state,
        ti=ti,
        w_p=W_p,
        om=omega,
        om_r=Om_r,
        mode="Laser",
        i=i
    ) # type: ignore
    # Store the Result class for when the Laser is on
    results = np.append(results, result) # type: ignore
    # Save the initial state for the next part of the protocol
    init_state = result.states[-1]
    # Make the initial time the end time of the previous part of the procotol
    ti = tf
    tfs_mg.append(tf)
    # Run the dynamics based on the mode choosen
    tf, result = dynamics_mg(
        dt_free,
        init_state,
        ti=ti,
        w_p=W_p,
        om=omega,
        om_r=Om_r,
        mode="Free",
        i=i
    ) # type: ignore
    # Store the Result class for MW on evolution
    results = np.append(results, result) # type: ignore
    # Save the times for coloring areas of the plot
    tis_mg.append(ti)
    tfs_mg.append(tf)
    # Make initial time to be the end time of the last part of the protocol
    ti = tf
    # Make the last state  of this evolution step be the initial states of the next iteration
    init_state = result.states[-1]
    # Run the dynamics_mg based on the mode choosen
    tf, result = dynamics_mg(dt_mw, 
                             init_state, 
                             ti=ti,
                             w_p=W_p,
                             om=omega,
                             om_r=Om_r, 
                             mode="MW", 
                             i=i) # type: ignore
    # Store the Result class for MW on evolution
    results = np.append(results, result) # type: ignore
    # Save the times for coloring areas of the plot
    tis_mg.append(ti)
    tfs_mg.append(tf)
    # Make initial time to be the end time of the last part of the protocol
    ti = tf
    # Make the last state  of this evolution step be the initial states of the next iteration
    init_state = result.states[-1]
# Gather the expectation values from the results array
times_mg = np.concatenate([res.times for res in results])
N = [n1, n2, n3, n4, n5, n6, n7, nc]
n_exp_mg = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in N])
S = [sx_gs, sy_gs, sz_gs, sx_es, sy_es, sz_es]
s_exp_mg = np.array([np.concatenate([qt.expect(M, res.states)  for res in results]) for M in S])

print("Calculations are finished!! Go to the next block for plots")

In [ ]:
plot_continuous_5_rep_v_1(n_exp_mg, times_mg, tis_mg, tfs_mg,n_reps)

In [ ]:
def continuous_mg(laser_time, mw_time, free_time,om_r=Om_r,
    w_p=W_p,om=omega,protocol_repeats=1,progress_bar="ON"):
    """
    Perform continuous calculations for a multi-level quantum system.
    Args:
        laser_time (float): Time interval for laser evolution.
        mw_time (float): Time interval for microwave evolution.
        free_time (float): Time interval for free evolution.
        om_r (float): Rabi frequency for the system (default: Om_r).
        w_p (float): Transition frequency for the system (default: W_p).
        args (dict): Additional arguments for the system (default: {"w": omega}).
        protocol_repeats (int): Number of times to repeat the protocol (default: 1).
        progress_bar (str): Option to display progress bar ("ON" or "OFF") (default: "ON").
    Returns:
        tuple: A tuple containing the following elements:
            - n_exp (ndarray): Expectation values for the system.
            - times (ndarray): Time points for the evolution.
            - tis (list): List of activation times for laser and microwaves.
            - tfs (list): List of deactivation times for laser and microwaves.
            - results (ndarray): Array of Result class instances.
    Raises:
        None.
    """      
    # Initial state where the ground state is evenly populated
    init_state = 1 / 3 * (n1 + n2 + n3)

    # Array to collect the result Class
    results = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    n = protocol_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same protocol as a pulsed protocol, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_laser,
                                 init_state,
                                 ti=ti,
                                 mode="Laser",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=i) # type:ignore
        # Store the Result class for when the Laser is on
        results = np.append(results, result) # type: ignore
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_free,
                                 init_state,
                                 ti=ti,
                                 mode="Free",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg(dt_mw,
                                 init_state,
                                 ti=ti,
                                 mode="MW",
                                 om_r=om_r,
                                 w_p=w_p,
                                 om=om,
                                 progress_bar=progress_bar,
                                 i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    times = np.concatenate([res.times for res in results])
    N = [n1, n2, n3, n4, n5, n6, n7, nc]
    n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in N])
    S = [sx_gs, sy_gs, sz_gs, sx_es, sy_es, sz_es]
    s_exp = np.array([np.concatenate([qt.expect(M, res.states)  for res in results]) for M in S])
    
    print("Calculations are finished!! param order(Expect,times,tis,tfs,results)")
    return n_exp, times, tis, tfs, results,s_exp

In [ ]:
def continuous_mg_hf(laser_time, mw_time, free_time,om_r=Om_r,
                     w_p=W_p,om = omega,protocol_repeats=1,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given protocol with continuos laser.

    Args:
        laser_time (float): The duration of the laser pulse in each iteration.
        mw_time (float): The duration of the microwave pulse in each iteration.
        free_time (float): The duration of the free evolution in each iteration.
        protocol_repeats (int, optional): The number of times to repeat the protocol. Defaults to 1.

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): An array of expectation values for different operators.
            - times (numpy.ndarray): An array of time values.
            - tis (numpy.ndarray): An array of initial times for each part of the protocol.
            - tfs (numpy.ndarray): An array of final times for each part of the protocol.
            - results (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    # Initial state where the ground state is evenly populated
    init_state = ((n1 + n2 + n3)&IdN15)/6

    # Array to collect the result Class
    results_hf = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the protocol
    n = protocol_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same protocol as a pulsed protocol, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg_hf(dt_laser,
                                    init_state,
                                    ti=ti,
                                    mode="Laser",
                                    om_r=om_r,
                                    w_p=w_p,
                                    om = om,
                                    progress_bar=progress_bar,
                                    i=i) # type:ignore
        # Store the Result class for when the Laser is on
        results_hf = np.append(results_hf, result) # type: ignore
        # Save the initial state for the next part of the protocol
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg_hf(dt_free,
                                    init_state,
                                    ti=ti,
                                    mode="Free",
                                    om_r=om_r,
                                    w_p=w_p,
                                    om=om,
                                    progress_bar=progress_bar,
                                    i=i) # type:ignore
        # Store the Result class for MW on evolution
        results_hf = np.append(results_hf, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_mg_hf(dt_mw,
                                    init_state,
                                    ti=ti,
                                    mode="MW",
                                    om_r=om_r,
                                    w_p=w_p,
                                    om=om,
                                    progress_bar=progress_bar,
                                    i=i) # type:ignore
        # Store the Result class for MW on evolution
        results_hf = np.append(results_hf, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the protocol
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    # Gather the expectation values from the results array
    times_hf = np.concatenate([res.times for res in results_hf])
    N = [n1&IdN15, n2&IdN15, n3&IdN15,
         n4&IdN15, n5&IdN15, n6&IdN15,
         n7&IdN15, nc&IdN15]
    n_exp_hf = np.array([np.concatenate([qt.expect(M, res.states) for res in results_hf]) for M in N])
    S = [sx_gs&IdN15, sy_gs&IdN15, sz_gs&IdN15,
         sx_es&IdN15, sy_es&IdN15, sz_es&IdN15]
    s_exp_hf = np.array([np.concatenate([qt.expect(M, res.states)  for res in results_hf]) for M in S])
    Nit = [IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag()] #type: ignore
    nit_exp_hf = np.array([np.concatenate([qt.expect(M, res.states)  for res in results_hf]) for M in Nit])
    
    print("Calculations are finished!! param order(n_exp,times,tis,tfs,s_exp,nit_exp)")
    return n_exp_hf, times_hf, tis, tfs, s_exp_hf, nit_exp_hf

In [ ]:
def plot_cont_hf_vs_cont(n_exp_hf,times_hf,tis_hf,tfs_hf,n_exp,times,tis,tfs):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "darkgoldenrod",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    hf="{hf}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    for i in range(len(n_exp_hf) - 1):
        plt.plot(times_hf, n_exp_hf[i], label=f"$n^{hf}_{i+1}$", color=colors[i])
    plt.plot(times_hf, n_exp_hf[-1], label="$n^{hf}_c$", color=colors[-1])
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i],ls='--')
    plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1],ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps = 5*1e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(n_exp) - eps, np.max(n_exp) + eps],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(n_exp) - eps, np.max(n_exp) + eps],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=25)
    plt.xlabel(r"Time($\mu$s)",fontsize=25)
    plt.title(f"Comparisson between Hyperfine and No-HF evolution\n Laser:{t_laser:.2f} $\\mu$s, MW:{t_mw:.2f} $\\mu$s, Free:{t_free:.2f} $\\mu$s",fontsize=25)
    # set the time limits you want to show in the plot
    t_lim = (-5 * eps, tfs[-1] + 5 * eps)
    plt.xlim(t_lim)
    plt.ylim((np.min(n_exp) - eps, np.max(n_exp) + eps))
    plt.minorticks_on()
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=20)
    plt.show()

In [ ]:
cont_hf=continuous_mg_hf(10.0, 4.5, 1.0)
cont=continuous_mg(10.0,4.5,1.0)

In [ ]:
plot_cont_hf_vs_cont(cont_hf[0],cont_hf[1],cont_hf[2],cont_hf[3],cont[0],cont[1],cont[2],cont[3])

# End